<a href="https://colab.research.google.com/github/frankausberlin/notebook-collection/blob/main/snippetpearls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table width=1000><tr></tr><tr><td align='center'><font size='+10' color='#005F6A'><b>
SnippetPearls</b></font></td></tr><tr></tr><tr><td align='center'><p>
This is a jupyter notebook with several and I hope useful<br> code snippets around very interesting technologies.</td></tr><tr></tr><tr><td align='center'>
optimized for google colab</td></tr></table><br><br>

---

<br>
<font size='+2'>Installation<br>

It's realy easy:
1. Copy the url of your (or this) notebook,
2. go to menu tools / settings / websites
3. and past your url to field 'Custom snippet notebook url'.

It will now, after reloading the page, treat all text cells with chapter entries ('#') and their subsequent code cells as a snippet and display them in the colab snippet selector


#_sp: ToEnglish

In [ ]:
""" Colab Code-Snippets
It uses colab forms to make a top-level setting of the code snippet (optional) and give
a little description. You can run the cell to execute the code snippet.
"""
#@markdown <font size='+2' color='#005F6A'>**ToEnglish**</font>
#@markdown <br>A little **English translator** as **class** with optional **gui** - uses the **PyPi** lib [**'translators'**](https://pypi.org/project/translators/)
#@markdown * After run that cell you can use the mini GUI to translate.
#@markdown * You can also access the class object using the variable 'te'.<br>
#@markdown > ```te.translateToEnglish   ('hallo aus berlin und einen schönen tag')```<br>
#@markdown > ```te.translateFromEnglish ('hello from berlin and a beautiful day')```<br>
#@markdown * Use **```help (ToEnglish)```** to get more detailed informations
#@markdown * It is not necessary to use the language specified in **country_code** - automatic detection is always performed, so you can use other languages ​​as well. The country_code parameter is **only used** in the **translateFromEnglish** function as the target language.
from ipywidgets import  HTML, VBox, Textarea, Button, Layout, Dropdown, HBox, Label, Accordion

try:
  import translators as ts
except:
  !pip install translators

import translators as ts

# *******************************************************************************************************************************************************************************************
# *** class *********************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
class ToEnglish:
  """
  A little class/gui to translate text to english

  Attributes
  ----------
  * translator : str | default: 'bing'
    the translator to use (eg. 'google', 'bing'...)
  * country_code : str | default: None
    the two letter country code to use as destination language in the translateFromEnglish function
  * hasGUI : Boolean | default: True
    if True (default) a gui is built

  Methods
  -------
  * translateToEnglish (txt)
    translates txt to english
  * translateFromEnglish (txt)
    translates txt from english

  Behavior
  --------
  * If you instantiate the class with 'hasGUI' is true the gui is built. There you can translate from/to english.
  * With 'hasGUI' is false it just delivers a 'translator'-object with the methods.
  * If you use one of the translate methods it updates the gui if 'hasGUI' is true.
  * If no country code given to the constructor it uses the Colab forms parameter.

  Use cases
  ---------
  * The simplest way to use it is to create an Instance to show the gui
    >>>ToEnglish () # or >>>te = ToEnglish ()
  * start gui with google as default translator (selected in list)
    >>>ToEnglish ('google') # or >>>ToEnglish (translator='google')
  * start with an alternative foreign language (eg. french)
    >>>ToEnglish (country_code='fr') # or >>>ToEnglish ('bing', 'fr')
  * using the object - note that the given text is also inserted in the gui if available
    >>>te = ToEnglish ()
    >>>print ('  to:', te.translateToEnglish   ('hallo aus berlin und einen schönen tag'))
    >>>print ('from:', te.translateFromEnglish ('hello from berlin and a beautiful day'))
  """


  # *******************
  # *** Colab Forms ***
  # *******************
  country_code = 'de' #@param {type:"string"}

  def __init__(self, translator='bing', country_code=None, hasGUI=True):
    """ The constructor check and store the parameters and build the gui.
    """
    # attributes
    self.translator, self.hasGUI = translator, hasGUI

    # check country code valid
    self.country_code = ToEnglish.country_code if not country_code else country_code
    if not self.country_code.lower() in ts.get_languages()['en']:
      raise Exception (f'\x1b[106m"{self.country_code}" is no valid country code')

    # build gui
    if hasGUI:
      # widgets
      self.dd_translators      = Dropdown  (description='Translators', options=ts.translators_pool, layout=Layout(width='auto'),value=translator)
      self.bu_translateTo      = Button    (description='translate to english',
                                            layout=Layout(width='auto'),
                                            style={'button_color':'powderblue'})
      self.bu_translateFrom    = Button    (description='translate from english',
                                            layout=Layout(width='auto'),
                                            style={'button_color':'powderblue'})
      self.ta_textForeign      = Textarea  (description=self.country_code.lower(), layout=Layout(width='auto',height='200px'), value='')
      self.ta_textEnglish      = Textarea  (description='en', layout=Layout(width='auto',height='200px'), value='')
      self.hb_controlPanel     = HBox      (children=[self.dd_translators, self.bu_translateTo, self.bu_translateFrom])
      self.vb_editor           = VBox      (children=[self.hb_controlPanel, self.ta_textForeign, self.ta_textEnglish])
      self.ac_trala            = Accordion (children=[self.vb_editor])
      # events
      self.bu_translateTo.on_click   (lambda b: self.translateToEnglish   (self.ta_textForeign.value))
      self.bu_translateFrom.on_click (lambda b: self.translateFromEnglish (self.ta_textEnglish.value))
      # show
      display (self.ac_trala)

  def translateToEnglish (self, txt):
    """ Translates txt to english
    """
    if self.hasGUI:
      self.translator = self.dd_translators.value
    trans = ts.translate_text (query_text   = txt,
                               translator   = self.translator,
                               to_language  = 'en')
    if self.hasGUI:
      self.ta_textForeign.value = txt
      self.ta_textEnglish.value = trans
    return trans

  def translateFromEnglish (self, txt):
    """ Translates txt from english
    """
    if self.hasGUI:
      self.translator = self.dd_translators.value
    trans = ts.translate_text (query_text   = txt,
                               translator   = self.translator,
                               to_language  = self.country_code)
    if self.hasGUI:
      self.ta_textEnglish.value = txt
      self.ta_textForeign.value = trans
    return trans

te = ToEnglish () #; help (ToEnglish)


#_sp: gpt4all


In [ ]:
# *******************************************************************************************************************************************************************************************
# *** Colab Forms ***************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************

#@markdown <font size='+2' color='#005F6A'>**gpt4all**</font><br>
#@markdown A little client for the [gpt4all](https://gpt4all.io/) models.
#@markdown * You can access your **gpu device** via **'cuda:'** or **'kompute:'**. (With my hardware 'cuda:' always **crashes** but 'kompute:' works fine.)
#@markdown * Models will be **downloaded at first usage** (~ **4-8 GB** per model) - your gpu should have **at least 8 GB**.
#@markdown * **No-English-Ubuntus**: if you install the [gpt4all-chat client](https://github.com/nomic-ai/gpt4all) be aware to have an **folder 'Desktop'** in your home - it trys to copy the .desktop file there
#@markdown * The **models** are **located** (ubuntu) in:<br>- ```~/.local/share/nomic.ai/GPT4All``` if gpt4all-chat client is installed<br>- ```~/.cache/gpt4all``` else<br>- ```/content``` in hosted runtime by colab (working folder)

default_prompt = "Here is a classic math text problem: Uwe and Peter are talking. Uwe says: \"If you give me an apple from yours, then we will both have the same number of apples.\" Then Peter says, \"But if you give me one of yours, I'll have twice as many as you.\" How many apples does each of them have?" #@param {type:"string"}

import os, time, platform

from ipywidgets import VBox, HBox, Dropdown, Label, BoundedIntText, BoundedFloatText, Checkbox, Text, Button, Textarea, HTML, Layout, RadioButtons, HTML
try:     from gpt4all import GPT4All
except:  os.system("pip install gpt4all")

from gpt4all import GPT4All

# check if chat client is installed in system to use its model folder else current folder
if not os.path.isdir (f"{os.path.expanduser('~')}/.local/share/nomic.ai/GPT4All"):  local_model_folder = f'{os.getcwd()}'
else: local_model_folder = f"{os.path.expanduser('~')}/.local/share/nomic.ai/GPT4All"

# devices German
devices = []
try: devices += GPT4All.list_gpus()
except: pass
if len (devices): devices = devices + [f'cpu:{platform.processor()}']
else:             devices = [f'cpu:{platform.processor()}']

sideBySide = """<style> .widget-radio-box {flex-direction: row !important;} .widget-radio-box label{margin:5px !important; width: 400px !important;}</style>"""
sideBySide = sideBySide.replace ('400px', f'{1200//len(devices)}px')
display (HTML (sideBySide))

rb_devices = RadioButtons(indent=False, options=devices, description="devices")

hb_params = HBox (children=[
  Button (description='clear', layout=Layout(width='60px',height='60px'), style={'button_color':'powderblue'}),# 0
  VBox (layout=Layout(width='200px'), children=[ #1
    Button   (description='Model', layout=Layout(width='auto'), tooltip='Name of the model'),
    Dropdown (options=[m['name'] for m in GPT4All.list_models()], layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='100px'), children=[#2
    Button(description='max_tokens', layout=Layout(width='auto'), tooltip='The maximum number of tokens to generate.'),
    BoundedIntText (value=2000, layout=Layout(width='auto'), max=20000)],),
  VBox (layout=Layout(width='100px'), children=[#3
    Button(description='temperatur', layout=Layout(width='auto'), tooltip='The model temperature. Larger values increase creativity but decrease factuality.'),
    BoundedFloatText  (value=0.7, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='80px'), children=[#4
    Button(description='top_k', layout=Layout(width='auto'), tooltip='Randomly sample from the top_k most likely tokens at each generation step. Set this to 1 for greedy decoding.'),
    BoundedIntText (value=40, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='80px'), children=[#5
    Button(description='top_p', layout=Layout(width='auto'), tooltip='Randomly sample at each generation step from the top most likely tokens whose probabilities add up to top_p'),
    BoundedFloatText (value=0.4, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='80px'), children=[#6
    Button(description='min_p', layout=Layout(width='auto'), tooltip='Randomly sample at each generation step from the top most likely tokens whose probabilities are at least min_p.'),
    BoundedFloatText (value=0.0, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='120px'), children=[#7
    Button(description='repeat_penalty', layout=Layout(width='auto'), tooltip='Penalize the model for repetition. Higher values result in less repetition.'),
    BoundedFloatText (value=1.18, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='120px'), children=[#8
    Button(description='repeat_last_n', layout=Layout(width='auto'), tooltip='How far in the models generation history to apply the repeat penalty.'),
    BoundedIntText (value=64, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='80px'), children=[#9
    Button(description='n_batch', layout=Layout(width='auto'), tooltip='Number of prompt tokens processed in parallel. Larger values decrease latency but increase resource requirements.'),
    BoundedIntText (value=8, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='100px'), children=[#10
    Button(description='streaming', layout=Layout(width='auto'), tooltip='If True, this method will instead return a generator that yields tokens as the model generates them.'),
    Checkbox (description='activate', value=True, indent=False, layout=Layout(width='auto'))]),
  VBox (layout=Layout(width='160px'), children=[#11
    Button(description='translate to', layout=Layout(width='auto'), style={'button_color':'powderblue'}, tooltip='Translate the prompts to the language below'),
    Text (value='German', layout=Layout(width='auto'))])
])

hb_prompt = HBox (children=[
  Textarea (description='System', value='Conversation between a curious user and an AI assistant.\n', layout=Layout(width='800px', height='60px')),
  Textarea (description='Template', value='USER: {0}\nASSISTANT: ', layout=Layout(width='500px', height='60px'))
])

control = VBox (children=[hb_params, HTML(value='<hr>'), rb_devices, hb_prompt],
                layout=Layout(width='1350px', border='solid 2px'))
chat    = VBox (children=[
  Textarea (description='Prompt',layout=Layout(width='1300px', height='200px'),value=default_prompt),
  HBox(children=[Button (description='Send', style={'button_color':'lightgreen'}),
                 HTML (value=f'<p align="right">using {local_model_folder} as model folder',layout=Layout(width='550px'))]),
  Textarea (description='Response',layout=Layout(width='1300px', height='400px'))
])
filenameForSelected = lambda:[m['filename'] for m in GPT4All.list_models()][control.children[0].children[1].children[1].options.index(control.children[0].children[1].children[1].value)]

def clickSendButton (b):
  chat.children[1].children[1].value = """<marquee style='width: 100%; color: blue;'>waiting for response...</marquee>"""

  gpt4allModel = GPT4All (
                          model_name      = filenameForSelected(),
                          model_path      = local_model_folder,
                          allow_download  = True,
                          device          = rb_devices.value.split(':')[0]
                         )
  def _generate (streaming):
    return gpt4allModel.generate (
                                  chat.children[0].value,
                                  max_tokens              = hb_params.children[2].children[1].value,
                                  temp                    = hb_params.children[3].children[1].value,
                                  top_k                   = hb_params.children[4].children[1].value,
                                  top_p                   = hb_params.children[5].children[1].value,
                                  min_p                   = hb_params.children[6].children[1].value,
                                  repeat_penalty          = hb_params.children[7].children[1].value,
                                  repeat_last_n           = hb_params.children[8].children[1].value,
                                  n_batch                 = hb_params.children[9].children[1].value,
                                  streaming               = streaming
                                )


  with gpt4allModel.chat_session (
                                  system_prompt   = hb_prompt.children[0].value,
                                  prompt_template = hb_prompt.children[1].value
                                 ):
    chat.children[2].value += '\n\n{}\n{} - ({}, {}, {}, {}, {}, {}, {}, {})\n{}\n'.format ('-'*80,
                                control.children[0].children[1].children[1].value,
                                hb_params.children[2].children[1].value,
                                hb_params.children[3].children[1].value,
                                hb_params.children[4].children[1].value,
                                hb_params.children[5].children[1].value,
                                hb_params.children[6].children[1].value,
                                hb_params.children[7].children[1].value,
                                hb_params.children[8].children[1].value,
                                hb_params.children[9].children[1].value, '-'*80)

    if hb_params.children[10].children[1].value:
      for token in _generate (True): chat.children[2].value += token
    else:
      chat.children[2].value += _generate (False)

  chat.children[1].children[1].value = f'<p align="right">using {local_model_folder} as model folder'

def clickClearButton (b):
  chat.children[2].value = ''

chat.children[1].children[0].on_click (clickSendButton)
hb_params.children[0].on_click (clickClearButton)

display (control,chat)



#_sp: lazy student

In [ ]:
#@markdown [<font size='+2' color='#005F6A'>**lazy student**</font>](https://github.com/frankausberlin/lazy-student)<br>
#@markdown This is for the work-optimized student - openai's [prompt engineering](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) (**GPT**) with access to (collections of) video transicriptions, focused on bilingualism and yaml persistence.
#@markdown * Choose your language and (your collection - select a course and) a video - build your chapters - clean it, prompt it, loop it - be creative...
#@markdown * The prompt functions needs an **[openai](https://platform.openai.com/account/usage)** account for send your gpt request direct out of the widget.
#@markdown * It manages collections on your **session**, **gdrive** (~ 100 MB) or **local runtime** - there are some examples (new ones can easily be added):
#@markdown > * ['Practical Deeplearning for Coders'](https://course.fast.ai/) by fastai (**folder dlfc**) and
#@markdown > * ['MIT Introduction to Deep Learning'](http://introtodeeplearning.com/) by MIT (**folder mit**) and
#@markdown > * [A collection by Stanford](https://www.youtube.com/@stanfordonline/playlists?view=50&sort=dd&shelf_id=4): CS221, CS224, CS229, CS330 (**folder stanford**).
#@markdown > * [StatQuest](https://statquest.org/) Collections by the singing guy [Josh Starmer](https://www.youtube.com/@statquest) (**folder statquest**).
#@markdown * A **collection** is a list of several youtube **playlists** (playlists.yml) with its own **folder** for the **created yml** files.
#@markdown * As alternative to the collections there is a **single video** selection mode.
#@markdown * More questions? **Look in the help tab or code** - there are more detailed descriptions.
#@markdown ---

helptext="""Basic concepts:
* Bilangualism
  - The playlists.yml holds the description and the videos id of every playlist.
  - You have language specific (<cc>) yamls:
    - <vid>_<cc>.yml       - the transcription for a video
    - <vid>_loops_<cc>.yml - the loops for a video
  - Inside the Yamls 'prompts' and '<vid>_info.yml' you have have multi
    languages for every prompt / aichapters.
* The yamls created
  - playlists.yml           : Infos and videos list for every playlist
  - prompts.yml             : The prompts - for all languages
  - <vid>_en.yml            : The english transcription
  - <vid>_<cc>.yml          : The <cc> transcription (see country codes below)
  - <vid>_info.yml          : Description, chapters and aichapters for video
  - <vid>_loops_<en|cc>.yml : Prompt loops history for video for a language
* There is a persistence mechanic:
  - Select the correct working folder first:
    - if you use local runtime the working folder is located in your home folder
                      '~/<main_folder>'
    - if you are hosted and you checked 'use_gdrive...' your working folder is
                      '/gdrive/MyDrive/<main_folder>'
    - if you are hosted and you unchecked 'use_gdrive...' your working folder is
                      '/content/<main_folder>'
  - It only creates folders (main folder, collection folders) if not exists.
  - Yamls are only generated when they are needed:
    - playlists.yaml               by first run
    - *_en, *_<cc>, *_info files   by selection of a video
    - prompts.yml                  by save
    - *_loops_<cc>.yml             after doing loop.
  - Simple prompts are not stored.
* Without api-key for openai the features using the api are disabled
  ('send', 'loop', 'build' and 'translate') but the prompt editor works.
* The 'aichapters' prompt is a system prompt and can not be deleted but modified
  with a 70 char limit used for response.
* The textareas does show the token count (the cyan text), the token factor
  ('*'-textfield, editable), the request temperatur ('°'-textfield, editable).
* Select a prompt (green button) and a chapter -> click show-button to create
  the prompt in the textarea and click send to send the textarea (!) as request
  and wait for response.
* Between show and send you have the possibility to edit the prompt.
* The loop mechanic takes the transcript from every chapter as input for the
  selected prompt and show the response in the textarea - be carefull it cannot
  be interupted (check token count in chapters - 16000 token max).
* The api is not very stable - sometimes it hangs (about 1-3 minutes)
* There are 3 attempts for all requests - so mostly the result is given.

Notes UI:
* When you run the cell it builds the widgets for the selected collection / mode.
* You have a accordion tabed gridbox for every plalist and the prompt editor.
* On start it takes first video of first playlist and try to create the chapters
  out of the description.
* It creates the video radio buttons and the buttons to work with video:
    'all >>>>', 'clean', 'from description', 'auto', 'aichaps', 'build'
  and the prompt-buttons
* If no chapters in description you can use the auto button. It takes the number
  (value left) as count for tokens in a chapter and generate a corresponding
  timestamp list for use for chapter buttons.
* There are important infos in tooltips:
  - 'all >>>>'-button: the original description of the video.
  - prompt-buttons: the prompt textes
  - all other buttons: the description what they do.
* There are are log message line below the accordion tab - shows current events.
* With a small **editor** you have the possibility to have a little prompt
  engineering **fun** (note the 4000 token limit).
* When you add a new prompt in the editor it will be shown as green prompt
  button for using with videos.

Notes miscellaneous:
* The current select video / playlist is set in globals: function globalize().
* The request uses the textare text - you can reduce token len when you use
  the clean-button before send.
* The libs pytube, youtubesearchpython, youtube_transcript_api and openai
  (if selected) will be installed.

Widgets structure:

mainAccordion
   |
   + [gb_maingb] - gridbox for every playlist -> set in globalize() to current selection
   |    + vb_vidsel - area left with video selection and buttons
   |    |    + wi_firstw - HTML youtube link / HBox[tx_singvid, bu_singvid]
   |    |    + rb_vidsel - the rbs for video selection
   |    |    + bu_allchp - the 'all >>>>' button
   |    |    + hb_cleanb
   |    |    |    + children0 txt for parameter - no global
   |    |    |    + bu_cleanb - clean button
   |    |    + bu_frodes - from description button
   |    |    + hb_autchp
   |    |    |    + children0 txt for parameter - no global
   |    |    |    + bu_autchp - auto button
   |    |    + hb_aichbu
   |    |    |    + bu_fromai, bu_aichap - buttons 'from ai' and 'aichaps'
   |    |    + [prompt_buttons] - button for every prompt
   |    + vb_chpare - big area with chapter selector and transcriptions
   |         + bx_chpsel - box for chapter selection
   |         |    + childrenN - button for every chapter
   |         + hb_contrl - control areas for en / xx
   |         |    + vb_enCtrl
   |         |    |    + hb_enCtrl
   |         |    |    |    + children0 (HTML) - token count - no global
   |         |    |    |    + children1 (Label) - prompt name - no global
   |         |    |    |    + children2 (Text) - input tf for temperatur - no global
   |         |    |    |    + bu_enShow, bu_enSend, bu_enLoop - buttons
   |         |    |    + hb_enSele
   |         |    |         + children0(Box)
   |         |    |              + bu_enView, bu_enResu - the green and red buttons
   |         |                   + [children2-n] - blue button for every loop
   |         |    |         + children1(Box)
   |         |    |              + bu_enLdel - the loop del button
   |         |    + vb_xxCtrl
   |         |         + hb_xxCtrl
   |         |         |    + children0 (HTML) - token count - no global
   |         |         |    + children1 (Label) - prompt name - no global
   |         |         |    + children2 (Text) - input tf for temperatur - no global
   |         |         |    + bu_xxShow, bu_xxSend, bu_xxLoop - buttons
   |         |         + hb_xxSele
   |         |              + children0(Box)
   |         |                   + bu_xxView, bu_xxResu - the green and red buttons
   |         |                   + [children2-n] - button for every loop
   |         |              + children1(Box)
   |         |                   + bu_xxLdel - the loop del button
   |         + hb_transc - area with transcriptions
   |              + ta_enTran - english transcription
   |              + ta_xxTran - xx transcription
   + gb_proedi - prompt editor tab
   |    + rb_prompt - the prompt radio-buttons
   |    + vb_proedi - parent for the prompt widgets
   |         + hb_probut - the buttons line
   |              + bu_addprmt, bu_delprmt, bu_savprmt, bu_transla
   |         + tx_protit - prompt title
   |         + hb_prompt - prompt text
   + ta_helper - help tab
"""
import os, yaml, getpass, time
try:
  from pytube                   import  Playlist
  from youtubesearchpython      import  Video
  from youtube_transcript_api   import  YouTubeTranscriptApi
except:
  print ('stay tuned - installing stuff')
  os.system ('pip install pytube youtube-search-python youtube-transcript-api')
from   ipywidgets               import  Accordion, VBox, HBox, Box, GridBox, HTML, Tab
from   ipywidgets               import  Textarea, Text, RadioButtons, Button, Layout, Label
from   datetime                 import  datetime
from   pytube                   import  Playlist
from   youtubesearchpython      import  Video
from   youtube_transcript_api   import  YouTubeTranscriptApi

# colab forms
use_gdrive_for_persistence                = True   #@param {type:"boolean"}
country_code_for_the_translation_language = 'de'   #@param {type:"string"}
yes_i_have_this_chatgpt_openai_account    = True   #@param {type:"boolean"}
use_english_for_ai_auto_chapters          = False  #@param {type:"boolean"}
used_model                                = 'gpt-3.5-turbo' #@param ['gpt-3.5-turbo', 'gpt-3.5-instruct', 'gpt-4', 'gpt-4-turbo', 'gpt-4o']


####################################################################################################################################
################################################## defaults / customization ########################################################
####################################################################################################################################
playlist_prefix       = 'https://www.youtube.com/playlist?list='
video_prefix          = 'https://www.youtube.com/watch?v='
main_folder           = 'lazy_collections'
selected_collection   = 'StatQuest!!!-Collections by the singing guy Josh Starmer' #@param ['Single video selection','StatQuest!!!-Collections by the singing guy Josh Starmer','MIT courses around deep learning','Practical Deep Learning for Coders / fast.ai live coding & tutorials', "A collection of free Stanford courses: CS221, CS224, CS229, CS330"]

#
#         ||      your                                   /\
#      \ \||/ /   collection                            /||\
#       \ \/ /    here                                   ||
#        \  /                              (don't forget: add to param list)
#         \/
#
# if 'some unique' in selected_collection:
#   courses = { 'Course 1'  : 'PL******',
#               'Course 2'  : 'PL******',
#               'Course 3'  : 'PL******'}
#   folder_for_playlist_collection = 'yourfolder'

# The collections
if 'StatQuest' in selected_collection:
  courses = { 'Histograms Clearly Explained - #66DaysOfData'                                                : 'PLblh5JKOoLUJUNlfvCNhJMNjNNpt5ljcR',
              'Histograms Clearly Explained - Statistics Fundamentals'                                      : 'PLblh5JKOoLUK0FLuzwntyYI10UQFUhsY9',
              'A Gentle Introduction to Machine Learning'                                                   : 'PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI',
              'Neural Networks / Deep Learning'                                                             : 'PLblh5JKOoLUIxGDQs4LFFD--41Vzf-ME1' }
  folder_for_playlist_collection = 'statquest'

if 'MIT courses' in selected_collection:
  courses = { '6.0001 Introduction to Computer Science and Programming in Python'                           : 'PLUl4u3cNGP63WbdFxL8giv4yhgdMGaZNA',
              'MIT 18.S096 Matrix Calculus For Machine Learning And Beyond'                                 : 'PLUl4u3cNGP62EaLLH92E_VCN4izBKK6OE',
              'MIT 6.S191: Introduction to Deep Learning'                                                   : 'PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI',
              'EfficientML.ai Lecture, Fall 2023, MIT 6.5940'                                               : 'PL80kAHvQbh-pT4lCkDT53zT8DKmhE0idB' }
  folder_for_playlist_collection = 'mit'

if 'Stanford' in selected_collection:
  courses = { 'CS221:  2021 - Artificial Intelligence: Principles and Techniques (Percy Liang)'             : 'PLoROMvodv4rOca_Ovz1DvdtWuz8BfSWL2',
              'CS221:  2019 - Artificial Intelligence: Principles and Techniques (Percy Liang)'             : 'PLoROMvodv4rO1NB9TD4iUZ3qghGEGtqNX',
              'CS224N: 2021 - NLP with Deep Learning (Christopher Manning)'                                 : 'PLoROMvodv4rOSH4v6133s9LFPRHjEmbmJ',
              'CS224W: 2021 - Machine Learning with Graphs (Jure Leskovec)'                                 : 'PLoROMvodv4rPLKxIpqhjhPgdQy7imNkDn',
              'CS224U: 2021 - Natural Language Understanding (Christopher Potts)'                           : 'PLoROMvodv4rPt5D0zs3YhbWSZA8Q_DyiJ',
              'CS229:  2018 - Machine Learning Full Course (Andrew Ng)'                                     : 'PLoROMvodv4rMiGQp3WXShtMGgzqpfVfbU',
              'CS229:  2019 - Machine Learning Course (Anand Avati)'                                        : 'PLoROMvodv4rNH7qL6-efu_q2_bPuy0adh',
              'CS330:  2022 - Deep Multi-Task & Meta Learning - What is multi-task learning? (Chelsea Finn)': 'PLoROMvodv4rNjRoawgt72BBNwL2V7doGI' }
  folder_for_playlist_collection = 'stanford'

if 'Deep Learning for Coders' in selected_collection:
  courses = { 'Practical Deep Learning for Coders 2022'                                                     : 'PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU',
              'Practical Deep Learning 2022 Part 2'                                                         : 'PLfYUBJiXbdtRUvTUYpLdfHHp9a58nWVXP',
              'fast.ai live coding & tutorials'                                                             : 'PLfYUBJiXbdtSLBPJ1GMx-sQWf6iNhb8mM',
              'Computational Linear Algebra'                                                                : 'PLtmWHNX-gukIc92m1K0P6bIOnZb-mg0hY',
              'Practical Deep Learning for Coders (2020)'                                                   : 'PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM',
              'Practical Deep Learning for Coders 2019'                                                     : 'PLfYUBJiXbdtSIJb-Qd3pw0cqCbkGeS0xn' }
  folder_for_playlist_collection = 'dlfc'


####################################################################################################################################
########################################## single video / folders / globals / playlists.yaml #######################################
####################################################################################################################################
# Single video
if 'Single video selection' in selected_collection:
  courses = {'Single video selection':'no playlist'}
  folder_for_playlist_collection = 'singles'

# gdrive only for hosted runtime
if use_gdrive_for_persistence and os.path.expanduser('~') == '/root':
  from google.colab import drive
  if not 'gdrive'  in os.listdir('/'): drive.mount('/gdrive')
  # working folder gdrive
  if not 'MyDrive' in os.getcwd(): os.chdir ('/gdrive/MyDrive')
else:
  # working folder hosted / local
  if os.path.expanduser('~') == '/root':  os.chdir ('/content')
  else:                                   os.chdir (os.path.expanduser('~'))

# create / change to main folder
if main_folder in os.listdir():  os.chdir (main_folder)
else:
  # check if parent main folder
  if main_folder in os.getcwd(): os.chdir('..')
  else:
    os.mkdir (main_folder)
    os.chdir(main_folder)

# create / change to collection folder
if folder_for_playlist_collection in os.listdir(): os.chdir (folder_for_playlist_collection)
else:
  os.mkdir (folder_for_playlist_collection)
  os.chdir(folder_for_playlist_collection)

# load
if os.path.exists('playlists.yml'):
  playlists_yaml = yaml.load(open('playlists.yml', 'r'), Loader=yaml.FullLoader)
# or new (language)
try:    playlists_yaml['language'] = country_code_for_the_translation_language.split(' ')[0]
except: playlists_yaml             = {'language':country_code_for_the_translation_language.split(' ')[0]}
cc                                 = playlists_yaml['language']

# playlist loop
for i,title in enumerate(courses):
  # single video
  if courses[title] == 'no playlist':
    try:    vl = playlists_yaml['no playlist']['videos']
    except: vl = []
    playlists_yaml['no playlist'] = {'title':'single videos', 'description':'hand added videos', 'videos':vl}
    break

  # check if infos exists
  if not courses[title] in playlists_yaml:
    # lazy using pytube
    from pytube import Playlist
    try:      pl     = Playlist(f'{playlist_prefix}{courses[title]}')
    except:   raise    Exception ('BIG_OOPS')
    try:      descr  = pl.description
    except:   descr  = 'no description in pytube'
    try:      links  = [link for link in pl]
    except:   links  = []
    videos           = [f"{link.split('?v=')[-1]} Video {vnr+1}" for vnr, link in enumerate(links)]
    playlists_yaml[courses[title]] = {'title':title, 'description':descr, 'videos':videos}
# write playlists.yml
yaml.dump (playlists_yaml, open('playlists.yml', 'w'))


####################################################################################################################################
########################################################## helper ##################################################################
####################################################################################################################################
def globalize ():
  """ set globals (widgets and relevant infos) to the current accordion tab / selected video
      see widgets structure in help text
  """
  # relevant infos
  global vPositions, videoInfos, enTrans, xxTrans, courses, playlists_yaml, cc, acIndex, plid, rbIndex, vid, countryCodes
  global store_en, store_xx
  # widgets
  global mainAccordion,        gb_maingb, vb_vidsel, vb_chpare, wi_firstw, rb_vidsel, bu_allchp, hb_cleanb, bu_frodes
  global hb_autchp, hb_aichbu, bu_cleanb, bu_autchp, bu_fromai, bu_aichap, bx_chpsel, hb_contrl, hb_transc, vb_enCtrl
  global vb_xxCtrl, hb_enCtrl, hb_xxCtrl, bu_enShow, bu_enSend, bu_enLoop, bu_xxShow, bu_xxSend, bu_xxLoop, hb_enSele
  global hb_xxSele, bu_enView, bu_enResu, bu_enLsel, bu_xxView, bu_xxResu, bu_xxLsel, ta_enTran, ta_xxTran

  # set globals to current displayed playlist
  gb_maingb                       = mainAccordion.children[mainAccordion.selected_index]
  vb_vidsel, vb_chpare            = gb_maingb.children[0], gb_maingb.children[1]

  # vb_vidsel - area left with video selection and buttons
  wi_firstw, rb_vidsel, bu_allchp = vb_vidsel.children[0], vb_vidsel.children[1], vb_vidsel.children[2]
  hb_cleanb, bu_frodes, hb_autchp = vb_vidsel.children[3], vb_vidsel.children[4], vb_vidsel.children[5]
  hb_aichbu                       = vb_vidsel.children[6]
  bu_cleanb, bu_autchp            = hb_cleanb.children[1], hb_autchp.children[1]
  bu_fromai, bu_aichap            = hb_aichbu.children[0], hb_aichbu.children[1]

  # vb_chpare - big area with chapter selector and transcriptions
  bx_chpsel, hb_contrl, hb_transc = vb_chpare.children[0], vb_chpare.children[1], vb_chpare.children[2]
  vb_enCtrl, vb_xxCtrl            = hb_contrl.children[0], hb_contrl.children[1]
  hb_enCtrl, hb_xxCtrl            = vb_enCtrl.children[0], vb_xxCtrl.children[0]
  bu_enShow, bu_enSend, bu_enLoop = hb_enCtrl.children[3],hb_enCtrl.children[4],hb_enCtrl.children[5]
  bu_xxShow, bu_xxSend, bu_xxLoop = hb_xxCtrl.children[3],hb_xxCtrl.children[4],hb_xxCtrl.children[5]
  hb_enSele, hb_xxSele            = vb_enCtrl.children[1], vb_xxCtrl.children[1]
  bu_enView, bu_enResu, bu_enLsel = hb_enSele.children[0].children[0],hb_enSele.children[0].children[1], hb_enSele.children[1].children[0]
  bu_xxView, bu_xxResu, bu_xxLsel = hb_xxSele.children[0].children[0],hb_xxSele.children[0].children[1], hb_xxSele.children[1].children[0]
  ta_enTran, ta_xxTran            = hb_transc.children[0], hb_transc.children[1]

  # current selection
  acIndex = mainAccordion.selected_index
  plid    = [courses[all] for all in courses][acIndex]
  rbIndex = rb_vidsel.options.index(rb_vidsel.value) if rb_vidsel.value else -1
  vid     = playlists_yaml[plid]['videos'][rbIndex][:11] if rbIndex > -1 else ''

def parseChaptersFromDescription (description):
  lines = description.split('\n')
  chaps = []
  for l in lines:
    if ':' in l:
      for i,c in enumerate(l):
        if c == ':':
          if i > 0 and l[i-1].isdigit() and i < len(l)-1 and l[i+1].isdigit():
            start = i - (2 if i > 1 and l[i-2].isdigit() else 1)
            for end,c in enumerate (l[i+1:]):
              if not c.isdigit() and c != ':': break
            chaps.append(l[start:end+i+1]+' '+l[end+i+2:])
            break
  return chaps

def timestampTabView (trans):
  """ make the transcription text format for the textareas """
  txt, max_block_line_len = '', 60
  for all in trans:
    tmp, block, sec = all['text'].replace('\xa0','').replace('\n',''), [], int(all['start'])
    h, i = sec//3600, len (tmp)
    m    = (sec - (h*3600)) // 60
    s    = sec - h*3600 - m*60
    # make a text block
    while len (tmp) > max_block_line_len:
      for i in reversed(range(max_block_line_len)):
        if tmp[i] == ' ': break
      block.append(tmp[:i])
      tmp = tmp[i+1:]
    block.append(tmp[:i])
    # and show it with tabs
    for i,l in enumerate (block):
      if i == 0: txt += f"{h}:{m:02d}:{s:02d}\t{l}\n"
      else:      txt += f"\t{l}\n"
  return txt

def getStartEndFromChapterButton (b):
  gb_maingb                       = mainAccordion.children[mainAccordion.selected_index]
  vb_vidsel, vb_chpare            = gb_maingb.children[0], gb_maingb.children[1]
  bx_chpsel, hb_contrl, hb_transc = vb_chpare.children[0], vb_chpare.children[1], vb_chpare.children[2]

  # start ts from button
  ts = b.description.split(' ')[0]
  if len(ts.split(':')) == 2: start = int(ts.split(':')[0])*60+int(ts.split(':')[1])
  else: start = int(ts.split(':')[0])*3600+int(ts.split(':')[1])*60+int(ts.split(':')[2])

  # search next button
  nb = None
  for i,sb in enumerate(bx_chpsel.children):
    if b.description == sb.description: break

  # end ts from search
  if i >= len(bx_chpsel.children)-1: ts = '99:00:00'
  else: ts = bx_chpsel.children[i+1].description.split(' ')[0]
  if len(ts.split(':')) == 2: end = int(ts.split(':')[0])*60+int(ts.split(':')[1])
  else: end = int(ts.split(':')[0])*3600+int(ts.split(':')[1])*60+int(ts.split(':')[2])
  return start, end

def makeAutoChapters (max,lines,tfac):
  # generate chapters (timestamp list)
  chapters = []
  while(True):
    # first in lines is new timestamp
    if lines[0].split('\t')[0] != '': chapters.append(lines[0].split('\t')[0])

    # search for the next timestamp so that the distance between the two timestamps (growRange)
    # is bigger than max tokens and rest is big enough for own chapter (1/5 max).
    for nextTS,l in enumerate (lines):
      growRange = len('\n'.join(lines[:nextTS]))//tfac
      restRange = len('\n'.join(lines[nextTS:]))//tfac
      if len(lines[nextTS]) and growRange > max and lines[nextTS][1]==':' and lines[nextTS][4]==':' and restRange > max/5: break

    # shrink lines or terminate
    if len(lines[:nextTS]) > 0:  lines = lines[nextTS:]
    else:                        break
  return chapters

def taCleaner (lines,tsCount,bs=None):
  if tsCount > len (lines): tsCount = 1

  org, tmp, trigger = tsCount, '', False
  for i,l in enumerate(lines): # all lines
    if tsCount and not i%(len(lines)//org): # trigger if ts should create
      tsCount -= 1
      trigger = True
    if trigger and l != '' and l[1] == ':': # create ts
      if i > 0: tmp += '\n'
      tmp += l.split('\t')[0]+'\n'          # create blockline
      trigger = False
    if '\t' in l:
      tmp += l.split('\t')[1]+'\n'
  # return tmp - if no block wanted
  if bs == None: bs = 60
  lines, tmp2, nl = tmp.split('\n'), '', ''
  for i,l in enumerate(lines): # build block
    if len (l) > 1 and l[1] != ':':
      words = l.split(' ')
      for w in words:
        if len (nl+' '+w) < bs:
          nl += ' '+w
        else:
          tmp2 += nl + '\n'
          nl = w
    else:
      tmp2 += l+'\n'
  return tmp2.replace('\n ','\n')

def storeLoop (vid, txt, ccode):
  lid = ''
  if txt[:13] == '... do prompt':
    # yaml
    if not os.path.exists(f'{vid}_loops_{ccode}.yml'): loops_yaml = {}
    else: loops_yaml = yaml.load (open(f'{vid}_loops_{ccode}.yml', 'r'), Loader=yaml.FullLoader)
    lid = datetime.now().strftime("%Y%m%d-%H%M%S")
    loops_yaml[lid] = txt
    yaml.dump (loops_yaml, open(f'{vid}_loops_{ccode}.yml', 'w'))
  return lid

def resetTransArea ():
  globalize()

  # unselect prompt / clear textareas / message / lastState
  hb_enCtrl.children[1].value = hb_xxCtrl.children[1].value = '<<<select prompt>>>'
  ta_enTran.value, ta_xxTran.value = "... select chapter or click 'all >>>>'", '...'

  # buttonstyles
  for but in bx_chpsel.children: but.button_style   = ''
  bu_allchp.button_style                            = ''




####################################################################################################################################
################################################## openai-stuff: prompts ###########################################################
####################################################################################################################################
# defaults
summary_50_en = """Below is an excerpt from a transcription of a video that starts after this string '##x##'.
The video is a course about Deep Learning.
Create a summary of the content of the excerpt in 50 words or less.
To do this, first clean up the raw text by removing the time stamps, merging the text, and ridding it of errors or clutter.
##x##
"""
summary_50_de = """Im Folgenden findest du einen Auszug aus der Transkription eines Videos, das nach dieser Zeichenfolge '##x##' beginnt.
Das Video ist ein Kurs über Deep Learning.
Erstelle eine Zusammenfassung des Inhalts des Auszugs in 50 Wörtern oder weniger.
Bereinige dazu zunächst den Rohtext, indem du die Zeitstempel entfernst, den Text zusammenführst und ihn von Fehlern oder Unordnung befreien.
##x##
"""
keywords_en = """The text is an excerpt from the transcription of a video and it starts after this character sequence: '##x##'. Create a keyword list in the following structure:

- Keyword 1
- ...
- Keyword n

##x##
"""
keywords_de = """Bei dem Text handelt es sich um einen Auszug aus der Transkription eines Videos und er beginnt nach dieser Zeichenfolge: '##x##'. Erstelle eine Schlagwortliste in folgender Struktur:

- Schlagwort 1
- ...
- Schlagwort n

##x##
"""
hints_en = """The text is an excerpt from the transcription of a video and it starts after this character sequence: '##x##'. Create a hint list in the following structure:

- Hint 1
- ...
- Hint n

##x##
"""
hints_de = """Bei dem Text handelt es sich um einen Auszug aus der Transkription eines Videos und er beginnt nach dieser Zeichenfolge: '##x##'. Erstelle eine Liste von Hinweisen in folgender Struktur:

- Hinweis 1
- ...
- Hinweis n

##x##
"""
aichapters_de = """Bei dem Text handelt es sich um einen Auszug aus der Transkription eines Videos und er beginnt nach dieser Zeichenfolge: '##x##'.
Das Video ist eines von vielen aus einer Tutorial Reihe zum Thema Deep Learning für Programmierer.
Finde einen Titel für diese Sektions des Videos, der zusammenfassende Schlagwörter beinhaltet und nicht mehr als 10 Wörter sein soll.
Gib den nur Titel als ergebnis zurück in deutscher Sprache.
##x##
"""
aichapters_en = """The text is an excerpt from the transcription of a video and it starts after this string: '##x##'.
The video is one of many in a tutorial series on Deep Learning for programmers.
Find a title for this section of the video that includes summary keywords and should be no more than 10 words.
Return only the title as the result.
##x##
"""
profile_de = """Bei dem Text handelt es sich um einen Auszug aus der Transkription eines Videos und er beginnt nach dieser Zeichenfolge: ##x##. Erstelle eine Kapitelbeschreibung für den Text in folgender Struktur:

Schlagwörter:
- Schlagwort 1
- ...
- Schlagwort n

Hinweise:
- Hinweis 1
- ...
- Hinweis n

Zusammenfassung:
<ein zusammenfassender Text mit nicht mehr als 100 Wörtern>

##x##
"""
profile_en = """The text is an excerpt from the transcription of a video and starts after this character string: ##x##. Create a chapter description for the text in the following structure:

Keywords:
- Keyword 1
- ...
- Keyword n

Notes:
- Note 1
- ...
- Note n

Summary:
<a summary text with no more than 100 words>

##x##
"""
# prompts_yaml
if not os.path.exists('prompts.yml'):
  prompts_yaml = {'summary 50':   {'en':summary_50_en , 'de':summary_50_de},
                  'keywords':     {'en':keywords_en   , 'de':keywords_de},
                  'hints':        {'en':hints_en      , 'de':hints_de},
                  'profile':      {'en':profile_en    , 'de':profile_de},
                  '_aichapters_': {'en':aichapters_en , 'de':aichapters_de}}
  yaml.dump (prompts_yaml, open('prompts.yml', 'w'))
else:
  prompts_yaml = yaml.load(open('prompts.yml', 'r'), Loader=yaml.FullLoader)


####################################################################################################################################
############################################ openai-stuff: libs / account / helper #################################################
####################################################################################################################################
if yes_i_have_this_chatgpt_openai_account:
  default_temperature = 0.2
  default_chartokfac  = 3.1
  #myApiKey            = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
  tmphm =  HTML("""Get <a href='https://platform.openai.com/account/api-keys' target='_blank'>here</a> your API-Key.""")
  display  (tmphm)
  try:     import openai
  except:  os.system ('pip install openai'); import openai
  try:     myApiKey = os.environ["OPENAI_API_KEY"]
  except:  pass
  try:     os.environ["OPENAI_API_KEY"] = openai.api_key = myApiKey
  except:  os.environ["OPENAI_API_KEY"] = openai.api_key = getpass.getpass (prompt='Your API key: ')
  tmphm.value = """<font color='green'>Set API-key done!</font>"""
  from openai import OpenAI
  chatclient = OpenAI()

def get_completion(prompt, model=None,temperature=None):
  if not yes_i_have_this_chatgpt_openai_account: return 'no account'
  # do an openai api call
  if not model:         model         = used_model
  if not temperature:   temperature   = default_temperature
  messages, maxTry                    = [{"role": "user", "content": prompt}], 3
  for t in range (maxTry):
    try: # try maxTry times
      time.sleep (t+1)
      chatclient.chat.completions
      return chatclient.chat.completions.create(model=model, messages=messages, temperature=temperature).choices[0].message.content
      #return 'under construction'
    except Exception as e:
     print (f"\r\x1b[91mApi-Error: {e}",end=' ')
  return 'oops - something goes wrong...'

def text_change(_):
  global textChangeFromYaml
  if textChangeFromYaml: return

  cyanbox = lambda s: "<table width='50'><tr><td align='center'><p style='background-color:#B0E0E6'>"+s+"</p></td></tr></table>"
  # compute tokens
  hb_enCtrl.children[0].value = cyanbox(str(int(len(ta_enTran.value) // default_chartokfac+1)))
  hb_xxCtrl.children[0].value = cyanbox(str(int(len(ta_xxTran.value) // default_chartokfac+1)))

  # store text
  for i,b in enumerate (hb_enSele.children[0].children):
    if b.layout.border != None: break
  if i >= 0 and i < 2: store_en[i] = ta_enTran.value

  for i,b in enumerate (hb_xxSele.children[0].children):
    if b.layout.border != None: break
  if i >= 0 and i < 2: store_xx[i] = ta_xxTran.value


def refreshPrompts():
  # tab loop
  for gb_maingb in mainAccordion.children[:-2]:
    vb_vidsel      = gb_maingb.children[0]
    prompt_buttons = [Button (description=p,
                              tooltip=str(prompts_yaml[p]['en']) if 'en' in prompts_yaml[p] else '' + '\n' +
                                      str(prompts_yaml[p][cc])  if cc in prompts_yaml[p] else '',
                              style={'button_color':'lightgreen'}) for p in prompts_yaml if p[0] != '_']
    for b in prompt_buttons: b.on_click (promptButtonClick)
    firstPromptButtonPos = 3 # for safety
    for i,but in enumerate (vb_vidsel.children):
      try: # somtimes python is ... - hm ... - special
        if but.style.button_color == 'lightgreen':
          firstPromptButtonPos = i
          break
      except: pass
    for oldbut in vb_vidsel.children[firstPromptButtonPos:]:  del oldbut
    vb_vidsel.children = [*vb_vidsel.children[:firstPromptButtonPos],*prompt_buttons]


####################################################################################################################################
################################################ openai-stuff: buttons bi-transcription area #######################################
####################################################################################################################################
def ccShowButtonClick (b,ccode):
  if ccode == 'en': hb_ccCtrl, bu_ccView, ta_ccTran = hb_enCtrl, bu_enView, ta_enTran
  else:             hb_ccCtrl, bu_ccView, ta_ccTran = hb_xxCtrl, bu_xxView, ta_xxTran
  # check prompt selected
  if hb_ccCtrl.children[1].value == '<<<select prompt>>>': return
  # select view areas
  if ccode == 'en':  areaSelect_en (bu_ccView)
  else: areaSelect_xx (bu_ccView)
  # generate prompts and show
  if len (prompts_yaml[hb_ccCtrl.children[1].value][ccode]) > 50:
    ta_ccTran.value = f"{prompts_yaml[hb_ccCtrl.children[1].value][ccode]}\n{ta_ccTran.value}"
def enShowButtonClick (b): ccShowButtonClick (b,'en')
def xxShowButtonClick (b): ccShowButtonClick (b,country_code_for_the_translation_language.split(' ')[0])

def ccSendButtonClick (b, ccode):
  # set language and select result area
  if ccode == 'en':
    hb_ccCtrl, store_cc, ta_ccTran = hb_enCtrl, store_en, ta_enTran
    areaSelect_en(bu_enResu)
  else:
    hb_ccCtrl, store_cc, ta_ccTran = hb_xxCtrl, store_xx, ta_xxTran
    areaSelect_xx(bu_xxResu)

  # show wait message / response
  prompt = hb_ccCtrl.children[1].value
  hb_ccCtrl.children[1].value = '<<<select prompt>>>'
  if yes_i_have_this_chatgpt_openai_account:
    ta_ccTran.value = '... waiting for response'
    ta_ccTran.value = f'>>>>> response for prompt {prompt} >>>>>\n'+get_completion (store_cc[0],temperature=float(hb_ccCtrl.children[2].value))
  else:
    ta_ccTran.value = 'sorry - no account'

def enSendButtonClick (b): ccSendButtonClick (b,'en')
def xxSendButtonClick (b): ccSendButtonClick (b,country_code_for_the_translation_language.split(' ')[0])

def ccLoopButtonClick (b,ccode):
  globalize ()
  if ccode == 'en': hb_ccCtrl, hb_ccSele, areaSelect_cc, store_cc, ta_ccTran, ccTrans = hb_enCtrl, hb_enSele, areaSelect_en, store_en, ta_enTran, enTrans
  else:             hb_ccCtrl, hb_ccSele, areaSelect_cc, store_cc, ta_ccTran, ccTrans = hb_xxCtrl, hb_xxSele, areaSelect_xx, store_xx, ta_xxTran, xxTrans

  # check prompt selected
  if hb_ccCtrl.children[1].value == '<<<select prompt>>>': return

  # create and select new loop button
  for bu in hb_ccSele.children[0].children: bu.layout = Layout (width='auto', height='15px')
  newbut = Button (layout=Layout(width='auto', height='15px', border='2px solid'), style={'button_color':'hotpink'},
                   tooltip='')
  newbut.on_click (areaSelect_cc)
  store_cc.append('')
  hb_ccSele.children[0].children = [*hb_ccSele.children[0].children, newbut]

  # show wait message / response
  prompt = hb_ccCtrl.children[1].value
  ta_ccTran.value = f'... do prompt <"'+prompt+'"> for ...\n'
  for b in bx_chpsel.children:
    ta_ccTran.value += '\n>>>>>>'+b.description + '>>>>>>\n'
    start, end       = getStartEndFromChapterButton (b)
    tmp              = taCleaner (timestampTabView ([t for t in ccTrans[vid] if t['start'] >= start and t['start'] < end]).split('\n'),0)
    response         = get_completion ( f"{prompts_yaml[prompt][ccode]}\n{tmp}", temperature=float(hb_ccCtrl.children[2].value) )
    ta_ccTran.value += f"{response}\n" if response[-1] != '\n' else f"{response}"

  # yaml it
  lid = storeLoop (vid, ta_ccTran.value, ccode)
  newbut.tooltip = f'{lid} - prompt: {prompt}'
  ta_ccTran.value += f"\n\n>>>>>> write to {f'{vid}_loops_{ccode}.yml >>>>>>'}"
def enLoopButtonClick (b): ccLoopButtonClick (b,'en')
def xxLoopButtonClick (b): ccLoopButtonClick (b,country_code_for_the_translation_language.split(' ')[0])

textChangeFromYaml = False
def areaSelect_cc (b, ccode):
  global textChangeFromYaml
  textChangeFromYaml = True

  if ccode == 'en': hb_ccSele, store_cc, ta_ccTran = hb_enSele, store_en, ta_enTran
  else:             hb_ccSele, store_cc, ta_ccTran = hb_xxSele, store_xx, ta_xxTran

  # save current textarea
  for i,bu in enumerate (hb_ccSele.children[0].children):
    if bu.layout.border: break
  if i >= 0 and i < 2: store_cc[i] = ta_ccTran.value

  # draw / undraw selection border
  for bu in hb_ccSele.children[0].children: bu.layout = Layout (width='auto', height='15px')
  b.layout = Layout (width='auto', height='15px', border='2px solid')

  # restore textarea
  for i,bu in enumerate (hb_ccSele.children[0].children):
    if bu == b: break
  if i >= 0 and i < len(store_cc): ta_ccTran.value = store_cc[i]
  textChangeFromYaml = False

def areaSelect_en (b): areaSelect_cc (b,'en')
def areaSelect_xx (b): areaSelect_cc (b,country_code_for_the_translation_language.split(' ')[0])

def loopDelete_cc (ccode):
  if ccode == 'en': hb_ccSele, store_cc, areaSelect_cc, bu_ccView = hb_enSele, store_en, areaSelect_en, bu_enView
  else:             hb_ccSele, store_cc, areaSelect_cc, bu_ccView = hb_xxSele, store_xx, areaSelect_xx, bu_xxView
  # find selected loop area
  for i,b in enumerate (hb_ccSele.children[0].children):
    if b.layout.border != None: break
  if i >= 2 and i < len (hb_ccSele.children[0].children):
    # delete from txt store, button list and yaml file / select view area
    hb_ccSele.children[0].children = [*hb_ccSele.children[0].children[:i],*hb_ccSele.children[0].children[i+1:]]
    store_cc = [store_cc[:i],store_cc[i+1:]]
    areaSelect_cc (bu_ccView);
    try:
      loops_yaml = yaml.load(open(f'{vid}_loops_{ccode}.yml', 'r'), Loader=yaml.FullLoader)
      del loops_yaml[b.tooltip.split(' - ')[0]]
      yaml.dump (loops_yaml, open(f'{vid}_loops_{ccode}.yml', 'w'))
    except: pass
def loopDelete_en (b): loopDelete_cc ('en')
def loopDelete_xx (b): loopDelete_cc (country_code_for_the_translation_language.split(' ')[0])


####################################################################################################################################
####################################################### buttons left area ##########################################################
####################################################################################################################################
def promptButtonClick (b):
  # set prompt actve -> show-button
  hb_enCtrl.children[1].value, hb_xxCtrl.children[1].value = b.description, b.description
  if 'info' in [b.button_style for b in [bu_allchp,*bx_chpsel.children]]: bu_enShow.disabled = bu_xxShow.disabled = False
  bu_enLoop.disabled = bu_xxLoop.disabled = False

def chapbutClick (b):
  globalize ()

  # select view areas
  areaSelect_en (bu_enView); areaSelect_xx (bu_xxView)

  # buttonstyles
  for but in bx_chpsel.children: but.button_style   = ''
  bu_allchp.button_style, b.button_style            = '', 'info'

  # show complete text
  if b.description == 'all >>>>':
    ta_enTran.value = timestampTabView (enTrans[vid])
    ta_xxTran.value = timestampTabView (xxTrans[vid])
  # or chapter filtered
  else:
    start, end    = getStartEndFromChapterButton (b)
    ta_enTran.value = timestampTabView ([t for t in enTrans[vid] if t['start'] >= start and t['start'] < end])
    ta_xxTran.value = timestampTabView ([t for t in xxTrans[vid] if t['start'] >= start and t['start'] < end])

def fromDescriptionButtonClick (_):
  globalize ()
  tmp = timestampTabView(enTrans[vid]).split('\n')

  # del old chapter buttons
  for oldbut in bx_chpsel.children: del oldbut

  # build buttons
  chapters = videoInfos[vid]['chapters']
  bu_allchp.tooltip = '>>>>Description from YouTube>>>\n'+str(videoInfos[vid]['description'])
  bu_allchp.button_style = ''
  bx_chpsel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                              tooltip      = c,
                              layout       = Layout(width='auto', height='21px'))
                      for i,c in enumerate (chapters)]
  for button in bx_chpsel.children: button.on_click(chapbutClick)

  # reset
  resetTransArea ()

def autoButtonClick (_):
  globalize ()
  # select green
  areaSelect_en (bu_enView); areaSelect_xx (bu_xxView)

  tmp = timestampTabView(enTrans[vid]).split('\n')

  # makeAutoChapters
  chapters = makeAutoChapters (int(hb_autchp.children[0].value),tmp,default_chartokfac)

  bu_allchp.tooltip = str(videoInfos[vid]['description'])
  # del old chapter buttons
  for oldbut in bx_chpsel.children: del oldbut
  # and make new
  bx_chpsel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                                tooltip      = c,
                                layout       = Layout(width='auto', height='21px'))
                        for i,c in enumerate (chapters)]
  for button in bx_chpsel.children: button.on_click(chapbutClick)

  # reset
  resetTransArea ()

def cleanButtonClick (_):
  # parse blocksize - check > 40 and < 200
  val = hb_cleanb.children[0].value
  if not ',' in val:  bs = None
  else:               bs = int(val.split(',')[1]) if int(val.split(',')[1]) < 200 and int(val.split(',')[1]) > 40 else None
  if "select chapter or click 'all >>>>'" in ta_enTran.value: return

  # rebuild ts view (if button clicked twice or mor)
  if not '\t' in ta_enTran.value:
    if bu_allchp.button_style == 'info': chapbutClick (bu_allchp)
    else: chapbutClick ([b for b in bx_chpsel.children if b.button_style=='info'][0])

  # use taCleaner (<linelist>,<count timestamps>,<blocksize>)
  ta_enTran.value = taCleaner ([l for l in ta_enTran.value.split('\n') if l != ''], int(val.split(',')[0]), bs)
  ta_xxTran.value = taCleaner ([l for l in ta_xxTran.value.split('\n') if l != ''], int(val.split(',')[0]), bs)

def aiShowButtonClick (_):
  globalize ()

  # select green
  areaSelect_en (bu_enView); areaSelect_xx (bu_xxView)

  if use_english_for_ai_auto_chapters: ccode = 'en'
  else: ccode = cc
  tmp = timestampTabView(enTrans[vid]).split('\n')

  # del old chapter buttons
  for oldbut in bx_chpsel.children: del oldbut

  # build ai chapter buttons
  chapters, lastTS = videoInfos[vid]['aichapters'][ccode] if ccode in videoInfos[vid]['aichapters'] else [], 0
  for child in bx_chpsel.children: del child
  bx_chpsel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                                tooltip      = c,
                                layout       = Layout(width='auto', height='21px'))
                      for i,c in enumerate (chapters)]
  for button in bx_chpsel.children: button.on_click(chapbutClick)

  # reset
  resetTransArea ()

def aiBuildButtonClick (_):
  globalize ()

  # select green
  areaSelect_en (bu_enView); areaSelect_xx (bu_xxView)

  # use existing chapters from buttons or do makeAutoChapters first
  if len(bx_chpsel.children) > 0:  chapters = [b.description.split(' ')[0] for b in bx_chpsel.children]
  else:                            chapters = makeAutoChapters (int(hb_autchp.children[0].value),tmp,tfac)

  # use existing chapters from buttons or do makeAutoChapters first
  if len(bx_chpsel.children) > 0:  chapters = [b.description.split(' ')[0] for b in bx_chpsel.children]
  else:                            chapters = makeAutoChapters (int(hb_autchp.children[0].value),timestampTabView(enTrans[vid]).split('\n'),default_chartokfac)

  # del old buttons
  for oldbut in bx_chpsel.children: del oldbut
  # and make new
  bx_chpsel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                                tooltip      = c,
                                layout       = Layout(width='auto', height='21px'))
                      for i,c in enumerate (chapters)]

  # wait message and start loop over chapters
  ta_enTran.value, ta_xxTran.value = "... please wait - chapter building in progress >>>>'", '...'
  for b in bx_chpsel.children:
    start, end = getStartEndFromChapterButton (b)
    # get ai chapter title
    tmp = []
    if use_english_for_ai_auto_chapters:
      ccode = 'en'
      tmp   = taCleaner (timestampTabView ([t for t in enTrans[vid] if t['start'] >= start and t['start'] < end]).split('\n'),0)
      title = get_completion ( f"{prompts_yaml['_aichapters_']['en']}\n{tmp}", temperature=float(hb_enCtrl.children[2].value) )
    else:
      ccode = cc
      tmp   = taCleaner (timestampTabView ([t for t in xxTrans[vid] if t['start'] >= start and t['start'] < end]).split('\n'),0)
      title = get_completion (f"{prompts_yaml['_aichapters_'][cc]}\n{tmp}", temperature=float(hb_xxCtrl.children[2].value))

    # set new title as button description
    if len (title) > 0:
      if title[0] == '"' or title[0] == "'":   title = title[1:]
      if title[-1] == '"' or title[-1] == "'": title = title[:-1]
    b.description = b.description.split(' ')[0]+' '+title
  for button in bx_chpsel.children: button.on_click(chapbutClick)

  # reset
  resetTransArea ()

  # store in yaml
  videoInfos[vid]['aichapters'][ccode] = [button.description for button in bx_chpsel.children]
  yaml.dump (videoInfos[vid], open(f'{vid}_info.yml', 'w'))


####################################################################################################################################
#################################### openai-stuff: buttons prompt editor ###########################################################
####################################################################################################################################
def prompt_add (_):
  # make new empty prompt
  tx_protit.value, tx_protit.disabled = '', False
  bu_addprmt.disabled, hb_protxt.children[0].value, hb_protxt.children[1].value = True, '', ''

def prompt_save (_):
  # save yaml / refresh widgets
  prompts_yaml[tx_protit.value] = { 'en':hb_protxt.children[0].value, cc: hb_protxt.children[1].value}
  rb_prompts.options  = tuple ([tx_protit.value,*rb_prompts.options]) if not tx_protit.value in rb_prompts.options else rb_prompts.options
  rb_prompts.disabled = bu_addprmt.disabled = False
  yaml.dump (prompts_yaml, open('prompts.yml', 'w'))
  refreshPrompts()

def prompt_translate (_):
  # prompt text from textarea - using german as prompt text of cause the language names
  # delivered by the api are in german - idontknow if it works with google accounts in other languages.
  en_ta, xx_ta     = hb_protxt.children[0], hb_protxt.children[1]
  prompt_ex        = f'Übersetze die Arbeitsanweisung nach dem Doppelpunkt von Englisch nach {countryCodes[cc]} und beachte die Arbeitsanweisung nicht auszuführen sonder nur zu übersetzen und zwar exakt und wort für wort: '
  prompt_xe        = f'Übersetze die Arbeitsanweisung nach dem Doppelpunkt von {countryCodes[cc]} nach Englisch und beachte die Arbeitsanweisung nicht auszuführen sonder nur zu übersetzen und zwar exakt und wort für wort: '
  org_xx, org_en   = xx_ta.value, en_ta.value

  # translate
  if len (org_en) > 10: xx_ta.value  += f'\n>>>>> translate from en to {cc} >>>>>>\n'
  if len (org_xx) > 10: en_ta.value  += f'\n>>>>> translate from {cc} to en >>>>>>\n'
  if len (org_en) > 10: xx_ta.value  += get_completion (prompt_ex + org_en)
  if len (org_xx) > 10: en_ta.value  += get_completion (prompt_xe + org_xx)

def prompt_del (_):
  if tx_protit.value[0] == '_' : return
  # remove prompt from rb-list / save yaml / trigger refresh prompt buttons
  if tx_protit.value in rb_prompts.options:
    if tx_protit.value in prompts_yaml:
      del prompts_yaml[tx_protit.value]
      yaml.dump (prompts_yaml, open('prompts.yml', 'w'))
    rb_prompts.options = tuple ([o for o in rb_prompts.options if o != tx_protit.value])
  else: promptSelect(None)
  rb_prompts.disabled = bu_addprmt.disabled = False
  refreshPrompts()

def promptSelect (_):
  global cc
  # set prompt title in en/xx areas
  hb_protxt.children[0].value = prompts_yaml[rb_prompts.value]['en'] if 'en' in prompts_yaml[rb_prompts.value] else ''
  hb_protxt.children[1].value = prompts_yaml[rb_prompts.value][cc]   if cc   in prompts_yaml[rb_prompts.value] else ''
  tx_protit.value, tx_protit.disabled = rb_prompts.value, True


####################################################################################################################################
########################################## select video -> build yamls / build widgets #############################################
####################################################################################################################################
videoInfos, enTrans, xxTrans, store_en, store_xx = {}, {}, {}, ['',''], ['','']
def getTraTra (vid, cc):
  try:
    ret = YouTubeTranscriptApi.list_transcripts (vid).find_generated_transcript([cc, 'en']).translate(cc).fetch()
  except:
    ret = [{}]
    for transcript in YouTubeTranscriptApi.list_transcripts (vid):
      if transcript.is_translatable:
        ret = transcript.translate(cc).fetch()
  return ret
def buildYamls (vid):
  if vid == '': return
  global store_en, store_xx
  # create info yaml if not exists
  if not os.path.exists (f'{vid}_info.yml'):
    if not vid in videoInfos:
      videoInfo = Video.getInfo(video_prefix+vid)
      description = videoInfo['description']

      # parse chapter list / infoYml
      chapters        = parseChaptersFromDescription (description)
      videoInfos[vid] = {'description':description, 'chapters':chapters, 'aichapters': {}}

    # write infoYml
    yaml.dump (videoInfos[vid], open(f'{vid}_info.yml', 'w'))
    print (f"\r\x1b[34mwrite {vid}_info.yml",end=' | ')

  # load infoYml
  else:
    if not vid in videoInfos:
      videoInfos[vid] = yaml.load(open(f"{vid}_info.yml", 'r'), Loader=yaml.FullLoader)
      print (f"\r\x1b[34mload {vid}_info.yml",end=' | ')
    else: print (f"\r\x1b[34mmemory {vid}_info.yml",end=' | ')

  # create en transcription if not exists
  if not os.path.exists (f"{vid}_en.yml"):
    # get transcriptions from youtube / write to yaml
    enTrans[vid] = YouTubeTranscriptApi.get_transcript (vid)
    yaml.dump (enTrans[vid], open(f'{vid}_en.yml', 'w'))
  # load en transcription
  else:
    if not vid in enTrans:
      enTrans[vid] = enYml = yaml.load(open(f"{vid}_en.yml", 'r'), Loader=yaml.FullLoader)
      print (f"\x1b[34mload {vid}_en",end=' | ')
    else: print (f"\x1b[34mmemory {vid}_en",end=' | ')

  # create xx transcription if not exists
  if not os.path.exists (f"{vid}_{cc}.yml"):
    # get transcriptions from youtube / transcriptions-dict / write to yaml
    xxTrans[vid] = getTraTra (vid,cc)
    yaml.dump (xxTrans[vid], open(f'{vid}_{cc}.yml', 'w'))
  # load xx transcription
  else:
    if not vid in xxTrans:
      xxTrans[vid] = xxYml = yaml.load(open(f"{vid}_{cc}.yml", 'r'), Loader=yaml.FullLoader)
      print (f"\x1b[34mload {vid}_{cc}.yml",end='')
    else: print (f"\x1b[34mmemory {vid}_{cc}.yml",end='')

  # loops
  try:
    loops_yaml, store_en  = yaml.load(open(f'{vid}_loops_en.yml', 'r'), Loader=yaml.FullLoader), store_en[:2]
    for ts in loops_yaml: store_en.append (loops_yaml[ts])
  except: pass
  try:
    loops_yaml, store_xx  = yaml.load(open(f'{vid}_loops_{cc}.yml', 'r'), Loader=yaml.FullLoader), store_xx[:2]
    for ts in loops_yaml: store_xx.append (loops_yaml[ts])
  except: pass


def selectVideo (_):
  # if help or prompt editor do nothing
  if mainAccordion.selected_index == None or mainAccordion.selected_index >= len(mainAccordion.children) - 2: return

  # set widgets and actual infos
  globalize()
  if vid == '': return

  # log line / disable rb-selection during work
  print ('\r\x1b[35mplease wait a moment ...',end='')
  rb_vidsel.disabled = True

  # check if single v mode
  if 'no playlist' == plid: tx_singvid.value = vid

  # create or load yamls
  buildYamls (vid)

  # chapters / tooltip
  chapters = videoInfos[vid]['chapters']
  bu_allchp.tooltip = '>>>>Description from YouTube>>>\n'+str(videoInfos[vid]['description'])

  # only if the video changed inside a playlist box
  if vPositions[acIndex] != rbIndex:

    # reset
    resetTransArea ()

    # new chapter buttons
    for child in bx_chpsel.children: del child
    bx_chpsel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                                  tooltip      = c,
                                  layout       = Layout(width='auto', height='21px'))
                          for i,c in enumerate (chapters)]
    for button in bx_chpsel.children: button.on_click(chapbutClick)
    vPositions[acIndex] = rbIndex

    # build area buttons en / xx
    hb_enSele.children[0].children = [bu_enView, bu_enResu,*xareaSelectors('en')]
    hb_xxSele.children[0].children = [bu_xxView, bu_xxResu,*xareaSelectors(cc)]

    # draw / undraw selection border
    for bu in hb_enSele.children[0].children: bu.layout = Layout (width='auto', height='15px')
    hb_enSele.children[0].children[0].layout = Layout (width='auto', height='15px', border='2px solid')
    for bu in hb_xxSele.children[0].children: bu.layout = Layout (width='auto', height='15px')
    hb_xxSele.children[0].children[0].layout = Layout (width='auto', height='15px', border='2px solid')


  # one time geting country codes - note: the language names are in the language of your google account.
  if not '\nAvailable country codes:\n' in ta_helper.value:
    countryCodes = {}
    for transcript in YouTubeTranscriptApi.list_transcripts(vid):
      for l in transcript.translation_languages: countryCodes[l['language_code']] = l['language']
    ta_helper.value += "\nAvailable country codes:\n\n"+'\n'.join([', '.join([c for c in countryCodes][i:i+20]) for i in range (0,len([c for c in countryCodes]),20)])

  # re-enable rb's
  rb_vidsel.disabled = False


def xareaSelectors (ccode):
  if ccode == 'en': areaSelect_cc = areaSelect_en
  else:             areaSelect_cc = areaSelect_xx
  # build area buttons en / xx
  buttons = []
  try:
    loops_yaml = yaml.load(open(f'{vid}_loops_{ccode}.yml', 'r'), Loader=yaml.FullLoader)
    buttons    = [Button (layout=Layout(width='auto', height='15px'),
                          style={'button_color':'hotpink'},
                          tooltip=f"{id} - {loops_yaml[id].split('prompt <')[1].split('> for ...')[0]}" )
               for id in loops_yaml]
    for b in buttons: b.on_click (areaSelect_cc)
  except: pass
  return buttons


####################################################################################################################################
############################################################ main ##################################################################
####################################################################################################################################
# prompt editor widgets
prompt_buttons            = [Button (description  = p,
                                     tooltip      = str(prompts_yaml[p]['en']) if 'en' in prompts_yaml[p] else '' + '\n' +
                                                    str(prompts_yaml[p][cc])   if cc   in prompts_yaml[p] else '',
                                     style        = {'button_color':'lightgreen'}) for p in prompts_yaml if p[0] != '_']
rb_prompts, bu_addprmt    = RadioButtons(options=[p for p in prompts_yaml]), Button (description='add prompt',tooltip='add empty prompt')
bu_delprmt, bu_savprmt    = Button (description='del prompt',tooltip='delete prompt'), Button (description='save',tooltip='save prompt')
bu_transla                = Button (description='translate',tooltip='translate prompt if not empty')
hb_probut, tx_protit      = HBox (children=[bu_addprmt,bu_delprmt,bu_savprmt,bu_transla]), Text (layout=Layout(width='auto'))
hb_protxt                 = HBox (children=[Textarea(layout=Layout(width='50%',height='400px')),
                                            Textarea(layout=Layout(width='50%',height='400px'))])
vb_proedi                 = VBox (children=[hb_probut,tx_protit,hb_protxt])
gb_proedi                 = GridBox (children=[rb_prompts,vb_proedi],
                                     layout=Layout (grid_template_rows='auto',
                                                    grid_template_columns='150px auto',
                                                    grid_template_areas='"sidebar main"'))
# events
for b in prompt_buttons: b.on_click (promptButtonClick)
bu_addprmt                .on_click (prompt_add)
bu_savprmt                .on_click (prompt_save)
bu_transla                .on_click (prompt_translate)
bu_delprmt                .on_click (prompt_del)
rb_prompts                .observe  (promptSelect,names=['value'])
# show first
promptSelect(None)

# help tab
ta_helper = Textarea(value=helptext.replace('\n*','\n\n*'),layout=Layout(width='auto',height='600px'))

# build and config widgets
mainLayout    = Layout    (grid_template_rows='auto', grid_template_columns='150px auto', grid_template_areas='"sidebar main"')
mainAccordion = Accordion (children = [GridBox(layout=mainLayout) for all in courses] if len (courses) else [GridBox(layout=mainLayout)])
mainAccordion.children = tuple([*mainAccordion.children,gb_proedi,ta_helper])
mainAccordion.set_title(len(mainAccordion.children)-2,'prompt editor')
mainAccordion.set_title(len(mainAccordion.children)-1,'help')

# the pos of selected video inside a playlist box
vPositions = len(courses)*[-1]

# loop to build playlist related widgets structure - the object-names are used consistently
# and will be set in globalize() to the current displayed widgets: accordion tab / selected video
for i,title in enumerate(courses):

  # the widgets for a playlist
  mainAccordion.set_title(i,f'Playlist: {title}')
  pl        = playlists_yaml[courses[title]]
  gb_maingb = mainAccordion.children[i]
  wi_firstw = HTML          (f'<a href="{playlist_prefix}{courses[title]}" title="{playlist_prefix}{courses[title]}" target="_blank">youtube</a>')
  rb_vidsel = RadioButtons  (options=[v[12:] if len (v) > 12 else v for v in pl['videos']],layout=Layout(height='300px',overflow='scroll'))
  bu_allchp = Button        (description='all >>>>',tooltip='',layout=Layout(width='auto'))
  bu_frodes = Button        (description='from description',tooltip='Try parsing chapters out of video description.',
                             layout=Layout(width='auto'),style={'button_color':'powderblue'})
  bu_autchp = Button        (description='auto',tooltip='Division of the video into sections with max. tokens.',
                             layout=Layout(width='60%'),style={'button_color':'powderblue'})
  hb_autchp = HBox          (children=[Text (value='500',layout=Layout(width='40%')),bu_autchp])
  bu_fromai = Button        (description='from ai',tooltip='Show the ai generated chapters',
                             layout=Layout(width='50%'),style={'button_color':'powderblue'})
  bu_aichap = Button        (description='aichaps',tooltip='(Re)build the ai chapters - uses the token len from auto as chapter size.',
                             layout=Layout(width='50%'),style={'button_color':'hotpink'},disabled=not yes_i_have_this_chatgpt_openai_account)
  hb_aichbu = HBox          (children=[bu_fromai,bu_aichap])
  bu_cleanb = Button        (description='clean',tooltip='Comma seperated the number of generated timestamps and the max line lenght.',
                             layout=Layout(width='60%'))
  hb_cleanb = HBox          (children=[Text (value='3, 70',layout=Layout(width='40%')),bu_cleanb])
  #                                        0        1         2          3         4        5         6
  vb_vidsel = VBox          (children=[wi_firstw,rb_vidsel,bu_allchp,hb_cleanb,bu_frodes,hb_autchp,hb_aichbu,*prompt_buttons],layout=Layout(overflow="hidden"))
  bx_chpsel = Box           (layout=Layout(display='flex', flex_flow='wrap'))
  ta_enTran = Textarea      (layout=Layout(width='50%',height='400px'))
  ta_xxTran = Textarea      (layout=Layout(width='50%',height='400px'))
  hb_transc = HBox          (children=[ta_enTran,ta_xxTran])
  bu_enShow = Button        (description='show', style={'button_color':'lightgreen'}, layout=Layout(width='10%'),
                             tooltip='paste the prompt at the top of the textarea - do this before send')
  bu_enSend = Button        (description='send', style={'button_color':'orange'}, layout=Layout(width='10%'),
                             tooltip='send the hole textarea content to gpt and shows response - be careful it sends everything that stands there')
  bu_enLoop = Button        (description='loop', style={'button_color':'hotpink'}, layout=Layout(width='10%'),
                             tooltip='make auto chapters and loop over all to: clean it (0,60), send with selected prompt to gpt and show response in new loop area')
  bu_xxShow = Button        (description='show', style={'button_color':'lightgreen'}, layout=Layout(width='10%'),
                             tooltip=bu_enShow.tooltip)
  bu_xxSend = Button        (description='send', style={'button_color':'orange'}, layout=Layout(width='10%'),disabled=not yes_i_have_this_chatgpt_openai_account,
                             tooltip=bu_enSend.tooltip)
  bu_xxLoop = Button        (description='lopp', style={'button_color':'hotpink'}, layout=Layout(width='10%'),disabled=not yes_i_have_this_chatgpt_openai_account,
                             tooltip=bu_enLoop.tooltip)
  bu_enView = Button        (layout=Layout(width='auto', height='15px', border='2px solid'), style={'button_color':'lightgreen'},
                             tooltip='Area for the prompt: first select text than select prompt and show.')
  bu_enResu = Button        (layout=Layout(width='auto', height='15px'), style={'button_color':'orange'},
                             tooltip='Area with the result from gpt for the sended prompt.')
  bu_enLdel = Button        (layout=Layout(width='auto', height='15px'), style={'button_color':'Crimson'},
                             tooltip='delete selected loop out off the yaml')
  bu_xxView = Button        (layout=Layout(width='auto', height='15px', border='2px solid'), style={'button_color':'lightgreen'}, tooltip=bu_enView.tooltip)
  bu_xxResu = Button        (layout=Layout(width='auto', height='15px'), style={'button_color':'orange'},tooltip=bu_enResu.tooltip)
  bu_xxLdel = Button        (layout=Layout(width='auto', height='15px'), style={'button_color':'Crimson'},tooltip=bu_enLdel.tooltip)
  hb_enSele = HBox          (children=[Box(children=[bu_enView, bu_enResu],
                                           layout=Layout(display='flex', flex_flow='row', border='1px solid', width='95%')),
                                       Box(children=[bu_enLdel],
                                           layout=Layout(display='flex', flex_flow='row', border='1px solid', width='5%')) ])
  hb_xxSele = HBox          (children=[Box(children=[bu_xxView, bu_xxResu],
                                           layout=Layout(display='flex', flex_flow='row', border='1px solid', width='95%')),
                                       Box(children=[bu_xxLdel],
                                           layout=Layout(display='flex', flex_flow='row', border='1px solid', width='5%')) ])
  hb_enCtrl = HBox          (children=[HTML(value='',layout=Layout(width='10%')),
                                       Label(value='<<<select prompt>>>',layout=Layout(width='40%')),
                                       Text(description='°',value='0.1',layout=Layout(width='20%'),tooltip='temperature for api call'),
                                       bu_enShow, bu_enSend, bu_enLoop], layout=Layout(border='1px solid black',width='auto'))
  hb_xxCtrl = HBox          (children=[HTML(value='',layout=Layout(width='10%')),
                                       Label(value='<<<select prompt>>>',layout=Layout(width='40%')),
                                       Text(description='°',value='0.1',layout=Layout(width='20%')),
                                       bu_xxShow, bu_xxSend, bu_xxLoop], layout=Layout(border='1px solid black',width='auto'))
  vb_enCtrl = VBox          (children=[hb_enCtrl,hb_enSele],layout=Layout(width='50%'))
  vb_xxCtrl = VBox          (children=[hb_xxCtrl,hb_xxSele],layout=Layout(width='50%'))
  hb_contrl = HBox          (children=[vb_enCtrl,vb_xxCtrl])
  vb_chpare = VBox          (children=[bx_chpsel,hb_contrl,hb_transc])
  gb_maingb                 .children = [vb_vidsel,vb_chpare]

  # events
  rb_vidsel                 .observe  (selectVideo, names=['value'])
  mainAccordion             .observe  (selectVideo, names=['selected_index'])
  bu_allchp                 .on_click (chapbutClick)
  bu_enShow                 .on_click (enShowButtonClick)
  bu_xxShow                 .on_click (xxShowButtonClick)
  bu_enSend                 .on_click (enSendButtonClick)
  bu_xxSend                 .on_click (xxSendButtonClick)
  bu_enLoop                 .on_click (enLoopButtonClick)
  bu_xxLoop                 .on_click (xxLoopButtonClick)
  bu_autchp                 .on_click (autoButtonClick)
  bu_frodes                 .on_click (fromDescriptionButtonClick)
  bu_cleanb                 .on_click (cleanButtonClick)
  bu_fromai                 .on_click (aiShowButtonClick)
  bu_aichap                 .on_click (aiBuildButtonClick)
  ta_enTran                 .observe  (text_change, names=['value'])
  ta_xxTran                 .observe  (text_change, names=['value'])
  bu_enView                 .on_click (areaSelect_en)
  bu_enResu                 .on_click (areaSelect_en)
  bu_xxView                 .on_click (areaSelect_xx)
  bu_xxResu                 .on_click (areaSelect_xx)
  bu_enLdel                 .on_click (loopDelete_en)
  bu_xxLdel                 .on_click (loopDelete_xx)

# add / del / id input for single videos - no need for globalize
if 'no playlist' in playlists_yaml:
  tx_singvid, bu_singvid = Text (layout=Layout(width='100px')), Button (description='+',tooltip='delete known, add unknown')

  # kick wi_firstw HTML - replace with HBox
  vb_vidsel.children = [HBox (children=[tx_singvid, bu_singvid],layout=Layout(width='auto')), *vb_vidsel.children[1:]]

  # events
  def tx_singvid_change (_):
    bu_singvid.disabled    = len (tx_singvid.value) != 11
    bu_singvid.description = '-' if tx_singvid.value+' ' in playlists_yaml['no playlist']['videos'] else '+'

  def bu_singvid_click (b):
    vl = playlists_yaml['no playlist']['videos']
    if b.description == '+': vl.append (tx_singvid.value+' ')
    if b.description == '-':
      for filtered in [name for name in os.listdir() if tx_singvid.value in name]: os.remove (filtered)
      vl.remove (tx_singvid.value+' ')
    rb_vidsel.options=[v for v in vl]
    playlists_yaml['no playlist']['videos'] = vl
    yaml.dump (playlists_yaml, open('playlists.yml', 'w'))
    selectVideo(None)
    fromDescriptionButtonClick (bu_frodes)

  tx_singvid.observe (tx_singvid_change, names=['value'])
  bu_singvid.on_click (bu_singvid_click)

# display / globalize / unexpand accordion
display       (mainAccordion)
globalize     ()
mainAccordion.selected_index = None




# _sp: openai models

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**openai models**</font><br>
#@markdown * This little snippet uses the class ```openai.Model``` to get the actual model list.
#@markdown * It create a tab for every model base with all associated model ids.
#@markdown * Youn can select the model-id to view the string representation with all parameters.
from ipywidgets import HTML, Tab, HBox, RadioButtons, Textarea, Layout
import os, getpass

tmphm = HTML("""Get <a href='https://platform.openai.com/account/api-keys' target='_blank'>here</a> your API-Key.""")
display (tmphm)
try:     import openai
except:  os.system ('pip install openai'); import openai
try:     myApiKey = os.environ["OPENAI_API_KEY"]
except:  pass
try:     os.environ["OPENAI_API_KEY"] = openai.api_key = myApiKey
except:  os.environ["OPENAI_API_KEY"] = openai.api_key = getpass.getpass (prompt='Your API key: ')
tmphm.value = """<font color='green'>Set API-key done!</font>"""


# create dict with base name as key and the associated models list as value
ML = openai.models.list()
ids, models = [m.id for m in ML], {}
for all in set ([id.split('-')[0].split(':')[0] for id in ids]):
  models[all] = [model for model in ids if all == model.split('-')[0].split(':')[0]]

# little helper
def model_by_id (id):
  return 'None' if not id in [i.id for i in ML] else str([all for all in ML if all.id == id][0])
def some_change (c):
  selectedBox   = tab.children[tab.selected_index]
  tmpRB, tmpTA  = selectedBox.children[0], selectedBox.children[1]
  tmpTA.value   = model_by_id (tmpRB.value)

# create tab for every model base
tab = Tab (children=[HBox() for all in models])

# fill tabs
for i, title in enumerate(models):
  tmpRB         = RadioButtons  (options=[id for id in models[title]])
  tmpRB         .observe        (some_change, names=['value'])
  tmpTA         = Textarea      (layout=Layout(width='600px',height='500px'))
  tmpTA.value   = model_by_id   (tmpRB.value)

  tab.children[i].children = [tmpRB,tmpTA]
  tab.set_title (i,title)

print (f'Models total: {len(list(ML))}')
display (tab)



# _sp: chat gpt

In [ ]:
# *******************************************************************************************************************************************************************************************
# *** Colab Forms ***************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************

#@markdown <font size='+2' color='#005F6A'>**chatGPT**</font> <font size='-1'>from <b>openai</b></font>
#@markdown <br>A little **gui** for chat gpt with optional **yaml** persistence on **gdrive** or **local runtime** or both **synchronized**
#@markdown * This snippet needs a **personal api key** ([pricing](https://openai.com/pricing)) - you can get it [here](https://platform.openai.com/account/api-keys). You can control your [billing](https://platform.openai.com/settings/organization/billing/overview), [usage](https://platform.openai.com/account/usage) and [limitations](https://platform.openai.com/account/billing/limits).
#@markdown * There are [many models](https://platform.openai.com/docs/models) to choose from and a big [community](https://community.deeplearning.ai/). To dive deeper into **prompt engineering** there is a litte [course from deeplearning.ai](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers).
#@markdown * You can use **categories** to **organize** your dialogs (its **title** is generated **automatically**) - every categorie has a **subfolder** with its own yamls.
#@markdown * A **```chatgpt```** folder is created/used to store yamls and synchronize. Depending on the situation its located in:
#@markdown >* Local: ```./``` or ```<local_cloud_folder>/``` (if set in forms)
#@markdown >* Hosted: ```/content``` or ```/gdrive/MyDrive/``` (if gdrive checked in gui)
#@markdown * You have an **optional gdrive** support (local via [rclone](https://rclone.org) and hosted via colab gdrive). If you put a <b>local gdrive folder</b>, at start it <b>automaticly sync gdrive</b><br>to local and do a sync to gdrive by each 'send'. Sometimes you need to **reconnect rclone** with: ```rclone config reconnect gdrive:```
prefered_language  = 'German'   #@param {type:"string"}
prompt_title  = "You are an assistant and you are creating a title for the following chat history. The title should list all topics of the chat history well. Your output must have less than 9 words and is only the title and everything in {prefered_language}." #@param {type:"string"}
prompt_system = "You are a friendly assistant, you receive questions or instructions in any language and respond in {prefered_language}. You try to break down your instructions into steps, work through them and check the results of each step before presenting the overall result. If mathematical formulas are included in the answer, they are displayed in markdown format for jupyter notebooks, for example: $$y=x^\\frac{1}{2}$$ or $y=x^\\frac{1}{2}$ results in y=x^1/2 in tex view (enclosed in '$' for included in text or enclosed in '$$' to be displayed in its own line)." #@param {type:"string"}
prompt_title  = prompt_title.replace  ('{prefered_language}',prefered_language)
prompt_system = prompt_system.replace ('{prefered_language}',prefered_language)
#@markdown ---
#@markdown <font color='black' size='+1'><b>Local runtime</b></font><br><font color='#005F6A'>The location of the chatgpt-folder. In **hosted runtime** this parameter are **ignored**.</font>
local_cloud_folder = '~/labor/gdrive'   #@param {type:"string"}
rclone_name  = 'gdrive:' # fix use
#@markdown <font color='black' size='+1'><b>Chat</b></font><br><font color='#005F6A'>If **categorie_name** is not empty, it is used as a **subfolder** in the gptchat folder and its content is displayed in the **recent chats** selector.</font>
categorie_name  = 'gabi'   #@param {type:"string"}



# *******************************************************************************************************************************************************************************************
# *** Imports *******************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
import                    os, subprocess, getpass, yaml, time
from ipywidgets   import  HTML, VBox, Box, Tab, Textarea, Button, Layout, Dropdown, HBox, Text, Checkbox, Label, Accordion
from datetime     import  datetime
from pathlib      import  Path



# *******************************************************************************************************************************************************************************************
# *** snippet ButtonBox *********************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
class ButtonBox ():
  """ An alternative for the Tab widget
  """
  def _clicker (self, b):
    # search clicked button and remember
    for i, description in enumerate (self.descriptions):
      if b.tooltip == description:
        self.selected = i
        self.button   = b

    # unselect (color) all buttons and select new (list of colors here: https://www.quackit.com/css/css_color_codes.cfm)
    for b in self.buttons: b.style={'button_color':None}
    self.buttons[self.selected].style={'button_color':self.color}

    # fire event
    if self.clicker: self.clicker (self)

  def __init__ (self, descriptions, clicker=None, maxchar=80, color='powderblue'):
    # remember stuff
    self.clicker, self.maxchar, self.color, self.selected, self.button = clicker, maxchar, color, None, None

    # build buttons
    self.buildButtons (descriptions)

  def buildButtons (self, descriptions):
    # list to sorted set
    self.descriptions = sorted (descriptions)

    # make buttons
    self.buttons = [Button (description = i if len (i) <= self.maxchar else f'{i[:(self.maxchar-3)]}...',
                            layout      = Layout(width='auto', height='21px'),
                            tooltip     = f'{i}')
                    for i in self.descriptions]

    # put them in a box
    self.box = Box (layout = Layout (display='flex', flex_flow='wrap'), children = self.buttons)

    # bind event
    for button in self.buttons: button.on_click (self._clicker)

  def removeSelected (self):
    if self.selected != None:
      self.box.children = [*self.box.children[:self.selected], *self.box.children[self.selected+1:]]
      self.button = self.selected = None




# *******************************************************************************************************************************************************************************************
# *** class *********************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
class ChatGPT:
  """ A little chat client with yaml persistence
  widget-structure:
  hm_state              - HTML      - link api-key / done message
  ac_main               - Accordion - main accordion with 'new chat' and 'recent chats'
    + child0            - VBox      - recent chats
      + child0          - Box       - chat selector (managed by ButtonBox)
      + child1          - HTML      - cosmetic <hr>
      + hb_tools        - HBox      - Toolbar with buttons and html
      |   + child0      - HTML      - yaml filename
      |   + child1      - Box       - cosmetic (for html left, buttons right)
      |   + bu_conChat  - Button    - continue chat
      |   + bu_delChat  - Button    - delete chat
      + child3          - TextArea  - display chat
    + child1 VBox       - VBox      - new chat
      + hb_config       - HBox      - the config line
      |   + child0      - Dropdown  - the models
      |   + child1      - Text      - temperature
      |   + child2      - Checkbox  - write to yaml
      |   + child3      - Checkbox  - use cloud / local runtime
      |   + child4      - HTML      - filename and location yaml
  |   + ta_system       - TextArea  - system instruction
      + ta_titprm       - TextArea  - title prompt
      + child3          - HTML      - cosmetic <hr>
      + vb_main         - VBox      - growing Textareas
      |   + ta_user     - TextArea  - first question
      |   *+ child      - TextArea  - assistant (response)    # current last two textareas in list are
      |    + child      - TextArea  - user (next question)    # accessable in self.ta_assist and self.ta_user
      + hb_control      - HBox      - send button and waiter
          + bu_send     - Button    - send
          + cb_gentit   - Checkbox  - generate title
          + hm_waiter   - HTML      - wait animation
  """
# *******************************************************************************************************************************************************************************************
# *** init / helper *************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
  def __init__(self, local_cloud_folder=None, rclone_name=None, categorie_name=None):
    self.hm_state   = HTML      ("""<font size='+1' color='purple'><a href='https://platform.openai.com/account/api-keys'>api key here</a> - copy and paste your key in the input field.</font>""")
    self.bu_send    = Button    (description='>>', layout=Layout(width='60px'), style={'button_color':'lightgreen'}, tooltip='Send user prompt to gpt, show answer and write yaml if selected.')
    self.cb_gentit  = Checkbox  (description='generate title', value=True, disabled=True, tooltip='If checked it will generate a new title for this chat.')
    self.ta_system  = Textarea  (description='system:', value=prompt_system, layout=Layout(width='auto',height='80px'))
    self.ta_titprm  = Textarea  (description='title prompt:', value=prompt_title, layout=Layout(width='auto',height='80px'))
    self.ta_user    = Textarea  (description='user:', value='tell a joke', layout=Layout(width='auto',height='300px'))
    self.vb_main    = VBox      (children=[self.ta_user], layout=Layout(width='1200px'))
    self.hb_config  = HBox      ([Dropdown (description='model:'),
                                  Text     (description='temperature', value='0.0', layout=Layout(width='130px')),
                                  Checkbox (description='use gdrive', layout=Layout(width='220px')),
                                  Checkbox (description='write yaml', layout=Layout(width='180px')),
                                  HTML     (value='') ])
    self.hm_waiter  = HTML      ("""<marquee style='width: 100%; color: blue;'>waiting for response...</marquee>""",layout=Layout(width='200px'))
    self.hb_control = HBox      ([self.bu_send,self.cb_gentit],layout=Layout(width='auto'))
    self.bu_conChat = Button    (description='continue chat', disabled=True,style={'button_color':'khaki'})
    self.bu_delChat = Button    (description='delete chat', disabled=True,style={'button_color':'khaki'})
    self.hb_tools   = HBox      ([HTML(), Box(layout=Layout(flex='1 1 auto')), self.bu_conChat, self.bu_delChat])
    self.ac_main    = Accordion (children=[VBox(), VBox ([self.hb_config, self.ta_system, self.ta_titprm, HTML('<hr>'), self.vb_main, self.hb_control])])
    self.ac_main      .set_title  (0,f'recent chats - {categorie_name}' if categorie_name and len (categorie_name) else 'recent chats')
    self.ac_main      .set_title  (1,'new chat')
    self.ac_main      .selected_index = None

    # init / display state-html
    self.messages, self.yamlFile, self.chats, self.local_cloud_folder, self.rclone_name = [], '', {}, local_cloud_folder, rclone_name
    self.categorie_name = '/'+categorie_name if categorie_name and len (categorie_name) else ''
    display (self.hm_state)

    # runtime
    self.initializeRuntime ()

    # display chat home
    self.hb_config.children[-1].value = f'&nbsp;&nbsp;&nbsp; in &nbsp; {self.chatHome}'

    # local runtime
    if self.isLocal:
      self.hb_config.children[2].description = 'local runtime'
      self.hb_config.children[2].value       = True
      self.hb_config.children[2].disabled    = True

    # load yaml / create chat selector / connect openai
    try:    self.chats = yaml.safe_load (open(f'{self.chatHome}/chats.yaml', 'r', encoding='unicode_escape'))
    except: self.chats = {}
    self.createChatSelector()
    self.do_openai()

    # bind events
    if not self.hb_config.children[2].disabled:
      self.hb_config.children[2] .observe   (self.useGdrive,   names=['value'])
    self.hb_config.children[3]   .observe   (self.writeToYaml, names=['value'])
    self.bu_send                 .on_click  (self.send_click)
    self.bu_conChat              .on_click  (self.continue_click)
    self.bu_delChat              .on_click  (self.delete_click)

    # display main accordion
    display (self.ac_main)


  def initializeRuntime (self):
    # solve ~ in cloud folder / check: local runtrime / rclone installed / valid remote name / valid local cloud folder
    if '~' in self.local_cloud_folder: self.local_cloud_folder = self.local_cloud_folder.replace ('~', os.path.expanduser('~'))
    self.isLocal   = (os.path.expanduser('~') != '/root')
    self.doRclone  = (os.system ('rclone -V >/dev/null 2>&1')==0 and self.rclone_name!='' and self.local_cloud_folder!='' and self.isLocal)
    if self.doRclone and not self.rclone_name in str(subprocess.check_output("rclone listremotes", shell=True)):
      raise Exception (f'\x1b[91m{self.rclone_name} is no valid remote name')
    if self.doRclone and not Path(self.local_cloud_folder).exists():
      raise Exception (f'\x1b[91mfolder {self.local_cloud_folder} does not exist')

    # chat home folder
    if self.isLocal:  self.chatHome = '.' if not self.doRclone else self.local_cloud_folder
    else:             self.chatHome = '/content'
    if not 'gptchat' in os.listdir (f'{self.chatHome}'):  os.mkdir (f'{self.chatHome}/gptchat')
    if not 'gptchat' in self.chatHome:                    self.chatHome += f'/gptchat{self.categorie_name}'

    # sync
    if self.isLocal and self.doRclone and 'gptchat' in str (subprocess.check_output(f"rclone lsd  {self.rclone_name}", shell=True)):
      try: os.system(f"rclone sync {self.rclone_name}/gptchat {self.local_cloud_folder}/gptchat")
      except: pass


  def createChatSelector (self):
    # create chat selector with recent chats and textarea to display
    self.bb_selrecent = ButtonBox (set([self.chats[fname]['name'] for fname in self.chats]), clicker=self.chat_click)
    self.ac_main.children[0].children = [self.bb_selrecent.box, HTML(value='<hr>'), self.hb_tools,
                                         Textarea(layout=Layout(width='auto',height='600px'))]


# *******************************************************************************************************************************************************************************************
# *** events buttons +++++++++++++***********************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
  def delete_click (self, b):
    # remove button / buttons disable / clear textarea
    self.bb_selrecent.removeSelected ()
    self.bu_conChat.disabled = self.bu_delChat.disabled = True
    self.ac_main.children[0].children[3].value = self.hb_tools.children[0].value = ''

    # delete file & intern yamls / rewrite chats.yaml / sync removing
    os.system (f'rm {self.chatHome}/{self.fname}')
    del self.chats[self.fname]
    yaml.dump (self.chats, open (f'{self.chatHome}/chats.yaml',  'w'))
    if self.rclone_name and os.path.expanduser('~') != '/root':
      try:    os.system(f"rclone sync {self.local_cloud_folder}/gptchat {self.rclone_name}/gptchat")
      except: pass

    # special: continue clicked before delete
    if self.yamlFile == self.fname:
      self.ac_main.set_title(1,'new chat')
      self.hb_config.children[3].value, self.hb_config.children[3].disabled = False, False
      self.hb_config.children[3].observe (self.writeToYaml, names=['value'])
      self.yamlFile          = ''
      self.ta_user           = Textarea (description='user:', value='', layout=Layout(width='auto',height='300px'))
      self.vb_main.children  = [self.ta_user]


  def continue_click (self, b):
    # new title / checkbox writeyaml
    self.ac_main.set_title(1,self.bb_selrecent.button.tooltip)
    self.hb_config.children[3].unobserve_all()
    self.hb_config.children[3].value, self.hb_config.children[3].disabled = True, True

    # set new yaml
    self.yamlFile = f'{self.fname}'
    try:     self.messages = yaml.safe_load (open(f'{self.chatHome}/{self.fname}', 'r'))
    except:  self.messages = {}

    # rebuild textareas
    self.ta_system.value = self.messages[0]['content']
    taList = []
    for i in range (len(self.messages)-2):
      taList.append ( Textarea (description=self.messages[i+1]['role'], value=self.messages[i+1]['content'], layout=Layout(width='auto',height='65px')) )
    self.ta_assist  = Textarea (description='assistant:', value=self.messages[-1]['content'], layout=Layout(width='auto',height='300px'))
    self.ta_user    = Textarea (description='user:', value='', layout=Layout(width='auto',height='300px'))
    self.vb_main.children = [*taList, self.ta_assist, self.ta_user]

    # select chat accordion tab
    self.ac_main.selected_index = 1


  # send button click
  def send_click (self, b):
    # waiter
    b.disabled, self.hb_control.children = True, [self.bu_send, self.hm_waiter]

    # init messages
    if len (self.messages) == 0:  self.messages = [{'role':'system', 'content':f'{self.ta_system.value}'}]

    # add new question and its response to messages
    self.messages.append ({'role':'user', 'content':self.vb_main.children[-1].value})
    response = self.get_completion_from_messages (self.messages,
                                                  model       = self.hb_config.children[0].value,
                                                  temperature = float(self.hb_config.children[1].value)
                                                 ).choices[0].message.content
    self.messages.append ({'role':'assistant', 'content':f'{response}'})

    # make old textareas small and create new ones
    for ta in self.vb_main.children: ta.layout = Layout(width='auto',height='65px')
    self.ta_assist         = Textarea (description='assistant:', value=response, layout=Layout(width='auto',height='300px'))
    self.ta_user           = Textarea (description='user:', value='', layout=Layout(width='auto',height='300px'))
    self.vb_main.children  = [*self.vb_main.children,self.ta_assist,self.ta_user]

    # reset waiter / ask title / write yaml
    b.disabled, self.cb_gentit.disabled, self.hb_control.children = False, False, [self.bu_send, self.cb_gentit]
    if self.cb_gentit.value:
      tmpSys   = {'role':'system', 'content':self.ta_titprm.value}
      response = self.get_completion_from_messages ([tmpSys, *self.messages[1:]],
                                                    model       = "gpt-3.5-turbo", # self.hb_config.children[0].value,
                                                    temperature = float(self.hb_config.children[1].value)
                                                  ).choices[0].message.content
      while response[0]  in """ !§$%&()=?`'{}[]@~+*<>|\/-:"'#.,;_""": response = response[1:]
      while response[-1] in """ !§$%&()=?`'{}[]@~+*<>|\/-:"'#.,;_""": response = response[:-1]
      self.ac_main.set_title  (1,f'{response[:100]}')
    self.writeToYaml (None)


  def chat_click (self, bb):
    # create content for textarea
    fname    = [fn for fn in self.chats if self.chats[fn]['name'] == self.bb_selrecent.button.tooltip][0]
    recent   = yaml.safe_load (open(f'{self.chatHome}/{fname}', 'r'))
    con      = f"{self.chats[fname]['name']}\n\n"
    con     += f"File:       {fname}\n"
    con     += f"Model:      {self.chats[fname]['model']}\n"
    con     += f"Temperatur: {self.chats[fname]['temp']}\n\n"
    con     += f"System: {recent[0]['content']}\n\n"
    con     += f"\n{'*'*300}\n\n"
    for i in recent[1:]:
      con   += f">>>{i['role']}>>>\n"
      con   += f"{i['content']}\n"
      con   += "\n" if i['role'] != 'assistant' else f"\n{'*'*300}\n\n"
    self.ac_main.children[0].children[3].value = con

    # enable buttons / remember + show filename
    self.bu_conChat.disabled = self.bu_delChat.disabled = False
    self.fname = fname
    self.hb_tools.children[0].value = f"<font size='+2' color='Blue'>{fname}</font>"



# *******************************************************************************************************************************************************************************************
# *** yaml / gdrive / openai ****************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
  # checkbox write yaml observe
  def writeToYaml (self,cb):

    # call by checkbox
    if cb:
      # yaml file name / disable cb
      tmp_dat = datetime.now().strftime("%Y%m%d-%H%M%S")
      self.yamlFile                       = f'gpt_chat_{tmp_dat}.yaml'
      self.hb_config.children[4].value    = f'{self.yamlFile} &nbsp;&nbsp;&nbsp; in &nbsp; {self.chatHome}'
      self.hb_config.children[3].disabled = True
      self.hb_config.children[2].disabled = True

    # write yaml
    if self.messages and self.yamlFile:
      if self.categorie_name and not Path(self.chatHome).exists():
        os.mkdir (self.chatHome)

      self.chats[self.yamlFile] = {'name'  :f"{self.yamlFile[11:17]} - {self.ac_main.get_title(1)}",
                                   'model' :f'{self.hb_config.children[0].value}',
                                   'temp'  :f'{self.hb_config.children[1].value}'}
      yaml.dump (self.messages, open (f'{self.chatHome}/{self.yamlFile}', 'w'))
      yaml.dump (self.chats,    open (f'{self.chatHome}/chats.yaml',  'w'))

      # sync local to cloud
      if self.doRclone:
        try: os.system(f"rclone sync {self.local_cloud_folder}/gptchat {self.rclone_name}/gptchat")
        except: pass


  # checkbox 'use gdrive' observe
  def useGdrive (self,b):
    # colab gdrive stuff
    try:
      from google.colab import drive
      if not 'gdrive'  in os.listdir('/'):  drive.mount ('/gdrive')
      self.chatHome = f'/gdrive/MyDrive'
    except:
      self.hb_config.children[3].value       = False
      self.hb_config.children[3].description = 'failed'
    if not 'gptchat' in os.listdir(self.chatHome): os.mkdir (f'{self.chatHome}/gptchat')
    if not 'gptchat' in self.chatHome: self.chatHome += f'/gptchat{self.categorie_name}'
    self.hb_config.children[-1].value    = f'{self.yamlFile} &nbsp;&nbsp;&nbsp; in &nbsp; {self.chatHome}'
    self.hb_config.children[2].disabled  = True

    # load chats from gdrive
    try:    self.chats = yaml.safe_load (open(f'{self.chatHome}/chats.yaml', 'r', encoding='unicode_escape'))
    except: self.chats = {}
    self.createChatSelector()

  # init openai connection
  def do_openai (self):
    # openai api key input / widget models (+default) / done state
    os.environ["OPENAI_API_KEY"]        = openai.api_key = getpass.getpass (prompt='Your API key: ') if not openai.api_key or openai.api_key == '' else openai.api_key
    self.hb_config.children[0].options  = [m.id for m in openai.models.list() if 'gpt' in m.id]
    self.hb_config.children[0].value    = 'gpt-4'
    self.chatclient                     = OpenAI()
    self.hm_state.value                 = """<font size='+2' color='powderblue'>Set API-Key done!</font>"""

  # openai endpoint
  def get_completion_from_messages (self, messages, model="gpt-3.5-turbo", temperature=0):
    """ see https://platform.openai.com/docs/api-reference/introduction """
    return self.chatclient.chat.completions.create ( model       = model,
                                                     messages    = messages,
                                                     temperature = temperature )
try:              import  openai
except:                   os.system ('pip install openai'); import openai
from     openai   import  OpenAI



# *******************************************************************************************************************************************************************************************
try:     chat = ChatGPT (local_cloud_folder=local_cloud_folder, rclone_name=rclone_name, categorie_name=categorie_name)
except:  os.environ["OPENAI_API_KEY"] = openai.api_key = ''



# _sp: command shell

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**command shell**</font>
#@markdown * using [jQuery Terminal](https://terminal.jcubic.pl/), here is a little [example](https://github.com/alessandrobessi/colab-shell) and [colab advanced outputs](https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb)
#@markdown * sometimes it doesn't display the shell (big white area) - than you have to re-run the cell again till it displays (have no idea why - seems to be quite experimental)
#
# *******************************************************************************************************************************************************************************************
# **** snippet button box *******************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
from ipywidgets import Button, Box, Layout

class ButtonBox ():
  # An alternative for the Tab widget
  def _clicker (self, b):
    # search clicked button and remember
    self.position, self.button = [(i,but) for i, but in enumerate(self.buttons) if b == but][0]

    # unselect (color) all buttons and select new (list of colors here: https://www.quackit.com/css/css_color_codes.cfm)
    for b in self.buttons: b.style={'button_color':None}
    self.buttons[self.position].style={'button_color':self.color}

    # fire event
    if self.clicker: self.clicker (self)

  def append (self, description, select=True):
    # add new selctor button at the end
    b = Button (description = description if len (description) <= self.maxchar else f'{description[:(self.maxchar-3)]}...',
                layout      = Layout(width='auto', height='21px'),
                tooltip     = f'{description}')
    self.buttons.append(b)
    self.box.children = [*self.box.children, b]

    # select new button / bind event
    if select:
      self.button, self.position = b, len(self.buttons) - 1
      self._clicker (b)
    b.on_click (self._clicker)

  def remove (self, position=None):
    # remove selected if no position given
    if not position: position = self.position
    if not position or not position < len(self.buttons) or position < 0: return

    # clean up
    self.box.children = [*self.box.children[:position], *self.box.children[position+1:]]
    self.buttons = [*self.buttons[:position], *self.buttons[position+1:]]
    self.button, self.position = None, -1

  def __init__ (self, descriptions, clicker=None, maxchar=60, color='powderblue'):
    # remember stuff - list to set
    self.descriptions = descriptions if len (descriptions) == len (set (descriptions)) else set (descriptions)
    self.clicker, self.maxchar, self.color, self.position, self.button = clicker, maxchar, color, -1, None

    # make buttons
    self.buttons = [Button (description = i if len (i) <= maxchar else f'{i[:(maxchar-3)]}...',
                            layout      = Layout(width='auto', height='21px'),
                            tooltip     = f'{i}')
                    for i in descriptions]

    # put them in a box / bind event
    self.box = Box (layout   = Layout (display='flex', flex_flow='wrap'),
                    children = self.buttons)
    for button in self.buttons: button.on_click (self._clicker)

##############################################################################################################################################
##############################################################################################################################################
# google.colab.output-Patch
# It raises: AttributeError: module 'IPython.utils.traitlets' has no attribute 'Unicode'
# I searched for a long time about this error. Finally I found a solution in this thread:
# https://stackoverflow.com/questions/64886414/i-have-a-warning-error-when-plotting-with-plotly
##############################################################################################################################################
##############################################################################################################################################
import os, time
srcfile     = os.__file__.replace ('os.py','site-packages/google/colab/data_table.py')
bad_source  = 'from IPython.utils import traitlets as _traitlets'
good_source = 'import traitlets as _traitlets'

if os.path.exists(srcfile):
  with open(srcfile, 'r') as file: content = file.read()
  if bad_source in content:
    with open(srcfile, 'w') as file: file.write(content.replace(bad_source,good_source))
    time.sleep(1)
    from google.colab import output
    with open(srcfile, 'w') as file: file.write(content)
else:
  from google.colab import output
######end_patch###############################################################################################################################
##############################################################################################################################################
from   IPython.display    import  display, HTML, JSON
from   subprocess         import  getoutput
from   ipywidgets         import  Accordion, Textarea, Output, VBox

width  = 1200 #@param {type: "slider", min: 400, max: 1900, step: 100}
height = 400 #@param {type: "slider", min: 100, max: 900, step: 100}

# function for invoke-callback-mechanism
def shell(command):
  # catch cd-command
  if command.startswith('cd'):
    os.chdir(os.path.abspath (command[3:].replace('~',os.path.expanduser ('~'))))
    ret = JSON([''])
  else:
    ret = JSON([getoutput(command)])

  # history
  historyOuptuts.append (ret.data[0])
  acMain.set_title (1,f'history - last command: {command.split(" ")[0]}')
  bbSelhist.append (command)

  return ret

# register to invoke
output.register_callback('shell', shell)
ouShell = Output(layout={'width':'auto', 'height':f'{height}px'})

# create command tab using javascript with 'display(HTML(...'
with ouShell:
  display(HTML('''
    <div id=term_demo></div>
    <script src="https://code.jquery.com/jquery-latest.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
    <link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
    <script>
      $('#term_demo').terminal(async function(command) {
          if (command !== '') {
              try {
                  let res = await google.colab.kernel.invokeFunction('shell', [command])
                  let out = res.data['application/json'][0]
                  this.echo(new String(out))
              } catch(e) {
                  this.error(new String(e));
              }
          } else {
              this.echo('');
          }
      }, {
          greetings: 'Welcome to Colab Shell',
          name: 'colab_shell',
          height: '''+str(height)+''',
          width: '''+str(width)+''',
          prompt: 'colab > '
      });
    </script>'''))

def selectHistory (bb):
  taCmdoutput.value = historyOuptuts[bb.position]

historyOuptuts = []
bbSelhist = ButtonBox ([], clicker=selectHistory)
taCmdoutput = Textarea(layout={'width':'auto', 'height':f'{height}px'})
vbHist = VBox (children=[bbSelhist.box, taCmdoutput])

acMain = Accordion (children=[ouShell, vbHist])
acMain.set_title (0,'command shell')
acMain.set_title (1,'history')
acMain.selected_index = 0

display (acMain)





# _sp: command shell simple

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**command shell simple**</font>
#@markdown * simple version: using [jQuery Terminal](https://terminal.jcubic.pl/), here is a little [example](https://github.com/alessandrobessi/colab-shell) and [colab advanced outputs](https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb)
#
##############################################################################################################################################
##############################################################################################################################################
# google.colab.output-Patch
# It raises: AttributeError: module 'IPython.utils.traitlets' has no attribute 'Unicode'
# I searched for a long time about this error. Finally I found a solution in this thread:
# https://stackoverflow.com/questions/64886414/i-have-a-warning-error-when-plotting-with-plotly
##############################################################################################################################################
##############################################################################################################################################
try:
  import os, time
  srcfile     = os.__file__.replace ('os.py','site-packages/google/colab/data_table.py')
  bad_source  = 'from IPython.utils import traitlets as _traitlets'
  good_source = 'import traitlets as _traitlets'

  if os.path.exists(srcfile):
    with open(srcfile, 'r') as file: content = file.read()
    if bad_source in content:
      with open(srcfile, 'w') as file: file.write(content.replace(bad_source,good_source))
      time.sleep(1)
      from google.colab import output
      with open(srcfile, 'w') as file: file.write(content)
except:
  from google.colab import output
##############################################################################################################################################
##############################################################################################################################################
import                            os
from   IPython.display    import  display, HTML, JSON
from   subprocess         import  getoutput

width  = 1200
height = 400

# function for invoke-callback-mechanism
def shell(command):
  # catch cd-command
  if command.startswith('cd'):
    os.chdir(os.path.abspath (command[3:].replace('~',os.path.expanduser ('~'))))
    return JSON([''])
  return JSON([getoutput(command)])

# register to invoke
output.register_callback('shell', shell)

# create command tab using javascript with 'display(HTML(...'
display(HTML('''
  <div id=term_demo></div>
  <script src="https://code.jquery.com/jquery-latest.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
  <link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
  <script>
    $('#term_demo').terminal(async function(command) {
        if (command !== '') {
            try {
                let res = await google.colab.kernel.invokeFunction('shell', [command])
                let out = res.data['application/json'][0]
                this.echo(new String(out))
            } catch(e) {
                this.error(new String(e));
            }
        } else {
            this.echo('');
        }
    }, {
        greetings: 'Welcome to Colab Shell',
        name: 'colab_shell',
        height: '''+str(height)+''',
        width: '''+str(width)+''',
        prompt: 'colab > '
    });
  </script>'''))



# _sp: image search

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**image search**</font><br>
#@markdown This little snippet does a picture search with DuckDuckGo (jpg, png, gif, bmp) and show the results (thumbnails).
#@markdown * It creates a folder content/images (local runtime: ~/labor/content/images) and there a new folder for every search.
#@markdown * The image thumbnails (and optional the images) will be downloaded in the search folder.
#@markdown * It will generate a bash script (easy remove bad pictures) for downloading the images later (or again).

import                            time, json, re, ipywidgets, os, \
                                  requests, matplotlib
from matplotlib           import  pyplot as plt
from urllib.parse         import  urlencode, quote, urlparse, urlunparse
from urllib.request       import  build_opener, Request
from urllib.error         import  HTTPError, URLError
from google.colab         import  widgets
from PIL                  import  Image
from tqdm                 import  tqdm

query                     = 'stupid cat' #@param {type:"string"}
results                   = 30 #@param {type: "slider", min: 1, max: 999, step: 1}
downloadImages            = False #@param {type:"boolean"}
                            #@markdown **thumbnails display**
columns                   = 10 #@param {type: "slider", min: 1, max: 20, step: 1}
rows                      = 3 #@param {type: "slider", min: 1, max: 100, step: 1}

##########################################################################################################
########################## reversed from https://course.fast.ai/ #########################################
##########################################################################################################
url_default_headers = {
    "Accept":
    "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

ExceptionsHTTP = {}

def urlquote(url):
    "Update url's path with `urllib.parse.quote`"
    subdelims = "!$&'()*+,;="
    gendelims = ":?#[]@"
    safe = subdelims+gendelims+"%/"
    p = list(urlparse(url))
    p[2] = quote(p[2], safe=safe)
    for i in range(3,6): p[i] = quote(p[i], safe=safe)
    return urlunparse(p)

def loads(s, **kw):
    "Same as `json.loads`, but handles `None`"
    if not s: return {}
    try: import ujson as json
    except ModuleNotFoundError: import json
    return json.loads(s, **kw)

def urlwrap(url, data=None, headers=None):
    "Wrap `url` in a urllib `Request` with `urlquote`"
    return url if isinstance(url,Request) else Request(urlquote(url), data=data, headers=headers or {})

def urlopener():
    _opener = build_opener()
    _opener.addheaders = list(url_default_headers.items())
    return _opener

def urlopen(url, data=None, headers=None, timeout=None, **kwargs):
    "Like `urllib.request.urlopen`, but first `urlwrap` the `url`, and encode `data`"
    if kwargs and not data: data=kwargs
    if data is not None:
        if not isinstance(data, (str,bytes)): data = urlencode(data)
        if not isinstance(data, bytes): data = data.encode('ascii')
    try: return urlopener().open(urlwrap(url, data=data, headers=headers), timeout=timeout)
    except HTTPError as e:
        e.msg += f"\n====Error Body====\n{e.read().decode(errors='ignore')}"
        raise

def urlread(url, data=None, headers=None, decode=True, return_json=False, return_headers=False, timeout=None, **kwargs):
    "Retrieve `url`, using `data` dict or `kwargs` to `POST` if present"
    try:
        with urlopen(url, data=data, headers=headers, timeout=timeout, **kwargs) as u: res,hdrs = u.read(),u.headers
    except HTTPError as e:
        if 400 <= e.code < 500: raise ExceptionsHTTP[e.code](e.url, e.hdrs, e.fp) from None
        else: raise

    if decode: res = res.decode()
    if return_json: res = loads(res)
    return (res,dict(hdrs)) if return_headers else res

def isddg(term, max_images=200):
    "Search for `term` with DuckDuckGo and return a unique urls of about `max_images` images"
    assert max_images<1000
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    assert searchObj
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    xu,data = list(),{'next':1} # little change: don't use the mysterious L class
    headers = dict(referer='https://duckduckgo.com/')
    while len(xu)<max_images and 'next' in data:
        try:
            res = urlread(requestUrl, data=params, headers=headers)
            data = json.loads(res) if res else {}
            for i in range(len(data['results'])):
              xu.append(data['results'][i])
            requestUrl = url + data['next']
        except (URLError,HTTPError): pass
        time.sleep(1)
    return xu[:max_images]
##########################################################################################################
##########################################################################################################
##########################################################################################################

# download the ddg results page (links)
links                 = isddg (query,results)
if downloadImages:    print (f'checking {len(links)} links - start download thumbnails and images')
else:                 print (f'checking {len(links)} links - start download thumbnails')

# mkdir
qfolder       = query.replace (' ','')
if os.path.exists('/content'):
  os.makedirs   ('/content/images',exist_ok=True)
  os.makedirs   (f'/content/images/{qfolder}',exist_ok=True)
  os.chdir      (f'/content/images/{qfolder}')
else:
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content',exist_ok=True)
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content/images',exist_ok=True)
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content/images/{qfolder}',exist_ok=True)
  os.chdir      (f'{os.path.expanduser("~")}/labor/content/images/{qfolder}')


# inits
pos, bad, thumbs, bash = 0, 0, [], f'#!/bin/bash\nmkdir -p {qfolder}\n'

# loop the links
for all in tqdm(links):

  # extract infos
  url  = all['image']
  turl = all['thumbnail']
  ext  = url.split('.')[-1]
  if '?' in ext: ext = ext.split('?')[0]
  dest = f"{qfolder}{pos-bad}.{ext}"
  pos+=1

  # check extension
  if ext.lower() in 'jpg png gif bmp':

    # download image und thumbnail
    try:

      # image
      if downloadImages:
        im = Image.open(requests.get(url, stream = True).raw)
        im.save(dest)

      # bash script entry
      bash += f'# Picture {dest}\ncurl "{url}" --output {qfolder}/{dest}\n'

      # thumbnail
      im = Image.open(requests.get(turl, stream = True).raw)
      im.save('thumb'+dest)
      thumbs.append('thumb'+dest)

    # count bad links
    except Exception:
      bad += 1
  else:
    bad += 1

# output infos
if not downloadImages:  txt = f'downloaded {len(thumbs)} thumbnails'
else:                   txt = f'downloaded {len(thumbs)} images and thumbnails'
print                   (f'\n\n\x1b[32m{txt}\x1b[0m - \x1b[91mfound {bad} bad links!!!\x1b[0m')

# write bash
if os.path.exists('/content'):
  print    (f'Create bash script: \x1b[34m/content/images/dl_{qfolder}.sh\x1b[0m')
  open     (f'/content/images/dl_{qfolder}.sh', 'w').write(bash)
else:
  print    (f'Create bash script: \x1b[34m~/labor/content/images/dl_{qfolder}.sh\x1b[0m')
  open     (f'{os.path.expanduser("~")}/labor/content/images/dl_{qfolder}.sh', 'w').write(bash)


# build the thumbnails tabs
titles, len_thumbs = [], len(thumbs)
for i in range(0,len_thumbs,(columns*rows)):
  x = ((i//(columns*rows))+1)*(columns*rows)-1
  if x >= len_thumbs: x = len_thumbs-1
  titles.append(f'{i}-{x}')
tabs = widgets.TabBar(titles)

# title font
matplotlib.rc('font', **{'sans-serif':    'DejaVu Sans',
                          'family':        'sans-serif',
                          'weight':        'bold',
                          'size':          (columns+10 if columns > 12 else 16) })

# fill the tabs
for i,t in enumerate(titles):
  with tabs.output_to (t):

    # figure dimension (inch)
    fig = plt.figure(figsize=(4*columns, 4.25*rows))

    # all images for the tab i
    for j in range(columns*rows):
      x = i*(columns*rows)+j

      # subplot image
      if x < len_thumbs:
        fig.add_subplot(rows, columns, j+1, xticks=[], yticks=[], title=f'{x}')
        im = Image.open (thumbs[x])
        plt.imshow(im)

# cd to image folder
if os.path.exists('/content'): os.chdir (f'/content/images')
else: os.chdir (f'{os.path.expanduser("~")}/labor/content/images')

# _sp: dictor

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**dictor**</font><br>
#@markdown * A little dict editor using ipywidgets - easy expandable and customizable.
#@markdown ```python
#@markdown def doit (d): print (d)
#@markdown dictor (data := {'a':'one', 'b':'two'}, doit);
#@markdown ```

from ipywidgets import Tab, Text, Box, VBox, HBox, Button, Textarea, Layout, Label
import time

class dictor ():
  def doitClick (self, b):
    try:    self.doit (self)
    except: pass

  def deleteItem (self, b):
    try:
      self.data.pop (self.textKey.value);
      self.buildTab ()
    except: pass

  def storeItem (self, b):
    try:
      if self.textKey.value:
        self.data[self.textKey.value] = self.textareaValue.value
      self.buildTab ()
    except: pass

  def keyChange (self, change):
    titles  = [self.tabElements.get_title (i+1) for i in range(len(self.tabElements.children)-1)]
    pos     = titles.index(self.textKey.value) + 1 if self.textKey.value in titles else 0
    self.tabElements.unobserve_all ()
    self.tabElements.selected_index = pos
    self.textareaValue.value = str(self.data[titles[pos-1]]) if pos else ''
    self.tabElements.observe (self.tabSelect, names='selected_index')

  def tabSelect (self, change):
    self.textKey.value  = self.tabElements.get_title(self.tabElements.selected_index) if self.tabElements.selected_index else ''
    self.textareaValue.value = str (self.data[self.textKey.value]) if self.textKey.value in self.data.keys() else ''

  def show (self):
    display (self.vboxMain)

  def buildTab (self):
    self.tabElements.unobserve_all ()
    self.tabElements.children = self.tabElements.children[:1]
    if len (self.data):
      for i, (k, v) in enumerate (self.data.items()):
        self.tabElements.children = [*self.tabElements.children, Label (value=str (v))]
        self.tabElements.set_title (i+1, k)
    self.textareaValue.value = self.textKey.value = ''
    time.sleep (0.1) # old bitch: without sleep a strange error: insert a 'c' tab, than delete b -> no selction tab 0
    self.tabElements.selected_index  = 0
    self.tabElements.observe (self.tabSelect, names='selected_index')

  def __init__(self, dic=None, doit=None):
    self.doit,self.data = doit,dic    if dic else {}
    self.buttonStore    = Button      (description='Store')
    self.buttonDelete   = Button      (description='Delete')
    self.buttonDoit     = Button      (description='Do it')
    self.textKey        = Text        (description='Key',    layout=Layout(width='550px'))
    self.textareaValue  = Textarea    (description='Value',  layout=Layout(width='550px',height='100px'))
    self.boxButtons     = Box         (layout=Layout(display='flex', flex_flow='wrap'))
    self.tabElements    = Tab         (children=[Label()],layout=Layout(display='flex', flex_flow='wrap') )
    self.vboxControls   = VBox        (children=[HBox(children=[self.buttonStore,self.buttonDelete,self.buttonDoit]), self.textKey, self.textareaValue])
    self.vboxMain       = VBox        (children=[self.boxButtons, self.vboxControls, self.tabElements], layout=Layout(width='auto'))
    self.textKey        .observe      (self.keyChange, names='value')
    self.tabElements    .observe      (self.tabSelect, names='selected_index')
    self.buttonStore    .on_click     (self.storeItem)
    self.buttonDelete   .on_click     (self.deleteItem)
    self.buttonDoit     .on_click     (self.doitClick)
    self.buildTab ()
    display (self.vboxMain)

# event function parameter is the dictor object
def doit (d): print (f'\rdoit {d.tabElements.selected_index}',end='')
dictor (data := {'a':'one', 'b':'two'}, doit);




# _sp: button box

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**button box**</font><br>
#@markdown * An alternative for the Tab widget for selecting **large amount of items**
#@markdown * Uses **Buttons** and a **Box** with **flex wrap layout**.
#@markdown * Provide a **mechanism for select, append and delete** an item .
#@markdown ```python
#@markdown bbc  = lambda bb: print(bb.position)    # event function: gets self as parameter
#@markdown bbox = ButtonBox (lst,                  # lst contains realy long descriptions
#@markdown                     color   = 'lavender', # selection color (default is powderblue)
#@markdown                     maxchar = 40,         # cut description position
#@markdown                     clicker = bbc)        # event
#@markdown display (bbox.box)                      # show widget
#@markdown ```
from ipywidgets import Button, Box, Layout, Textarea

class ButtonBox ():
  # An alternative for the Tab widget
  def _clicker (self, b):
    # search clicked button and remember
    self.position, self.button = [(i,but) for i, but in enumerate(self.buttons) if b == but][0]

    # unselect (color) all buttons and select new (list of colors here: https://www.quackit.com/css/css_color_codes.cfm)
    for b in self.buttons: b.style={'button_color':None}
    self.buttons[self.position].style={'button_color':self.color}

    # fire event
    if self.clicker: self.clicker (self)

  def append (self, description, select=True):
    # add new selctor button at the end
    b = Button (description = description if len (description) <= self.maxchar else f'{description[:(self.maxchar-3)]}...',
                layout      = Layout(width='auto', height='21px'),
                tooltip     = f'{description}')
    self.buttons.append(b)
    self.box.children = [*self.box.children, b]

    # select new button / bind event
    if select:
      self.button, self.position = b, len(self.buttons) - 1
      self._clicker (b)
    b.on_click (self._clicker)

  def remove (self, position=None):
    # remove selected if no position given
    if not position: position = self.position
    if not position or not position < len(self.buttons) or position < 0: return

    # clean up
    self.box.children = [*self.box.children[:position], *self.box.children[position+1:]]
    self.buttons = [*self.buttons[:position], *self.buttons[position+1:]]
    self.button, self.position = None, -1

  def __init__ (self, descriptions, clicker=None, maxchar=60, color='powderblue'):
    # remember stuff - list to set
    self.descriptions = descriptions if len (descriptions) == len (set (descriptions)) else set (descriptions)
    self.clicker, self.maxchar, self.color, self.position, self.button = clicker, maxchar, color, -1, None

    # make buttons
    self.buttons = [Button (description = i if len (i) <= maxchar else f'{i[:(maxchar-3)]}...',
                            layout      = Layout(width='auto', height='21px'),
                            tooltip     = f'{i}')
                    for i in descriptions]

    # put them in a box / bind event
    self.box = Box (layout   = Layout (display='flex', flex_flow='wrap'),
                    children = self.buttons)
    for button in self.buttons: button.on_click (self._clicker)

# *******************************************************************************************************************************************************************************************
# ****usage**********************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
import pydoc, ipywidgets

lst      = list(__builtins__.__dict__.keys())
helpText = ipywidgets.Textarea (layout=Layout(width='1000px', height='300px'))

def bbclicker (bb):
  try:    helpText.value = pydoc.plain(pydoc.render_doc (bb.button.tooltip))
  except: helpText.value = f'error calling render_doc with {bb.button.tooltip}'

bbox   = ButtonBox (lst, color='lavender', maxchar=40, clicker=bbclicker)
(delb := ipywidgets.Button (description="delete")).on_click (lambda b: bbox.remove ())
(appb := ipywidgets.Button (description="append")).on_click (lambda b: bbox.append ('teest'))

display (bbox.box, helpText, appb, delb)


#_sp: breakpoint

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**debug**</font><br>
#@markdown * When an error happens you can run **%debug** in a new code cell.
#@markdown * Or you set a breakpoint with ```from IPython.core.debugger import Pdb; Pdb().set_trace()```
#@markdown * Some commands:<br><table align='left'><tr><td><b>'q': quit</b> - Exit the debugger.<br>_</td><td><b>'c': continue</b> - Run to the next breakpoint.<br>_</td><td><b>'l' : list</b> - Shows 11 lines of code<br>around the actual position.</td></tr><tr><td><b>'n': next</b> - Run the next line<br>in current function.</td><td><b>'s': step</b> - step in to a function.<br>_</td><td><b>'r' : return</b> - Continue execution until<br>the current function returns.</td></tr></tr><tr><td><b>'h': help</b> - Shows help for a command<br>or all (if none parameter)</td><td><b>'b': breakpoint</b> - Makes a breakpoint for<br>a function or codeline (nr).</td><td><b>'cl' : clear</b> - Deletes breakpoint(s)<br>_</td></tr></table>

# breakpoint
# from IPython.core.debugger import Pdb; Pdb().set_trace()
pass

#_sp: breakpoint - short

In [ ]:
#from IPython.core.debugger import Pdb; Pdb().set_trace()

#_sp: escape codes & emojis 😎

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**escape codes & emojis 😎**</font><br>
#@markdown * selection from [here](https://gist.github.com/iansan5653/c4a0b9f5c30d74258c5f132084b78db9), [wikipedia](https://en.wikipedia.org/wiki/ANSI_escape_code), [python docs](https://docs.python.org/3/reference/lexical_analysis.html#escape-sequences)
#@markdown * emojis from [here](https://getemoji.com/)
#@markdown ```python
#@markdown #        +---------------------------* '\x': hex code as character ('\u': unicode16, '\U': unicode32 )
#@markdown #        | +-------------------------* '1b': code for ESC sequence (decimal 27)
#@markdown #        | |+------------------------* '[': start parameter sequence
#@markdown #        | ||       +----------------* ';': seperated parameter (1: bold, 3: italic, 4: underline, 34: color)
#@markdown #        | ||       |+---------------* 'm': end of parameter sequence
#@markdown #        | ||       ||  +------------* content 'abc'
#@markdown #        | ||       ||  |      +-----* '\x1b[0m': ESC sequence to set color back to black
#@markdown #        | ||       ||  |      |  +--* content 'xyz'
#@markdown print ("\x1b[1;3;4;34mabc\x1b[0mxyz")
#@markdown ```
#@markdown <font color='blue'><b><i><u>abc</u></i></b></font>xyz
print("\x1b[1;3;4;34mabc\x1b[0mxyz")
print('\x1b[0m Reset / Normal \x1b[0m')
print('\x1b[1m Bold or increased intensity \x1b[0m')
print('\x1b[3m Italic \x1b[0m')
print('\x1b[4m Underline \x1b[0m')

print(' Basic Foreground Colors:')
print('\t\x1b[30m Black foreground\x1b[0m')
print('\t\x1b[31m Red foreground\x1b[0m')
print('\t\x1b[32m Green foreground\x1b[0m')
print('\t\x1b[33m Yellow foreground\x1b[0m')
print('\t\x1b[34m Blue foreground\x1b[0m')
print('\t\x1b[35m Magenta foreground\x1b[0m')
print('\t\x1b[36m Cyan foreground\x1b[0m')
print('\t\x1b[37m White foreground\x1b[0m')
print('\t\x1b[39m Default foreground color \x1b[0m')

print(' Basic Background Colors:')
print('\t\x1b[40m Black background\x1b[0m')
print('\t\x1b[41m Red background\x1b[0m')
print('\t\x1b[42m Green background\x1b[0m')
print('\t\x1b[43m Yellow background\x1b[0m')
print('\t\x1b[44m Blue background\x1b[0m')
print('\t\x1b[45m Magenta background\x1b[0m')
print('\t\x1b[46m Cyan background\x1b[0m')
print('\t\x1b[47m White background\x1b[0m')
print('\t\x1b[49m Default background color \x1b[0m')

print(' Bright Foreground Colors:')
print('\t\x1b[90m Bright Black foreground\x1b[0m')
print('\t\x1b[91m Bright Red foreground\x1b[0m')
print('\t\x1b[92m Bright Green foreground\x1b[0m')
print('\t\x1b[93m Bright Yellow foreground\x1b[0m')
print('\t\x1b[94m Bright Blue foreground\x1b[0m')
print('\t\x1b[95m Bright Magenta foreground\x1b[0m')
print('\t\x1b[96m Bright Cyan foreground\x1b[0m')
print('\t\x1b[97m Bright White foreground\x1b[0m')

print(' Bright Background Colors:')
print('\t\x1b[100m Bright Black background\x1b[0m')
print('\t\x1b[101m Bright Red background\x1b[0m')
print('\t\x1b[102m Bright Green background\x1b[0m')
print('\t\x1b[103m Bright Yellow background\x1b[0m')
print('\t\x1b[104m Bright Blue background\x1b[0m')
print('\t\x1b[105m Bright Magenta background\x1b[0m')
print('\t\x1b[106m Bright Cyan background\x1b[0m')
print('\t\x1b[107m Bright White background\x1b[0m')

print ('Smileys\n 😀 😃 😄 😁 😆 😅 😂 🤣 🥲 🥹 ☺️ 😊 😇 🙂 🙃 😉 😌 😍 🥰 😘 😗 😙 😚 😋 😛 😝 😜 🤪 🤨 🧐 🤓 😎 🥸 🤩 🥳 🙂‍↕️ 😏 😒 🙂‍↔️ 😞 😔 😟 😕 🙁 ☹️ 😣 😖 😫 😩 🥺 😢 😭 😮‍💨 😤 😠 😡 🤬 🤯 😳 🥵 🥶 😱 😨 😰 😥 😓 🫣 🤗 🫡 🤔 🫢 🤭 🤫 🤥 😶 😶‍🌫️ 😐 😑 😬 🫨 🫠 🙄 😯 😦 😧 😮 😲 🥱 😴 🤤 😪 😵 😵‍💫 🫥 🤐 🥴 🤢 🤮 🤧 😷 🤒 🤕 🤑 🤠 😈 👿 👹 👺 🤡 💩 👻 💀 ☠️ 👽 👾 🤖 🎃 😺 😸 😹 😻 😼 😽 🙀 😿 😾 ')
print ('Gestures and Body Parts\n 👋 🤚 🖐 ✋ 🖖 👌 🤌 🤏 ✌️ 🤞 🫰 🤟 🤘 🤙 🫵 🫱 🫲 🫸 🫷 🫳 🫴 👈 👉 👆 🖕 👇 ☝️ 👍 👎 ✊ 👊 🤛 🤜 👏 🫶 🙌 👐 🤲 🤝 🙏 ✍️ 💅 🤳 💪 🦾 🦵 🦿 🦶 👣 👂 🦻 👃 🫀 🫁 🧠 🦷 🦴 👀 👁 👅 👄 🫦 💋 🩸 ')
print ('People and Fantasy\n 👶 👧 🧒 👦 👩 🧑 👨 👩‍🦱 🧑‍🦱 👨‍🦱 👩‍🦰 🧑‍🦰 👨‍🦰 👱‍♀️ 👱 👱‍♂️ 👩‍🦳 🧑‍🦳 👨‍🦳 👩‍🦲 🧑‍🦲 👨‍🦲 🧔‍♀️ 🧔 🧔‍♂️ 👵 🧓 👴 👲 👳‍♀️ 👳 👳‍♂️ 🧕 👮‍♀️ 👮 👮‍♂️ 👷‍♀️ 👷 👷‍♂️ 💂‍♀️ 💂 💂‍♂️ 🕵️‍♀️ 🕵️ 🕵️‍♂️ 👩‍⚕️ 🧑‍⚕️ 👨‍⚕️ 👩‍🌾 🧑‍🌾 👨‍🌾 👩‍🍳 🧑‍🍳 👨‍🍳 👩‍🎓 🧑‍🎓 👨‍🎓 👩‍🎤 🧑‍🎤 👨‍🎤 👩‍🏫 🧑‍🏫 👨‍🏫 👩‍🏭 🧑‍🏭 👨‍🏭 👩‍💻 🧑‍💻 👨‍💻 👩‍💼 🧑‍💼 👨‍💼 👩‍🔧 🧑‍🔧 👨‍🔧 👩‍🔬 🧑‍🔬 👨‍🔬 👩‍🎨 🧑‍🎨 👨‍🎨 👩‍🚒 🧑‍🚒 👨‍🚒 👩‍✈️ 🧑‍✈️ 👨‍✈️ 👩‍🚀 🧑‍🚀 👨‍🚀 👩‍⚖️ 🧑‍⚖️ 👨‍⚖️ 👰‍♀️ 👰 👰‍♂️ 🤵‍♀️ 🤵 🤵‍♂️ 👸 🫅 🤴 🥷 🦸‍♀️ 🦸 🦸‍♂️ 🦹‍♀️ 🦹 🦹‍♂️ 🤶 🧑‍🎄 🎅 🧙‍♀️ 🧙 🧙‍♂️ 🧝‍♀️ 🧝 🧝‍♂️ 🧛‍♀️ 🧛 🧛‍♂️ 🧟‍♀️ 🧟 🧟‍♂️ 🧞‍♀️ 🧞 🧞‍♂️ 🧜‍♀️ 🧜 🧜‍♂️ 🧚‍♀️ 🧚 🧚‍♂️ 🧌 👼 🤰 🫄 🫃 🤱 👩‍🍼 🧑‍🍼 👨‍🍼 🙇‍♀️ 🙇 🙇‍♂️ 💁‍♀️ 💁 💁‍♂️ 🙅‍♀️ 🙅 🙅‍♂️ 🙆‍♀️ 🙆 🙆‍♂️ 🙋‍♀️ 🙋 🙋‍♂️ 🧏‍♀️ 🧏 🧏‍♂️ 🤦‍♀️ 🤦 🤦‍♂️ 🤷‍♀️ 🤷 🤷‍♂️ 🙎‍♀️ 🙎 🙎‍♂️ 🙍‍♀️ 🙍 🙍‍♂️ 💇‍♀️ 💇 💇‍♂️ 💆‍♀️ 💆 💆‍♂️ 🧖‍♀️ 🧖 🧖‍♂️ 💅 🤳 💃 🕺 👯‍♀️ 👯 👯‍♂️ 🕴 👩‍🦽 👩‍🦽‍➡️ 🧑‍🦽 🧑‍🦽‍➡️ 👨‍🦽 👨‍🦽‍➡️ 👩‍🦼 👩‍🦼‍➡️ 🧑‍🦼 🧑‍🦼‍➡️ 👨‍🦼 👨‍🦼‍➡️ 🚶‍♀️ 🚶‍♀️‍➡️ 🚶 🚶‍➡️ 🚶‍♂️ 🚶‍♂️‍➡️ 👩‍🦯 👩‍🦯‍➡️ 🧑‍🦯 🧑‍🦯‍➡️ 👨‍🦯 👨‍🦯‍➡️ 🧎‍♀️ 🧎‍♀️‍➡️ 🧎 🧎‍➡️ 🧎‍♂️ 🧎‍♂️‍➡️ 🏃‍♀️ 🏃‍♀️‍➡️ 🏃 🏃‍➡️ 🏃‍♂️ 🏃‍♂️‍➡️ 🧍‍♀️ 🧍 🧍‍♂️ 👭 🧑‍🤝‍🧑 👬 👫 👩‍❤️‍👩 💑 👨‍❤️‍👨 👩‍❤️‍👨 👩‍❤️‍💋‍👩 💏 👨‍❤️‍💋‍👨 👩‍❤️‍💋‍👨 👪 👨‍👩‍👦 👨‍👩‍👧 👨‍👩‍👧‍👦 👨‍👩‍👦‍👦 👨‍👩‍👧‍👧 👨‍👨‍👦 👨‍👨‍👧 👨‍👨‍👧‍👦 👨‍👨‍👦‍👦 👨‍👨‍👧‍👧 👩‍👩‍👦 👩‍👩‍👧 👩‍👩‍👧‍👦 👩‍👩‍👦‍👦 👩‍👩‍👧‍👧 👨‍👦 👨‍👦‍👦 👨‍👧 👨‍👧‍👦 👨‍👧‍👧 👩‍👦 👩‍👦‍👦 👩‍👧 👩‍👧‍👦 👩‍👧‍👧 🧑‍🧑‍🧒 🧑‍🧑‍🧒‍🧒 🧑‍🧒 🧑‍🧒‍🧒 🗣 👤 👥 🫂 ')
print ('Clothing and Accessories\n 🧳 🌂 ☂️ 🧵 🪡 🪢 🪭 🧶 👓 🕶 🥽 🥼 🦺 👔 👕 👖 🧣 🧤 🧥 🧦 👗 👘 🥻 🩴 🩱 🩲 🩳 👙 👚 👛 👜 👝 🎒 👞 👟 🥾 🥿 👠 👡 🩰 👢 👑 👒 🎩 🎓 🧢 ⛑ 🪖 💄 💍 💼 ')
print ('Pale Emojis\n 👋🏻 🤚🏻 🖐🏻 ✋🏻 🖖🏻 👌🏻 🤌🏻 🤏🏻 ✌🏻 🤞🏻 🫰🏻 🤟🏻 🤘🏻 🤙🏻 🫵🏻 🫱🏻 🫲🏻 🫸🏻 🫷🏻 🫳🏻 🫴🏻 👈🏻 👉🏻 👆🏻 🖕🏻 👇🏻 ☝🏻 👍🏻 👎🏻 ✊🏻 👊🏻 🤛🏻 🤜🏻 👏🏻 🫶🏻 🙌🏻 👐🏻 🤲🏻 🙏🏻 ✍🏻 💅🏻 🤳🏻 💪🏻 🦵🏻 🦶🏻 👂🏻 🦻🏻 👃🏻 👶🏻 👧🏻 🧒🏻 👦🏻 👩🏻 🧑🏻 👨🏻 👩🏻‍🦱 🧑🏻‍🦱 👨🏻‍🦱 👩🏻‍🦰 🧑🏻‍🦰 👨🏻‍🦰 👱🏻‍♀️ 👱🏻 👱🏻‍♂️ 👩🏻‍🦳 🧑🏻‍🦳 👨🏻‍🦳 👩🏻‍🦲 🧑🏻‍🦲 👨🏻‍🦲 🧔🏻‍♀️ 🧔🏻 🧔🏻‍♂️ 👵🏻 🧓🏻 👴🏻 👲🏻 👳🏻‍♀️ 👳🏻 👳🏻‍♂️ 🧕🏻 👮🏻‍♀️ 👮🏻 👮🏻‍♂️ 👷🏻‍♀️ 👷🏻 👷🏻‍♂️ 💂🏻‍♀️ 💂🏻 💂🏻‍♂️ 🕵🏻‍♀️ 🕵🏻 🕵🏻‍♂️ 👩🏻‍⚕️ 🧑🏻‍⚕️ 👨🏻‍⚕️ 👩🏻‍🌾 🧑🏻‍🌾 👨🏻‍🌾 👩🏻‍🍳 🧑🏻‍🍳 👨🏻‍🍳 👩🏻‍🎓 🧑🏻‍🎓 👨🏻‍🎓 👩🏻‍🎤 🧑🏻‍🎤 👨🏻‍🎤 👩🏻‍🏫 🧑🏻‍🏫 👨🏻‍🏫 👩🏻‍🏭 🧑🏻‍🏭 👨🏻‍🏭 👩🏻‍💻 🧑🏻‍💻 👨🏻‍💻 👩🏻‍💼 🧑🏻‍💼 👨🏻‍💼 👩🏻‍🔧 🧑🏻‍🔧 👨🏻‍🔧 👩🏻‍🔬 🧑🏻‍🔬 👨🏻‍🔬 👩🏻‍🎨 🧑🏻‍🎨 👨🏻‍🎨 👩🏻‍🚒 🧑🏻‍🚒 👨🏻‍🚒 👩🏻‍✈️ 🧑🏻‍✈️ 👨🏻‍✈️ 👩🏻‍🚀 🧑🏻‍🚀 👨🏻‍🚀 👩🏻‍⚖️ 🧑🏻‍⚖️ 👨🏻‍⚖️ 👰🏻‍♀️ 👰🏻 👰🏻‍♂️ 🤵🏻‍♀️ 🤵🏻 🤵🏻‍♂️ 👸🏻 🫅🏻 🤴🏻 🥷🏻 🦸🏻‍♀️ 🦸🏻 🦸🏻‍♂️ 🦹🏻‍♀️ 🦹🏻 🦹🏻‍♂️ 🤶🏻 🧑🏻‍🎄 🎅🏻 🧙🏻‍♀️ 🧙🏻 🧙🏻‍♂️ 🧝🏻‍♀️ 🧝🏻 🧝🏻‍♂️ 🧛🏻‍♀️ 🧛🏻 🧛🏻‍♂️ 🧜🏻‍♀️ 🧜🏻 🧜🏻‍♂️ 🧚🏻‍♀️ 🧚🏻 🧚🏻‍♂️ 👼🏻 🤰🏻 🫄🏻 🫃🏻 🤱🏻 👩🏻‍🍼 🧑🏻‍🍼 👨🏻‍🍼 🙇🏻‍♀️ 🙇🏻 🙇🏻‍♂️ 💁🏻‍♀️ 💁🏻 💁🏻‍♂️ 🙅🏻‍♀️ 🙅🏻 🙅🏻‍♂️ 🙆🏻‍♀️ 🙆🏻 🙆🏻‍♂️ 🙋🏻‍♀️ 🙋🏻 🙋🏻‍♂️ 🧏🏻‍♀️ 🧏🏻 🧏🏻‍♂️ 🤦🏻‍♀️ 🤦🏻 🤦🏻‍♂️ 🤷🏻‍♀️ 🤷🏻 🤷🏻‍♂️ 🙎🏻‍♀️ 🙎🏻 🙎🏻‍♂️ 🙍🏻‍♀️ 🙍🏻 🙍🏻‍♂️ 💇🏻‍♀️ 💇🏻 💇🏻‍♂️ 💆🏻‍♀️ 💆🏻 💆🏻‍♂️ 🧖🏻‍♀️ 🧖🏻 🧖🏻‍♂️ 💃🏻 🕺🏻 🕴🏻 👩🏻‍🦽 👩🏻‍🦽‍➡️ 🧑🏻‍🦽 🧑🏻‍🦽‍➡️ 👨🏻‍🦽 👨🏻‍🦽‍➡️ 👩🏻‍🦼 👩🏻‍🦼‍➡️ 🧑🏻‍🦼 🧑🏻‍🦼‍➡️ 👨🏻‍🦼 👨🏻‍🦼‍➡️ 🚶🏻‍♀️ 🚶🏻‍♀️‍➡️ 🚶🏻 🚶🏻‍➡️ 🚶🏻‍♂️ 🚶🏻‍♂️‍➡️ 👩🏻‍🦯 👩🏻‍🦯‍➡️ 🧑🏻‍🦯 🧑🏻‍🦯‍➡️ 👨🏻‍🦯 👨🏻‍🦯‍➡️ 🧎🏻‍♀️ 🧎🏻‍♀️‍➡️ 🧎🏻 🧎🏻‍➡️ 🧎🏻‍♂️ 🧎🏻‍♂️‍➡️ 🏃🏻‍♀️ 🏃🏻‍♀️‍➡️ 🏃🏻 🏃🏻‍➡️ 🏃🏻‍♂️ 🏃🏻‍♂️‍➡️ 🧍🏻‍♀️ 🧍🏻 🧍🏻‍♂️ 👭🏻 🧑🏻‍🤝‍🧑🏻 👬🏻 👫🏻 🧗🏻‍♀️ 🧗🏻 🧗🏻‍♂️ 🏇🏻 🏂🏻 🏌🏻‍♀️ 🏌🏻 🏌🏻‍♂️ 🏄🏻‍♀️ 🏄🏻 🏄🏻‍♂️ 🚣🏻‍♀️ 🚣🏻 🚣🏻‍♂️ 🏊🏻‍♀️ 🏊🏻 🏊🏻‍♂️ ⛹🏻‍♀️ ⛹🏻 ⛹🏻‍♂️ 🏋🏻‍♀️ 🏋🏻 🏋🏻‍♂️ 🚴🏻‍♀️ 🚴🏻 🚴🏻‍♂️ 🚵🏻‍♀️ 🚵🏻 🚵🏻‍♂️ 🤸🏻‍♀️ 🤸🏻 🤸🏻‍♂️ 🤽🏻‍♀️ 🤽🏻 🤽🏻‍♂️ 🤾🏻‍♀️ 🤾🏻 🤾🏻‍♂️ 🤹🏻‍♀️ 🤹🏻 🤹🏻‍♂️ 🧘🏻‍♀️ 🧘🏻 🧘🏻‍♂️ 🛀🏻 🛌🏻 ')
print ('Cream White Emojis\n 👋🏼 🤚🏼 🖐🏼 ✋🏼 🖖🏼 👌🏼 🤌🏼 🤏🏼 ✌🏼 🤞🏼 🫰🏼 🤟🏼 🤘🏼 🤙🏼 🫵🏼 🫱🏼 🫲🏼 🫸🏼 🫷🏼 🫳🏼 🫴🏼 👈🏼 👉🏼 👆🏼 🖕🏼 👇🏼 ☝🏼 👍🏼 👎🏼 ✊🏼 👊🏼 🤛🏼 🤜🏼 👏🏼 🫶🏼 🙌🏼 👐🏼 🤲🏼 🙏🏼 ✍🏼 💅🏼 🤳🏼 💪🏼 🦵🏼 🦶🏼 👂🏼 🦻🏼 👃🏼 👶🏼 👧🏼 🧒🏼 👦🏼 👩🏼 🧑🏼 👨🏼 👩🏼‍🦱 🧑🏼‍🦱 👨🏼‍🦱 👩🏼‍🦰 🧑🏼‍🦰 👨🏼‍🦰 👱🏼‍♀️ 👱🏼 👱🏼‍♂️ 👩🏼‍🦳 🧑🏼‍🦳 👨🏼‍🦳 👩🏼‍🦲 🧑🏼‍🦲 👨🏼‍🦲 🧔🏼‍♀️ 🧔🏼 🧔🏼‍♂️ 👵🏼 🧓🏼 👴🏼 👲🏼 👳🏼‍♀️ 👳🏼 👳🏼‍♂️ 🧕🏼 👮🏼‍♀️ 👮🏼 👮🏼‍♂️ 👷🏼‍♀️ 👷🏼 👷🏼‍♂️ 💂🏼‍♀️ 💂🏼 💂🏼‍♂️ 🕵🏼‍♀️ 🕵🏼 🕵🏼‍♂️ 👩🏼‍⚕️ 🧑🏼‍⚕️ 👨🏼‍⚕️ 👩🏼‍🌾 🧑🏼‍🌾 👨🏼‍🌾 👩🏼‍🍳 🧑🏼‍🍳 👨🏼‍🍳 👩🏼‍🎓 🧑🏼‍🎓 👨🏼‍🎓 👩🏼‍🎤 🧑🏼‍🎤 👨🏼‍🎤 👩🏼‍🏫 🧑🏼‍🏫 👨🏼‍🏫 👩🏼‍🏭 🧑🏼‍🏭 👨🏼‍🏭 👩🏼‍💻 🧑🏼‍💻 👨🏼‍💻 👩🏼‍💼 🧑🏼‍💼 👨🏼‍💼 👩🏼‍🔧 🧑🏼‍🔧 👨🏼‍🔧 👩🏼‍🔬 🧑🏼‍🔬 👨🏼‍🔬 👩🏼‍🎨 🧑🏼‍🎨 👨🏼‍🎨 👩🏼‍🚒 🧑🏼‍🚒 👨🏼‍🚒 👩🏼‍✈️ 🧑🏼‍✈️ 👨🏼‍✈️ 👩🏼‍🚀 🧑🏼‍🚀 👨🏼‍🚀 👩🏼‍⚖️ 🧑🏼‍⚖️ 👨🏼‍⚖️ 👰🏼‍♀️ 👰🏼 👰🏼‍♂️ 🤵🏼‍♀️ 🤵🏼 🤵🏼‍♂️ 👸🏼 🫅🏼 🤴🏼 🥷🏼 🦸🏼‍♀️ 🦸🏼 🦸🏼‍♂️ 🦹🏼‍♀️ 🦹🏼 🦹🏼‍♂️ 🤶🏼 🧑🏼‍🎄 🎅🏼 🧙🏼‍♀️ 🧙🏼 🧙🏼‍♂️ 🧝🏼‍♀️ 🧝🏼 🧝🏼‍♂️ 🧛🏼‍♀️ 🧛🏼 🧛🏼‍♂️ 🧜🏼‍♀️ 🧜🏼 🧜🏼‍♂️ 🧚🏼‍♀️ 🧚🏼 🧚🏼‍♂️ 👼🏼 🤰🏼 🫄🏼 🫃🏼 🤱🏼 👩🏼‍🍼 🧑🏼‍🍼 👨🏼‍🍼 🙇🏼‍♀️ 🙇🏼 🙇🏼‍♂️ 💁🏼‍♀️ 💁🏼 💁🏼‍♂️ 🙅🏼‍♀️ 🙅🏼 🙅🏼‍♂️ 🙆🏼‍♀️ 🙆🏼 🙆🏼‍♂️ 🙋🏼‍♀️ 🙋🏼 🙋🏼‍♂️ 🧏🏼‍♀️ 🧏🏼 🧏🏼‍♂️ 🤦🏼‍♀️ 🤦🏼 🤦🏼‍♂️ 🤷🏼‍♀️ 🤷🏼 🤷🏼‍♂️ 🙎🏼‍♀️ 🙎🏼 🙎🏼‍♂️ 🙍🏼‍♀️ 🙍🏼 🙍🏼‍♂️ 💇🏼‍♀️ 💇🏼 💇🏼‍♂️ 💆🏼‍♀️ 💆🏼 💆🏼‍♂️ 🧖🏼‍♀️ 🧖🏼 🧖🏼‍♂️ 💃🏼 🕺🏼 🕴🏼 👩🏼‍🦽 👩🏼‍🦽‍➡️ 🧑🏼‍🦽 🧑🏼‍🦽‍➡️ 👨🏼‍🦽 👨🏼‍🦽‍➡️ 👩🏼‍🦼 👩🏼‍🦼‍➡️ 🧑🏼‍🦼 🧑🏼‍🦼‍➡️ 👨🏼‍🦼 👨🏼‍🦼‍➡️ 🚶🏼‍♀️ 🚶🏼‍♀️‍➡️ 🚶🏼 🚶🏼‍➡️ 🚶🏼‍♂️ 🚶🏼‍♂️‍➡️ 👩🏼‍🦯 👩🏼‍🦯‍➡️ 🧑🏼‍🦯 🧑🏼‍🦯‍➡️ 👨🏼‍🦯 👨🏼‍🦯‍➡️ 🧎🏼‍♀️ 🧎🏼‍♀️‍➡️ 🧎🏼 🧎🏼‍➡️ 🧎🏼‍♂️ 🧎🏼‍♂️‍➡️ 🏃🏼‍♀️ 🏃🏼‍♀️‍➡️ 🏃🏼 🏃🏼‍➡️ 🏃🏼‍♂️ 🏃🏼‍♂️‍➡️ 🧍🏼‍♀️ 🧍🏼 🧍🏼‍♂️ 👭🏼 🧑🏼‍🤝‍🧑🏼 👬🏼 👫🏼 🧗🏼‍♀️ 🧗🏼 🧗🏼‍♂️ 🏇🏼 🏂🏼 🏌🏼‍♀️ 🏌🏼 🏌🏼‍♂️ 🏄🏼‍♀️ 🏄🏼 🏄🏼‍♂️ 🚣🏼‍♀️ 🚣🏼 🚣🏼‍♂️ 🏊🏼‍♀️ 🏊🏼 🏊🏼‍♂️ ⛹🏼‍♀️ ⛹🏼 ⛹🏼‍♂️ 🏋🏼‍♀️ 🏋🏼 🏋🏼‍♂️ 🚴🏼‍♀️ 🚴🏼 🚴🏼‍♂️ 🚵🏼‍♀️ 🚵🏼 🚵🏼‍♂️ 🤸🏼‍♀️ 🤸🏼 🤸🏼‍♂️ 🤽🏼‍♀️ 🤽🏼 🤽🏼‍♂️ 🤾🏼‍♀️ 🤾🏼 🤾🏼‍♂️ 🤹🏼‍♀️ 🤹🏼 🤹🏼‍♂️ 🧘🏼‍♀️ 🧘🏼 🧘🏼‍♂️ 🛀🏼 🛌🏼 ')
print ('Brown Emojis\n 👋🏽 🤚🏽 🖐🏽 ✋🏽 🖖🏽 👌🏽 🤌🏽 🤏🏽 ✌🏽 🤞🏽 🫰🏽 🤟🏽 🤘🏽 🤙🏽 🫵🏽 🫱🏽 🫲🏽 🫸🏽 🫷🏽 🫳🏽 🫴🏽 👈🏽 👉🏽 👆🏽 🖕🏽 👇🏽 ☝🏽 👍🏽 👎🏽 ✊🏽 👊🏽 🤛🏽 🤜🏽 👏🏽 🫶🏽 🙌🏽 👐🏽 🤲🏽 🙏🏽 ✍🏽 💅🏽 🤳🏽 💪🏽 🦵🏽 🦶🏽 👂🏽 🦻🏽 👃🏽 👶🏽 👧🏽 🧒🏽 👦🏽 👩🏽 🧑🏽 👨🏽 👩🏽‍🦱 🧑🏽‍🦱 👨🏽‍🦱 👩🏽‍🦰 🧑🏽‍🦰 👨🏽‍🦰 👱🏽‍♀️ 👱🏽 👱🏽‍♂️ 👩🏽‍🦳 🧑🏽‍🦳 👨🏽‍🦳 👩🏽‍🦲 🧑🏽‍🦲 👨🏽‍🦲 🧔🏽‍♀️ 🧔🏽 🧔🏽‍♂️ 👵🏽 🧓🏽 👴🏽 👲🏽 👳🏽‍♀️ 👳🏽 👳🏽‍♂️ 🧕🏽 👮🏽‍♀️ 👮🏽 👮🏽‍♂️ 👷🏽‍♀️ 👷🏽 👷🏽‍♂️ 💂🏽‍♀️ 💂🏽 💂🏽‍♂️ 🕵🏽‍♀️ 🕵🏽 🕵🏽‍♂️ 👩🏽‍⚕️ 🧑🏽‍⚕️ 👨🏽‍⚕️ 👩🏽‍🌾 🧑🏽‍🌾 👨🏽‍🌾 👩🏽‍🍳 🧑🏽‍🍳 👨🏽‍🍳 👩🏽‍🎓 🧑🏽‍🎓 👨🏽‍🎓 👩🏽‍🎤 🧑🏽‍🎤 👨🏽‍🎤 👩🏽‍🏫 🧑🏽‍🏫 👨🏽‍🏫 👩🏽‍🏭 🧑🏽‍🏭 👨🏽‍🏭 👩🏽‍💻 🧑🏽‍💻 👨🏽‍💻 👩🏽‍💼 🧑🏽‍💼 👨🏽‍💼 👩🏽‍🔧 🧑🏽‍🔧 👨🏽‍🔧 👩🏽‍🔬 🧑🏽‍🔬 👨🏽‍🔬 👩🏽‍🎨 🧑🏽‍🎨 👨🏽‍🎨 👩🏽‍🚒 🧑🏽‍🚒 👨🏽‍🚒 👩🏽‍✈️ 🧑🏽‍✈️ 👨🏽‍✈️ 👩🏽‍🚀 🧑🏽‍🚀 👨🏽‍🚀 👩🏽‍⚖️ 🧑🏽‍⚖️ 👨🏽‍⚖️ 👰🏽‍♀️ 👰🏽 👰🏽‍♂️ 🤵🏽‍♀️ 🤵🏽 🤵🏽‍♂️ 👸🏽 🫅🏽 🤴🏽 🥷🏽 🦸🏽‍♀️ 🦸🏽 🦸🏽‍♂️ 🦹🏽‍♀️ 🦹🏽 🦹🏽‍♂️ 🤶🏽 🧑🏽‍🎄 🎅🏽 🧙🏽‍♀️ 🧙🏽 🧙🏽‍♂️ 🧝🏽‍♀️ 🧝🏽 🧝🏽‍♂️ 🧛🏽‍♀️ 🧛🏽 🧛🏽‍♂️ 🧜🏽‍♀️ 🧜🏽 🧜🏽‍♂️ 🧚🏽‍♀️ 🧚🏽 🧚🏽‍♂️ 👼🏽 🤰🏽 🫄🏽 🫃🏽 🤱🏽 👩🏽‍🍼 🧑🏽‍🍼 👨🏽‍🍼 🙇🏽‍♀️ 🙇🏽 🙇🏽‍♂️ 💁🏽‍♀️ 💁🏽 💁🏽‍♂️ 🙅🏽‍♀️ 🙅🏽 🙅🏽‍♂️ 🙆🏽‍♀️ 🙆🏽 🙆🏽‍♂️ 🙋🏽‍♀️ 🙋🏽 🙋🏽‍♂️ 🧏🏽‍♀️ 🧏🏽 🧏🏽‍♂️ 🤦🏽‍♀️ 🤦🏽 🤦🏽‍♂️ 🤷🏽‍♀️ 🤷🏽 🤷🏽‍♂️ 🙎🏽‍♀️ 🙎🏽 🙎🏽‍♂️ 🙍🏽‍♀️ 🙍🏽 🙍🏽‍♂️ 💇🏽‍♀️ 💇🏽 💇🏽‍♂️ 💆🏽‍♀️ 💆🏽 💆🏽‍♂️ 🧖🏽‍♀️ 🧖🏽 🧖🏽‍♂️ 💃🏽 🕺🏽 🕴🏽 👩🏽‍🦽 👩🏽‍🦽‍➡️ 🧑🏽‍🦽 🧑🏽‍🦽‍➡️ 👨🏽‍🦽 👨🏽‍🦽‍➡️ 👩🏽‍🦼 👩🏽‍🦼‍➡️ 🧑🏽‍🦼 🧑🏽‍🦼‍➡️ 👨🏽‍🦼 👨🏽‍🦼‍➡️ 🚶🏽‍♀️ 🚶🏽‍♀️‍➡️ 🚶🏽 🚶🏽‍➡️ 🚶🏽‍♂️ 🚶🏽‍♂️‍➡️ 👩🏽‍🦯 👩🏽‍🦯‍➡️ 🧑🏽‍🦯 🧑🏽‍🦯‍➡️ 👨🏽‍🦯 👨🏽‍🦯‍➡️ 🧎🏽‍♀️ 🧎🏽‍♀️‍➡️ 🧎🏽 🧎🏽‍➡️ 🧎🏽‍♂️ 🧎🏽‍♂️‍➡️ 🏃🏽‍♀️ 🏃🏽‍♀️‍➡️ 🏃🏽 🏃🏽‍➡️ 🏃🏽‍♂️ 🏃🏽‍♂️‍➡️ 🧍🏽‍♀️ 🧍🏽 🧍🏽‍♂️ 👭🏽 🧑🏽‍🤝‍🧑🏽 👬🏽 👫🏽 🧗🏽‍♀️ 🧗🏽 🧗🏽‍♂️ 🏇🏽 🏂🏽 🏌🏽‍♀️ 🏌🏽 🏌🏽‍♂️ 🏄🏽‍♀️ 🏄🏽 🏄🏽‍♂️ 🚣🏽‍♀️ 🚣🏽 🚣🏽‍♂️ 🏊🏽‍♀️ 🏊🏽 🏊🏽‍♂️ ⛹🏽‍♀️ ⛹🏽 ⛹🏽‍♂️ 🏋🏽‍♀️ 🏋🏽 🏋🏽‍♂️ 🚴🏽‍♀️ 🚴🏽 🚴🏽‍♂️ 🚵🏽‍♀️ 🚵🏽 🚵🏽‍♂️ 🤸🏽‍♀️ 🤸🏽 🤸🏽‍♂️ 🤽🏽‍♀️ 🤽🏽 🤽🏽‍♂️ 🤾🏽‍♀️ 🤾🏽 🤾🏽‍♂️ 🤹🏽‍♀️ 🤹🏽 🤹🏽‍♂️ 🧘🏽‍♀️ 🧘🏽 🧘🏽‍♂️ 🛀🏽 🛌🏽 ')
print ('Dark Brown Emojis\n 👋🏾 🤚🏾 🖐🏾 ✋🏾 🖖🏾 👌🏾 🤌🏾 🤏🏾 ✌🏾 🤞🏾 🫰🏾 🤟🏾 🤘🏾 🤙🏾 🫵🏾 🫱🏾 🫲🏾 🫸🏾 🫷🏾 🫳🏾 🫴🏾 👈🏾 👉🏾 👆🏾 🖕🏾 👇🏾 ☝🏾 👍🏾 👎🏾 ✊🏾 👊🏾 🤛🏾 🤜🏾 👏🏾 🫶🏾 🙌🏾 👐🏾 🤲🏾 🙏🏾 ✍🏾 💅🏾 🤳🏾 💪🏾 🦵🏾 🦶🏾 👂🏾 🦻🏾 👃🏾 👶🏾 👧🏾 🧒🏾 👦🏾 👩🏾 🧑🏾 👨🏾 👩🏾‍🦱 🧑🏾‍🦱 👨🏾‍🦱 👩🏾‍🦰 🧑🏾‍🦰 👨🏾‍🦰 👱🏾‍♀️ 👱🏾 👱🏾‍♂️ 👩🏾‍🦳 🧑🏾‍🦳 👨🏾‍🦳 👩🏾‍🦲 🧑🏾‍🦲 👨🏾‍🦲 🧔🏾‍♀️ 🧔🏾 🧔🏾‍♂️ 👵🏾 🧓🏾 👴🏾 👲🏾 👳🏾‍♀️ 👳🏾 👳🏾‍♂️ 🧕🏾 👮🏾‍♀️ 👮🏾 👮🏾‍♂️ 👷🏾‍♀️ 👷🏾 👷🏾‍♂️ 💂🏾‍♀️ 💂🏾 💂🏾‍♂️ 🕵🏾‍♀️ 🕵🏾 🕵🏾‍♂️ 👩🏾‍⚕️ 🧑🏾‍⚕️ 👨🏾‍⚕️ 👩🏾‍🌾 🧑🏾‍🌾 👨🏾‍🌾 👩🏾‍🍳 🧑🏾‍🍳 👨🏾‍🍳 👩🏾‍🎓 🧑🏾‍🎓 👨🏾‍🎓 👩🏾‍🎤 🧑🏾‍🎤 👨🏾‍🎤 👩🏾‍🏫 🧑🏾‍🏫 👨🏾‍🏫 👩🏾‍🏭 🧑🏾‍🏭 👨🏾‍🏭 👩🏾‍💻 🧑🏾‍💻 👨🏾‍💻 👩🏾‍💼 🧑🏾‍💼 👨🏾‍💼 👩🏾‍🔧 🧑🏾‍🔧 👨🏾‍🔧 👩🏾‍🔬 🧑🏾‍🔬 👨🏾‍🔬 👩🏾‍🎨 🧑🏾‍🎨 👨🏾‍🎨 👩🏾‍🚒 🧑🏾‍🚒 👨🏾‍🚒 👩🏾‍✈️ 🧑🏾‍✈️ 👨🏾‍✈️ 👩🏾‍🚀 🧑🏾‍🚀 👨🏾‍🚀 👩🏾‍⚖️ 🧑🏾‍⚖️ 👨🏾‍⚖️ 👰🏾‍♀️ 👰🏾 👰🏾‍♂️ 🤵🏾‍♀️ 🤵🏾 🤵🏾‍♂️ 👸🏾 🫅🏾 🤴🏾 🥷🏾 🦸🏾‍♀️ 🦸🏾 🦸🏾‍♂️ 🦹🏾‍♀️ 🦹🏾 🦹🏾‍♂️ 🤶🏾 🧑🏾‍🎄 🎅🏾 🧙🏾‍♀️ 🧙🏾 🧙🏾‍♂️ 🧝🏾‍♀️ 🧝🏾 🧝🏾‍♂️ 🧛🏾‍♀️ 🧛🏾 🧛🏾‍♂️ 🧜🏾‍♀️ 🧜🏾 🧜🏾‍♂️ 🧚🏾‍♀️ 🧚🏾 🧚🏾‍♂️ 👼🏾 🤰🏾 🫄🏾 🫃🏾 🤱🏾 👩🏾‍🍼 🧑🏾‍🍼 👨🏾‍🍼 🙇🏾‍♀️ 🙇🏾 🙇🏾‍♂️ 💁🏾‍♀️ 💁🏾 💁🏾‍♂️ 🙅🏾‍♀️ 🙅🏾 🙅🏾‍♂️ 🙆🏾‍♀️ 🙆🏾 🙆🏾‍♂️ 🙋🏾‍♀️ 🙋🏾 🙋🏾‍♂️ 🧏🏾‍♀️ 🧏🏾 🧏🏾‍♂️ 🤦🏾‍♀️ 🤦🏾 🤦🏾‍♂️ 🤷🏾‍♀️ 🤷🏾 🤷🏾‍♂️ 🙎🏾‍♀️ 🙎🏾 🙎🏾‍♂️ 🙍🏾‍♀️ 🙍🏾 🙍🏾‍♂️ 💇🏾‍♀️ 💇🏾 💇🏾‍♂️ 💆🏾‍♀️ 💆🏾 💆🏾‍♂️ 🧖🏾‍♀️ 🧖🏾 🧖🏾‍♂️ 💃🏾 🕺🏾 🕴🏿 👩🏾‍🦽 👩🏾‍🦽‍➡️ 🧑🏾‍🦽 🧑🏾‍🦽‍➡️ 👨🏾‍🦽 👨🏾‍🦽‍➡️ 👩🏾‍🦼 👩🏾‍🦼‍➡️ 🧑🏾‍🦼 🧑🏾‍🦼‍➡️ 👨🏾‍🦼 👨🏾‍🦼‍➡️ 🚶🏾‍♀️ 🚶🏾‍♀️‍➡️ 🚶🏾 🚶🏾‍➡️ 🚶🏾‍♂️ 🚶🏾‍♂️‍➡️ 👩🏾‍🦯 👩🏾‍🦯‍➡️ 🧑🏾‍🦯 🧑🏾‍🦯‍➡️ 👨🏾‍🦯 👨🏾‍🦯‍➡️ 🧎🏾‍♀️ 🧎🏾‍♀️‍➡️ 🧎🏾 🧎🏾‍➡️ 🧎🏾‍♂️ 🧎🏾‍♂️‍➡️ 🏃🏾‍♀️ 🏃🏾‍♀️‍➡️ 🏃🏾 🏃🏾‍➡️ 🏃🏾‍♂️ 🏃🏾‍♂️‍➡️ 🧍🏾‍♀️ 🧍🏾 🧍🏾‍♂️ 👭🏾 🧑🏾‍🤝‍🧑🏾 👬🏾 👫🏾 🧗🏾‍♀️ 🧗🏾 🧗🏾‍♂️ 🏇🏾 🏂🏾 🏌🏾‍♀️ 🏌🏾 🏌🏾‍♂️ 🏄🏾‍♀️ 🏄🏾 🏄🏾‍♂️ 🚣🏾‍♀️ 🚣🏾 🚣🏾‍♂️ 🏊🏾‍♀️ 🏊🏾 🏊🏾‍♂️ ⛹🏾‍♀️ ⛹🏾 ⛹🏾‍♂️ 🏋🏾‍♀️ 🏋🏾 🏋🏾‍♂️ 🚴🏾‍♀️ 🚴🏾 🚴🏾‍♂️ 🚵🏾‍♀️ 🚵🏾 🚵🏾‍♂️ 🤸🏾‍♀️ 🤸🏾 🤸🏾‍♂️ 🤽🏾‍♀️ 🤽🏾 🤽🏾‍♂️ 🤾🏾‍♀️ 🤾🏾 🤾🏾‍♂️ 🤹🏾‍♀️ 🤹🏾 🤹🏾‍♂️ 🧘🏾‍♀️ 🧘🏾 🧘🏾‍♂️ 🛀🏾 🛌🏾 ')
print ('Black Emojis\n 👋🏿 🤚🏿 🖐🏿 ✋🏿 🖖🏿 👌🏿 🤌🏿 🤏🏿 ✌🏿 🤞🏿 🫰🏿 🤟🏿 🤘🏿 🤙🏿 🫵🏿 🫱🏿 🫲🏿 🫸🏿 🫷🏿 🫳🏿 🫴🏿 👈🏿 👉🏿 👆🏿 🖕🏿 👇🏿 ☝🏿 👍🏿 👎🏿 ✊🏿 👊🏿 🤛🏿 🤜🏿 👏🏿 🫶🏿 🙌🏿 👐🏿 🤲🏿 🙏🏿 ✍🏿 💅🏿 🤳🏿 💪🏿 🦵🏿 🦶🏿 👂🏿 🦻🏿 👃🏿 👶🏿 👧🏿 🧒🏿 👦🏿 👩🏿 🧑🏿 👨🏿 👩🏿‍🦱 🧑🏿‍🦱 👨🏿‍🦱 👩🏿‍🦰 🧑🏿‍🦰 👨🏿‍🦰 👱🏿‍♀️ 👱🏿 👱🏿‍♂️ 👩🏿‍🦳 🧑🏿‍🦳 👨🏿‍🦳 👩🏿‍🦲 🧑🏿‍🦲 👨🏿‍🦲 🧔🏿‍♀️ 🧔🏿 🧔🏿‍♂️ 👵🏿 🧓🏿 👴🏿 👲🏿 👳🏿‍♀️ 👳🏿 👳🏿‍♂️ 🧕🏿 👮🏿‍♀️ 👮🏿 👮🏿‍♂️ 👷🏿‍♀️ 👷🏿 👷🏿‍♂️ 💂🏿‍♀️ 💂🏿 💂🏿‍♂️ 🕵🏿‍♀️ 🕵🏿 🕵🏿‍♂️ 👩🏿‍⚕️ 🧑🏿‍⚕️ 👨🏿‍⚕️ 👩🏿‍🌾 🧑🏿‍🌾 👨🏿‍🌾 👩🏿‍🍳 🧑🏿‍🍳 👨🏿‍🍳 👩🏿‍🎓 🧑🏿‍🎓 👨🏿‍🎓 👩🏿‍🎤 🧑🏿‍🎤 👨🏿‍🎤 👩🏿‍🏫 🧑🏿‍🏫 👨🏿‍🏫 👩🏿‍🏭 🧑🏿‍🏭 👨🏿‍🏭 👩🏿‍💻 🧑🏿‍💻 👨🏿‍💻 👩🏿‍💼 🧑🏿‍💼 👨🏿‍💼 👩🏿‍🔧 🧑🏿‍🔧 👨🏿‍🔧 👩🏿‍🔬 🧑🏿‍🔬 👨🏿‍🔬 👩🏿‍🎨 🧑🏿‍🎨 👨🏿‍🎨 👩🏿‍🚒 🧑🏿‍🚒 👨🏿‍🚒 👩🏿‍✈️ 🧑🏿‍✈️ 👨🏿‍✈️ 👩🏿‍🚀 🧑🏿‍🚀 👨🏿‍🚀 👩🏿‍⚖️ 🧑🏿‍⚖️ 👨🏿‍⚖️ 👰🏿‍♀️ 👰🏿 👰🏿‍♂️ 🤵🏿‍♀️ 🤵🏿 🤵🏿‍♂️ 👸🏿 🫅🏿 🤴🏿 🥷🏿 🦸🏿‍♀️ 🦸🏿 🦸🏿‍♂️ 🦹🏿‍♀️ 🦹🏿 🦹🏿‍♂️ 🤶🏿 🧑🏿‍🎄 🎅🏿 🧙🏿‍♀️ 🧙🏿 🧙🏿‍♂️ 🧝🏿‍♀️ 🧝🏿 🧝🏿‍♂️ 🧛🏿‍♀️ 🧛🏿 🧛🏿‍♂️ 🧜🏿‍♀️ 🧜🏿 🧜🏿‍♂️ 🧚🏿‍♀️ 🧚🏿 🧚🏿‍♂️ 👼🏿 🤰🏿 🫄🏿 🫃🏿 🤱🏿 👩🏿‍🍼 🧑🏿‍🍼 👨🏿‍🍼 🙇🏿‍♀️ 🙇🏿 🙇🏿‍♂️ 💁🏿‍♀️ 💁🏿 💁🏿‍♂️ 🙅🏿‍♀️ 🙅🏿 🙅🏿‍♂️ 🙆🏿‍♀️ 🙆🏿 🙆🏿‍♂️ 🙋🏿‍♀️ 🙋🏿 🙋🏿‍♂️ 🧏🏿‍♀️ 🧏🏿 🧏🏿‍♂️ 🤦🏿‍♀️ 🤦🏿 🤦🏿‍♂️ 🤷🏿‍♀️ 🤷🏿 🤷🏿‍♂️ 🙎🏿‍♀️ 🙎🏿 🙎🏿‍♂️ 🙍🏿‍♀️ 🙍🏿 🙍🏿‍♂️ 💇🏿‍♀️ 💇🏿 💇🏿‍♂️ 💆🏿‍♀️ 💆🏿 💆🏿‍♂️ 🧖🏿‍♀️ 🧖🏿 🧖🏿‍♂️ 💃🏿 🕺🏿 🕴🏿 👩🏿‍🦽 👩🏿‍🦽‍➡️ 🧑🏿‍🦽 🧑🏿‍🦽‍➡️ 👨🏿‍🦽 👨🏿‍🦽‍➡️ 👩🏿‍🦼 👩🏿‍🦼‍➡️ 🧑🏿‍🦼 🧑🏿‍🦼‍➡️ 👨🏿‍🦼 👨🏿‍🦼‍➡️ 🚶🏿‍♀️ 🚶🏿‍♀️‍➡️ 🚶🏿 🚶🏿‍➡️ 🚶🏿‍♂️ 🚶🏿‍♂️‍➡️ 👩🏿‍🦯 👩🏿‍🦯‍➡️ 🧑🏿‍🦯 🧑🏿‍🦯‍➡️ 👨🏿‍🦯 👨🏿‍🦯‍➡️ 🧎🏿‍♀️ 🧎🏿‍♀️‍➡️ 🧎🏿 🧎🏿‍➡️ 🧎🏿‍♂️ 🧎🏿‍♂️‍➡️ 🏃🏿‍♀️ 🏃🏿‍♀️‍➡️ 🏃🏿 🏃🏿‍➡️ 🏃🏿‍♂️ 🏃🏿‍♂️‍➡️ 🧍🏿‍♀️ 🧍🏿 🧍🏿‍♂️ 👭🏿 🧑🏿‍🤝‍🧑🏿 👬🏿 👫🏿 🧗🏿‍♀️ 🧗🏿 🧗🏿‍♂️ 🏇🏿 🏂🏿 🏌🏿‍♀️ 🏌🏿 🏌🏿‍♂️ 🏄🏿‍♀️ 🏄🏿 🏄🏿‍♂️ 🚣🏿‍♀️ 🚣🏿 🚣🏿‍♂️ 🏊🏿‍♀️ 🏊🏿 🏊🏿‍♂️ ⛹🏿‍♀️ ⛹🏿 ⛹🏿‍♂️ 🏋🏿‍♀️ 🏋🏿 🏋🏿‍♂️ 🚴🏿‍♀️ 🚴🏿 🚴🏿‍♂️ 🚵🏿‍♀️ 🚵🏿 🚵🏿‍♂️ 🤸🏿‍♀️ 🤸🏿 🤸🏿‍♂️ 🤽🏿‍♀️ 🤽🏿 🤽🏿‍♂️ 🤾🏿‍♀️ 🤾🏿 🤾🏿‍♂️ 🤹🏿‍♀️ 🤹🏿 🤹🏿‍♂️ 🧘🏿‍♀️ 🧘🏿 🧘🏿‍♂️ 🛀🏿 🛌🏿 ')
print ('Animals & Nature\n 🐶 🐱 🐭 🐹 🐰 🦊 🐻 🐼 🐻‍❄️ 🐨 🐯 🦁 🐮 🐷 🐽 🐸 🐵 🙈 🙉 🙊 🐒 🐔 🐧 🐦 🐦‍⬛ 🐤 🐣 🐥 🦆 🦅 🦉 🦇 🐺 🐗 🐴 🦄 🐝 🪱 🐛 🦋 🐌 🐞 🐜 🪰 🪲 🪳 🦟 🦗 🕷 🕸 🦂 🐢 🐍 🦎 🦖 🦕 🐙 🦑 🦐 🦞 🦀 🪼 🪸 🐡 🐠 🐟 🐬 🐳 🐋 🦈 🐊 🐅 🐆 🦓 🫏 🦍 🦧 🦣 🐘 🦛 🦏 🐪 🐫 🦒 🦘 🦬 🐃 🐂 🐄 🐎 🐖 🐏 🐑 🦙 🐐 🦌 🫎 🐕 🐩 🦮 🐕‍🦺 🐈 🐈‍⬛ 🪽 🪶 🐓 🦃 🦤 🦚 🦜 🦢 🪿 🦩 🕊 🐇 🦝 🦨 🦡 🦫 🦦 🦥 🐁 🐀 🐿 🦔 🐾 🐉 🐲 🐦‍🔥 🌵 🎄 🌲 🌳 🌴 🪹 🪺 🪵 🌱 🌿 ☘️ 🍀 🎍 🪴 🎋 🍃 🍂 🍁 🍄 🍄‍🟫 🐚 🪨 🌾 💐 🌷 🪷 🌹 🥀 🌺 🌸 🪻 🌼 🌻 🌞 🌝 🌛 🌜 🌚 🌕 🌖 🌗 🌘 🌑 🌒 🌓 🌔 🌙 🌎 🌍 🌏 🪐 💫 ⭐️ 🌟 ✨ ⚡️ ☄️ 💥 🔥 🌪 🌈 ☀️ 🌤 ⛅️ 🌥 ☁️ 🌦 🌧 ⛈ 🌩 🌨 ❄️ ☃️ ⛄️ 🌬 💨 💧 💦 🫧 ☔️ ☂️ 🌊 ')
print ('Food & Drink\n 🍏 🍎 🍐 🍊 🍋 🍋‍🟩 🍌 🍉 🍇 🍓 🫐 🍈 🍒 🍑 🥭 🍍 🥥 🥝 🍅 🍆 🥑 🥦 🫛 🥬 🥒 🌶 🫑 🌽 🥕 🫒 🧄 🧅 🫚 🥔 🍠 🫘 🥐 🥯 🍞 🥖 🥨 🧀 🥚 🍳 🧈 🥞 🧇 🥓 🥩 🍗 🍖 🦴 🌭 🍔 🍟 🍕 🫓 🥪 🥙 🧆 🌮 🌯 🫔 🥗 🥘 🫕 🥫 🍝 🍜 🍲 🍛 🍣 🍱 🥟 🦪 🍤 🍙 🍚 🍘 🍥 🥠 🥮 🍢 🍡 🍧 🍨 🍦 🥧 🧁 🍰 🎂 🍮 🍭 🍬 🍫 🍿 🍩 🍪 🌰 🥜 🍯 🥛 🍼 🫖 ☕️ 🍵 🧃 🥤 🧋 🫙 🍶 🍺 🍻 🥂 🍷 🫗 🥃 🍸 🍹 🧉 🍾 🧊 🥄 🍴 🍽 🥣 🥡 🥢 🧂 ')
print ('Activity and Sports\n ⚽️ 🏀 🏈 ⚾️ 🥎 🎾 🏐 🏉 🥏 🎱 🪀 🏓 🏸 🏒 🏑 🥍 🏏 🪃 🥅 ⛳️ 🪁 🏹 🎣 🤿 🥊 🥋 🎽 🛹 🛼 🛷 ⛸ 🥌 🎿 ⛷ 🏂 🪂 🏋️‍♀️ 🏋️ 🏋️‍♂️ 🤼‍♀️ 🤼 🤼‍♂️ 🤸‍♀️ 🤸 🤸‍♂️ ⛹️‍♀️ ⛹️ ⛹️‍♂️ 🤺 🤾‍♀️ 🤾 🤾‍♂️ 🏌️‍♀️ 🏌️ 🏌️‍♂️ 🏇 🧘‍♀️ 🧘 🧘‍♂️ 🏄‍♀️ 🏄 🏄‍♂️ 🏊‍♀️ 🏊 🏊‍♂️ 🤽‍♀️ 🤽 🤽‍♂️ 🚣‍♀️ 🚣 🚣‍♂️ 🧗‍♀️ 🧗 🧗‍♂️ 🚵‍♀️ 🚵 🚵‍♂️ 🚴‍♀️ 🚴 🚴‍♂️ 🏆 🥇 🥈 🥉 🏅 🎖 🏵 🎗 🎫 🎟 🎪 🤹 🤹‍♂️ 🤹‍♀️ 🎭 🩰 🎨 🎬 🎤 🎧 🎼 🎹 🥁 🪘 🪇 🎷 🎺 🪗 🎸 🪕 🎻 🪈 🎲 ♟ 🎯 🎳 🎮 🎰 🧩 ')
print ('Travel & Places\n 🚗 🚕 🚙 🚌 🚎 🏎 🚓 🚑 🚒 🚐 🛻 🚚 🚛 🚜 🦯 🦽 🦼 🛴 🚲 🛵 🏍 🛺 🚨 🚔 🚍 🚘 🚖 🛞 🚡 🚠 🚟 🚃 🚋 🚞 🚝 🚄 🚅 🚈 🚂 🚆 🚇 🚊 🚉 ✈️ 🛫 🛬 🛩 💺 🛰 🚀 🛸 🚁 🛶 ⛵️ 🚤 🛥 🛳 ⛴ 🚢 ⚓️ 🛟 🪝 ⛽️ 🚧 🚦 🚥 🚏 🗺 🗿 🗽 🗼 🏰 🏯 🏟 🎡 🎢 🛝 🎠 ⛲️ ⛱ 🏖 🏝 🏜 🌋 ⛰ 🏔 🗻 🏕 ⛺️ 🛖 🏠 🏡 🏘 🏚 🏗 🏭 🏢 🏬 🏣 🏤 🏥 🏦 🏨 🏪 🏫 🏩 💒 🏛 ⛪️ 🕌 🕍 🛕 🕋 ⛩ 🛤 🛣 🗾 🎑 🏞 🌅 🌄 🌠 🎇 🎆 🌇 🌆 🏙 🌃 🌌 🌉 🌁 ')
print ('Objects\n ⌚️ 📱 📲 💻 ⌨️ 🖥 🖨 🖱 🖲 🕹 🗜 💽 💾 💿 📀 📼 📷 📸 📹 🎥 📽 🎞 📞 ☎️ 📟 📠 📺 📻 🎙 🎚 🎛 🧭 ⏱ ⏲ ⏰ 🕰 ⌛️ ⏳ 📡 🔋 🪫 🔌 💡 🔦 🕯 🪔 🧯 🛢 🛍️ 💸 💵 💴 💶 💷 🪙 💰 💳 💎 ⚖️ 🪮 🪜 🧰 🪛 🔧 🔨 ⚒ 🛠 ⛏ 🪚 🔩 ⚙️ 🪤 🧱 ⛓ ⛓️‍💥 🧲 🔫 💣 🧨 🪓 🔪 🗡 ⚔️ 🛡 🚬 ⚰️ 🪦 ⚱️ 🏺 🔮 📿 🧿 🪬 💈 ⚗️ 🔭 🔬 🕳 🩹 🩺 🩻 🩼 💊 💉 🩸 🧬 🦠 🧫 🧪 🌡 🧹 🪠 🧺 🧻 🚽 🚰 🚿 🛁 🛀 🧼 🪥 🪒 🧽 🪣 🧴 🛎 🔑 🗝 🚪 🪑 🛋 🛏 🛌 🧸 🪆 🖼 🪞 🪟 🛍 🛒 🎁 🎈 🎏 🎀 🪄 🪅 🎊 🎉 🪩 🎎 🏮 🎐 🧧 ✉️ 📩 📨 📧 💌 📥 📤 📦 🏷 🪧 📪 📫 📬 📭 📮 📯 📜 📃 📄 📑 🧾 📊 📈 📉 🗒 🗓 📆 📅 🗑 🪪 📇 🗃 🗳 🗄 📋 📁 📂 🗂 🗞 📰 📓 📔 📒 📕 📗 📘 📙 📚 📖 🔖 🧷 🔗 📎 🖇 📐 📏 🧮 📌 📍 ✂️ 🖊 🖋 ✒️ 🖌 🖍 📝 ✏️ 🔍 🔎 🔏 🔐 🔒 🔓 ')
print ('Symbols\n ❤️ 🩷 🧡 💛 💚 💙 🩵 💜 🖤 🩶 🤍 🤎 ❤️‍🔥 ❤️‍🩹 💔 ❣️ 💕 💞 💓 💗 💖 💘 💝 💟 ☮️ ✝️ ☪️ 🪯 🕉 ☸️ ✡️ 🔯 🕎 ☯️ ☦️ 🛐 ⛎ ♈️ ♉️ ♊️ ♋️ ♌️ ♍️ ♎️ ♏️ ♐️ ♑️ ♒️ ♓️ 🆔 ⚛️ 🉑 ☢️ ☣️ 📴 📳 🈶 🈚️ 🈸 🈺 🈷️ ✴️ 🆚 💮 🉐 ㊙️ ㊗️ 🈴 🈵 🈹 🈲 🅰️ 🅱️ 🆎 🆑 🅾️ 🆘 ❌ ⭕️ 🛑 ⛔️ 📛 🚫 💯 💢 ♨️ 🚷 🚯 🚳 🚱 🔞 📵 🚭 ❗️ ❕ ❓ ❔ ‼️ ⁉️ 🔅 🔆 〽️ ⚠️ 🚸 🔱 ⚜️ 🔰 ♻️ ✅ 🈯️ 💹 ❇️ ✳️ ❎ 🌐 💠 Ⓜ️ 🌀 💤 🏧 🚾 ♿️ 🅿️ 🛗 🈳 🈂️ 🛂 🛃 🛄 🛅 🚹 🚺 🚼 ⚧ 🚻 🚮 🎦 🛜 📶 🈁 🔣 ℹ️ 🔤 🔡 🔠 🆖 🆗 🆙 🆒 🆕 🆓 0️⃣ 1️⃣ 2️⃣ 3️⃣ 4️⃣ 5️⃣ 6️⃣ 7️⃣ 8️⃣ 9️⃣ 🔟 🔢 *️⃣ ⏏️ ▶️ ⏸ ⏯ ⏹ ⏺ ⏭ ⏮ ⏩ ⏪ ⏫ ⏬ ◀️ 🔼 🔽 ➡️ ⬅️ ⬆️ ⬇️ ↗️ ↘️ ↙️ ↖️ ↕️ ↔️ ↪️ ↩️ ⤴️ ⤵️ 🔀 🔁 🔂 🔄 🔃 🎵 🎶 ➕ ➖ ➗ ✖️ 🟰 ♾ 💲 💱 ™️ ©️ ®️ 〰️ ➰ ➿ 🔚 🔙 🔛 🔝 🔜 ✔️ ☑️ 🔘 🔴 🟠 🟡 🟢 🔵 🟣 ⚫️ ⚪️ 🟤 🔺 🔻 🔸 🔹 🔶 🔷 🔳 🔲 ▪️ ▫️ ◾️ ◽️ ◼️ ◻️ 🟥 🟧 🟨 🟩 🟦 🟪 ⬛️ ⬜️ 🟫 🔈 🔇 🔉 🔊 🔔 🔕 📣 📢 👁‍🗨 💬 💭 🗯 ♠️ ♣️ ♥️ ♦️ 🃏 🎴 🀄️ 🕐 🕑 🕒 🕓 🕔 🕕 🕖 🕗 🕘 🕙 🕚 🕛 🕜 🕝 🕞 🕟 🕠 🕡 🕢 🕣 🕤 🕥 🕦 🕧 ')
print ('Non-Emoji Symbols More Unicode symbols, Hieroglpyhs and Pictographs\n ✢ ✣ ✤ ✥ ✦ ✧ ★ ☆ ✯ ✡︎ ✩ ✪ ✫ ✬ ✭ ✮ ✶ ✷ ✵ ✸ ✹ → ⇒ ⟹ ⇨ ⇾ ➾ ⇢ ☛ ☞ ➔ ➜ ➙ ➛ ➝ ➞ ♠︎ ♣︎ ♥︎ ♦︎ ♤ ♧ ♡ ♢ ♚ ♛ ♜ ♝ ♞ ♟ ♔ ♕ ♖ ♗ ♘ ♙ ⚀ ⚁ ⚂ ⚃ ⚄ ⚅ 🂠 ⚈ ⚉ ⚆ ⚇ 𓀀 𓀁 𓀂 𓀃 𓀄 𓀅 𓀆 𓀇 𓀈 𓀉 𓀊 𓀋 𓀌 𓀍 𓀎 𓀏 𓀐 𓀑 𓀒 𓀓 𓀔 𓀕 𓀖 𓀗 𓀘 𓀙 𓀚 𓀛 𓀜 𓀝 ')
print ('Flags\n 🏳️ 🏴 🏁 🚩 🏳️‍🌈 🏳️‍⚧️ 🏴‍☠️ 🇦🇫 🇦🇽 🇦🇱 🇩🇿 🇦🇸 🇦🇩 🇦🇴 🇦🇮 🇦🇶 🇦🇬 🇦🇷 🇦🇲 🇦🇼 🇦🇺 🇦🇹 🇦🇿 🇧🇸 🇧🇭 🇧🇩 🇧🇧 🇧🇾 🇧🇪 🇧🇿 🇧🇯 🇧🇲 🇧🇹 🇧🇴 🇧🇦 🇧🇼 🇧🇷 🇮🇴 🇻🇬 🇧🇳 🇧🇬 🇧🇫 🇧🇮 🇰🇭 🇨🇲 🇨🇦 🇮🇨 🇨🇻 🇧🇶 🇰🇾 🇨🇫 🇹🇩 🇨🇱 🇨🇳 🇨🇽 🇨🇨 🇨🇴 🇰🇲 🇨🇬 🇨🇩 🇨🇰 🇨🇷 🇨🇮 🇭🇷 🇨🇺 🇨🇼 🇨🇾 🇨🇿 🇩🇰 🇩🇯 🇩🇲 🇩🇴 🇪🇨 🇪🇬 🇸🇻 🇬🇶 🇪🇷 🇪🇪 🇪🇹 🇪🇺 🇫🇰 🇫🇴 🇫🇯 🇫🇮 🇫🇷 🇬🇫 🇵🇫 🇹🇫 🇬🇦 🇬🇲 🇬🇪 🇩🇪 🇬🇭 🇬🇮 🇬🇷 🇬🇱 🇬🇩 🇬🇵 🇬🇺 🇬🇹 🇬🇬 🇬🇳 🇬🇼 🇬🇾 🇭🇹 🇭🇳 🇭🇰 🇭🇺 🇮🇸 🇮🇳 🇮🇩 🇮🇷 🇮🇶 🇮🇪 🇮🇲 🇮🇱 🇮🇹 🇯🇲 🇯🇵 🎌 🇯🇪 🇯🇴 🇰🇿 🇰🇪 🇰🇮 🇽🇰 🇰🇼 🇰🇬 🇱🇦 🇱🇻 🇱🇧 🇱🇸 🇱🇷 🇱🇾 🇱🇮 🇱🇹 🇱🇺 🇲🇴 🇲🇰 🇲🇬 🇲🇼 🇲🇾 🇲🇻 🇲🇱 🇲🇹 🇲🇭 🇲🇶 🇲🇷 🇲🇺 🇾🇹 🇲🇽 🇫🇲 🇲🇩 🇲🇨 🇲🇳 🇲🇪 🇲🇸 🇲🇦 🇲🇿 🇲🇲 🇳🇦 🇳🇷 🇳🇵 🇳🇱 🇳🇨 🇳🇿 🇳🇮 🇳🇪 🇳🇬 🇳🇺 🇳🇫 🇰🇵 🇲🇵 🇳🇴 🇴🇲 🇵🇰 🇵🇼 🇵🇸 🇵🇦 🇵🇬 🇵🇾 🇵🇪 🇵🇭 🇵🇳 🇵🇱 🇵🇹 🇵🇷 🇶🇦 🇷🇪 🇷🇴 🇷🇺 🇷🇼 🇼🇸 🇸🇲 🇸🇦 🇸🇳 🇷🇸 🇸🇨 🇸🇱 🇸🇬 🇸🇽 🇸🇰 🇸🇮 🇬🇸 🇸🇧 🇸🇴 🇿🇦 🇰🇷 🇸🇸 🇪🇸 🇱🇰 🇧🇱 🇸🇭 🇰🇳 🇱🇨 🇵🇲 🇻🇨 🇸🇩 🇸🇷 🇸🇿 🇸🇪 🇨🇭 🇸🇾 🇹🇼 🇹🇯 🇹🇿 🇹🇭 🇹🇱 🇹🇬 🇹🇰 🇹🇴 🇹🇹 🇹🇳 🇹🇷 🇹🇲 🇹🇨 🇹🇻 🇻🇮 🇺🇬 🇺🇦 🇦🇪 🇬🇧 🏴󠁧󠁢󠁥󠁮󠁧󠁿 🏴󠁧󠁢󠁳󠁣󠁴󠁿 🏴󠁧󠁢󠁷󠁬󠁳󠁿 🇺🇳 🇺🇸 🇺🇾 🇺🇿 🇻🇺 🇻🇦 🇻🇪 🇻🇳 🇼🇫 🇪🇭 🇾🇪 🇿🇲 🇿🇼')


#_sp: youtube

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**youtube**</font><br>
from IPython.display import YouTubeVideo

jukebox = {"Kishi Bashi - Violin Tsunami"                       :"xlXwpaAVoJQ",
           "Fever Ray in Passengers - ARTE Concert"             :"mTyi42VYITw",
           "Black Pumas - Colors (Live in KUTX Studio 1A)"      :"ay6MLO2pd30",
           "Tash Sultana - Jungle"                              :"1ExkpBpYEPw"}

select = "Fever Ray in Passengers - ARTE Concert" #@param ["Fever Ray in Passengers - ARTE Concert","Kishi Bashi - Violin Tsunami","Tash Sultana - Jungle", "Black Pumas - Colors (Live in KUTX Studio 1A)"]

display ( YouTubeVideo ( jukebox [ select ] ) )

#_sp: radio buttons

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**radio buttons**</font><br>
import ipywidgets as widgets
import numpy


output_radio_selected = widgets.Text()
radio1 = widgets.RadioButtons(options=['mse'])
radio2 = widgets.RadioButtons(options=['mae'])

radio1.index = None
radio2.index = None

def radio1_observer(sender):
    #print(sender)
    radio2.unobserve(radio2_observer, names=['value'])
    radio2.index = None

    global selected_option
    output_radio_selected.value = radio1.value
    selected_option = output_radio_selected.value
    print('Selected option set to: ' + selected_option)

    radio2.observe(radio2_observer, names=['value'])

def radio2_observer(sender):
    radio1.unobserve(radio1_observer, names=['value'])
    radio1.index = None

    global selected_option2
    output_radio_selected.value = radio2.value
    selected_option2 = output_radio_selected.value
    print('Selected option set to: ' + selected_option2)

    radio1.observe(radio1_observer, names=['value'])


radio1.observe(radio1_observer, names=['value'])
radio2.observe(radio2_observer, names=['value'])

widgets.HBox([radio1,radio2])

#_sp: svg with graphviz

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**svg with graphviz**</font><br>
import graphviz
s='init->predict->loss->gradient->step->stop\nstep->predict[label=repeat]'
graphviz.Source('digraph G{ rankdir="LR"' + s + '; }')

#_sp: gdrive links


In [ ]:
#@markdown <font size='+2' color='#005F6A'>**gdrive links / hosting**</font><br>
#@markdown In gdrive get via context menu the share link. It looks like:
#@markdown > ```https://drive.google.com/file/d/1cehK_***************************/view?usp=sharing```

#@markdown Change it to the following format:
#@markdown > ```https://drive.google.com/uc?id=1cehK_***************************```

#@markdown you can use it in an image tag: ```<img src='your link'>```

print ('* link prefix: https://drive.google.com/uc?id=')
print ("* html: <img src='https://drive.google.com/uc?id=XXX' width='400' alt='alt text'>")
print ('* markdown: ![alt text](your link)')






# _sp: tex and greek letters

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**tex and greek letters**</font><br>
#@markdown You get this: &emsp;&emsp; $\large \sum {\frac {Frit^z} {\sqrt[the] {C\alpha t}} }$<br> with that:&emsp;&emsp;&emsp;&ensp; ```$\large \sum {\frac {Frit^z} {\sqrt[the] {C\alpha t}} }$```
#
#@markdown ----
#@markdown <table><tr><td>Alpha $\alpha$ / $A$</td><td>Beta $\beta$ / $B$</td><td>Gamma $\gamma$ / $\Gamma$</td><td>Delta $\delta$ / $\Delta$</td><td>Epsilon $\epsilon$ / $E$</td><td>Zeta $\zeta$ / $Z$</td></tr>
#@markdown <tr><td>Eta $\eta$ / $E$</td><td>Theta $\theta$ / $\Theta$</td><td>Iota $\iota$ / $I$</td><td>Kappa $\kappa$ / $K$</td><td>Lambda $\lambda$ / $\Lambda$</td><td>My $\mu$ / $M$</td></tr>
#@markdown <tr><td>Ny $\nu$ / $N$</td><td>Xi $\xi$ / $\Xi$</td><td>Omikron $\omicron$ / $O$</td><td>Pi $\pi$ / $\Pi$</td><td>Rho $\rho$ / $R$</td><td>Sigma $\sigma$ / $\Sigma$</td></tr>
#@markdown <tr><td>Tau $\tau$ / $T$</td><td>Ypsilon $\upsilon$ / $\Upsilon$</td><td>Phi $\phi$ / $\Phi$</td><td>Chi $\chi$ / $X$</td><td>Psi $\psi$ / $\Psi$</td><td>Omega $\omega$ / $\Omega$</td></tr></table>

for i in range (0x03b1, 0x03c7): print (chr(i), end=' ')
print()
for i in range (0x0391, 0x03a7): print (chr(i), end=' ')

from IPython.display import display, Math
display(Math(r'$\large \sum {\frac {Frit^z} {\sqrt[the] {C\alpha t}} }$'))


#_sp: text cell snippets

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**text cell snippets - a little hack**</font><br>
#@markdown * run the cell and copy the output in a colab text cell and you'll get something like this:
#@markdown <table><tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr><tr><td>r2d1</td><td colspan=2 rowspan=2 align=center>
#@markdown $$\rule{7cm}{0.4pt}$$ $\left.{\;\\\;\\}\right\vert$ r2d2 - colspan2 - rowspan2$\left.{\;\\\;\\}\right\vert$<br>$$\rule{7cm}{0.4pt}$$
#@markdown </td><td>r2d3</td></tr><tr><td>r3d1</td><td>r3d2</td></tr><tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr></table>
from ipywidgets import Text, Textarea, Layout
Textarea (description='copy', layout=Layout(width='auto',height='800px'), value="""
**markdown**

|x|y|
|-||
|a|b|

**tables and lines**

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td></tr>
<tr><td>r2d1</td><td>r2d2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td><td>r3d3</td></tr>
</table>

---
<table>
<tr><td colspan=3>r1d1 colspan3</td></tr>
<tr><td>r2d1</td><td>r2d2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td><td>r3d3</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td rowspan=3>r1d2<br>rowspan3</td><td>r1d3</td></tr>
<tr><td>r2d1</td><td>r2d2</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr>
<tr><td>r2d1</td><td colspan=2 rowspan=2>r2d2<br>colspan2<br>rowspan2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
<tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr>
</table>

---
I - inner table / O - outer table
<table>
<tr><td>O r1d1</td><td>O r1d2</td><td>O r1d3</td><td>O r1d4</td></tr>
<tr><td>O r2d1</td><td colspan=2 rowspan=2 align=center>O r2d2  colspan2 rowspan2
<table>
<tr><td>I r1d1</td><td>I r1d2</td><td>I r1d3</td><td>I r1d4</td></tr>
<tr><td>I r2d1</td><td colspan=2 rowspan=2 align=center>
I r2d2<br>colspan2<br>rowspan2
</td><td>I r2d3</td></tr>
<tr><td>I r3d1</td><td>I r3d2</td></tr>
<tr><td>I r4d1</td><td>I r4d2</td><td>I r4d3</td><td>I r4d4</td></tr>
</table>
</td><td>O r2d3</td></tr>
<tr><td>O r3d1</td><td>O r3d2</td></tr>
<tr><td>O r4d1</td><td>O r4d2</td><td>O r4d3</td><td>O r4d4</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr>
<tr><td>r2d1</td><td colspan=2 rowspan=2 align=center>
$$\\rule{7cm}{0.4pt}$$
$\\left.{\\\\;\\\\\\\\\\\\;\\\\\\\\\\;\\\\\\;\\\\}\\right\\vert$
r2d2 - colspan2 - rowspan2$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$<br>
$$\\rule{7cm}{0.4pt}$$
</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
<tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr>
</table>

---
<table><tr><td rowspan=4>
$\\left.{\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$

</td><td colspan=2>$\\rule{8cm}{0.4pt}$</td><td rowspan=4>
$\\left.{\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$

</td></tr >
<tr><td  align=center>r2d1</td><td  align=center>r2d2</td></tr>
<tr><td  align=center>r3d1</td><td  align=center>r3d2</td></tr>
<tr><td colspan=2>
$\\rule{8cm}{0.4pt}$

</td></tr></table>

---
<table><tr><td>

$\\left.{\\;}\\right\\vert$</td><td>

$\\left.{\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td align=center>

$\\rule{8cm}{0.4pt}$<br>
$\\rule{10cm}{1pt}$<br>
$\\rule{12cm}{4pt}$<br>
$\\rule{10cm}{1pt}$<br>
$\\rule{8cm}{0.4pt}$<br>

</td></tr></table>

""")


# _sp: vectorize

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**vectorize**</font><br>
#@markdown <img src='https://drive.google.com/uc?id=1cHGRQjBYpeXUzEGo53yYfg7mSYjJDjTh' width='400' alt='vectorize.png'><br>
#@markdown ```python
#@markdown np.vectorize (lambda w: loss(nn(x,w), t) ) (ws)
#@markdown ```
import numpy as np

np.random.seed(seed=42)

def f(x)          : return x * 2
def nn(x, w)      : return x * w
def loss(y, t)    : return np.mean((t - y)**2)

x                 = np.random.uniform (0, 1, 20)          # 20 values between 0 and 1 uniform distributed
noise             = np.random.randn (x.shape[0]) * 0.2    # 20 standard normal distribution
t                 = f(x) + noise                          # noisy result (y)
ws                = np.linspace(0, 4, num=100)            # weight vector to vectorize

result = np.vectorize (lambda w: loss(nn(x,w), t) ) (ws)  # for each element in vector ws calculate via nn the y - and with that and t the loss
                                                          # we calculate a loss vector for the passed vector (ws) related to an x and t
result.shape, result

